# **Introduction**

This is an intial Explanatory Data Analysis for the [Mercari Price Suggestion Challenge](https://www.kaggle.com/c/mercari-price-suggestion-challenge#description) with matplotlib and Plot.ly - a visualization tool that creates beautiful interactive plots and dashboards.  The competition is hosted by Mercari, the biggest Japanese community-powered shopping app with the main objective to predict an accurate price that Mercari should suggest to its sellers, given the item's information. 

***Update***: The abundant amount of food from my family's Thanksgiving dinner has really energized me to continue working on this model. I decided to dive deeper into the NLP analysis and found an amazing tutorial by Ahmed BESBES. The framework below is  based on his source code at: https://ahmedbesbes.com/how-to-mine-newsfeed-data-and-extract-interactive-insights-in-python.html. It provides guidance on pre-processing documents and  machine learning techniques (K-means and LDA) to clustering topics.  So that this kernel will be divided into 2 parts: 

1. Explanatory Data Analysis 
2. Text Processing  
    2.1. Pre-processing: Tokenizing and  tf-idf algorithm  
    2.2. K-means Clustering  
    2.3. Latent Dirichlet Allocation (LDA)  
 

In [2]:
import nltk
import string
import re
import numpy as np
import pandas as pd
import pickle

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="white")

from nltk.stem.porter import *
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction import stop_words

from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
import lda
from sklearn.feature_extraction.text import CountVectorizer
import logging
logging.getLogger("lda").setLevel(logging.WARNING)

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
%matplotlib inline

import bokeh.plotting as bp
from bokeh.models import HoverTool, BoxSelectTool
from bokeh.plotting import figure, show, output_notebook
#from bokeh.transform import factor_cmap


import warnings
warnings.filterwarnings('ignore')

# **Exploratory Data Analysis**
On the first look at the data, besides the unique identifier (item_id), there are 7 variables in this model. This notebook will sequentially go through each of them with a brief statistical summary. 

1. **Numerical/Continuous Features**
    1. price: the item's final bidding price. This will be our reponse / independent variable that we need to predict in the test set
    2. shipping cost     
 
1. **Categorical Features**: 
    1. shipping cost: A binary indicator, 1 if shipping fee is paid by seller and 0 if it's paid by buyer
    2. item_condition_id: The condition of the items provided by the seller
    1. name: The item's name
    2. brand_name: The item's producer brand name
    2. category_name: The item's single or multiple categories that are separated by "\" 
    3. item_description: A short description on the item that may include removed words, flagged by [rm]

In [ ]:
PATH = "../input/"

In [ ]:
train = pd.read_csv(f'{PATH}train.tsv', sep='\t')
test = pd.read_csv(f'{PATH}test.tsv', sep='\t')

In [ ]:
# size of training and dataset
print(train.shape)
print(test.shape)

In [ ]:
# different data types in the dataset: categorical (strings) and numeric
train.dtypes

In [ ]:
train.head()

## Target Variable: **Price**

The next standard check is with our response or target variables, which in this case is the `price` we are suggesting to the Mercari's marketplace sellers.  The median price of all the items in the training is about \$267 but given the existence of some extreme values of over \$100 and the maximum at \$2,009, the distribution of the variables is heavily skewed to the left. So let's make log-transformation on the price (we added +1 to the value before the transformation to avoid zero and negative values).

In [ ]:
train.price.describe()

In [ ]:
plt.subplot(1, 2, 1)
(train['price']).plot.hist(bins=50, figsize=(20,10), edgecolor='white',range=[0,250])
plt.xlabel('price+', fontsize=17)
plt.ylabel('frequency', fontsize=17)
plt.tick_params(labelsize=15)
plt.title('Price Distribution - Training Set', fontsize=17)

plt.subplot(1, 2, 2)
np.log(train['price']+1).plot.hist(bins=50, figsize=(20,10), edgecolor='white')
plt.xlabel('log(price+1)', fontsize=17)
plt.ylabel('frequency', fontsize=17)
plt.tick_params(labelsize=15)
plt.title('Log(Price) Distribution - Training Set', fontsize=17)
plt.show()

## **Shipping**

The shipping cost burden is decently splitted between sellers and buyers with more than half of the items' shipping fees are paid by the sellers (55%). In addition, the average price paid by users who have to pay for shipping fees is lower than those that don't require additional shipping cost. This matches with our perception that the sellers need a lower price to compensate for the additional shipping.

In [ ]:
train.shipping.value_counts()/len(train)

In [ ]:
prc_shipBySeller = train.loc[train.shipping==0, 'price']
prc_shipByBuyer = train.loc[train.shipping==1, 'price']

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
ax.hist(np.log(prc_shipBySeller+1), color='#8CB4E1', alpha=1.0, bins=50,
       label='Price when Seller pays Shipping')
ax.hist(np.log(prc_shipByBuyer+1), color='#007D00', alpha=0.7, bins=50,
       label='Price when Buyer pays Shipping')
ax.set(title='Histogram Comparison', ylabel='% of Dataset in Bin')
plt.xlabel('log(price+1)', fontsize=17)
plt.ylabel('frequency', fontsize=17)
plt.title('Price Distribution by Shipping Type', fontsize=17)
plt.tick_params(labelsize=15)
plt.show()

## **Item Category**

There are about **1,287** unique categories but among each of them, we will always see a main/general category firstly, followed by two more particular subcategories (e.g. Beauty/Makeup/Face or Lips). In adidition, there are about 6,327 items that do not have a category labels. Let's split the categories into three different columns. We will see later that this information is actually quite important from the seller's point of view and how we handle the missing information in the `brand_name` column will impact the model's prediction. 

In [ ]:
print("There are %d unique values in the category column." % train['category_name'].nunique())

In [ ]:
# TOP 5 RAW CATEGORIES
train['category_name'].value_counts()[:5]

In [ ]:
# missing categories
print("There are %d items that do not have a label." % train['category_name'].isnull().sum())

In [ ]:
# reference: BuryBuryZymon at https://www.kaggle.com/maheshdadhich/i-will-sell-everything-for-free-0-55
def split_cat(text):
    try: return text.split("/")
    except: return ("No Label", "No Label", "No Label")

In [ ]:
train['general_cat'], train['subcat_1'], train['subcat_2'] = \
zip(*train['category_name'].apply(lambda x: split_cat(x)))
train.head()

In [ ]:
# repeat the same step for the test set
test['general_cat'], test['subcat_1'], test['subcat_2'] = \
zip(*test['category_name'].apply(lambda x: split_cat(x)))

In [ ]:
print("There are %d unique first sub-categories." % train['subcat_1'].nunique())

In [ ]:
print("There are %d unique second sub-categories." % train['subcat_2'].nunique())

Overall, we have  **7 main categories** (114 in the first sub-categories and 871 second sub-categories): women's and beauty items as the two most popular categories (more than 50% of the observations), followed by kids and electronics. 

In [ ]:
x = train['general_cat'].value_counts().index.values.astype('str')
y = train['general_cat'].value_counts().values
pct = [("%.2f"%(v*100))+"%"for v in (y/len(train))]

In [ ]:
trace1 = go.Bar(x=x, y=y, text=pct)
layout = dict(title= 'Number of Items by Main Category',
              yaxis = dict(title='Count'),
              xaxis = dict(title='Category'))
fig=dict(data=[trace1], layout=layout)
py.iplot(fig)

In [ ]:
x = train['subcat_1'].value_counts().index.values.astype('str')[:15]
y = train['subcat_2'].value_counts().values[:15]
pct = [("%.2f"%(v*100))+"%"for v in (y/len(train))][:15]

In [ ]:
trace1 = go.Bar(x=x, y=y, text=pct,
                marker=dict(
                color = y,colorscale='Portland',showscale=True,
                reversescale = False
                ))
layout = dict(title= 'Number of Items by SubCategory (Top 15)',
              yaxis = dict(title='Count'),
              xaxis = dict(title='SubCategory'))
fig=dict(data=[trace1], layout=layout)
py.iplot(fig)

From the pricing (log of price) point of view, all the categories are pretty well distributed, with no category with an extraordinary pricing point 

In [ ]:
general_cats = train['general_cat'].unique()
x = [train.loc[train['general_cat']==cat, 'price'] for cat in general_cats]

In [ ]:
data = [go.Box(x=np.log(x[i]+1), name=general_cats[i]) for i in range(len(general_cats))]

In [ ]:
layout = dict(title="Price Distribution by General Category",
              yaxis = dict(title='Frequency'),
              xaxis = dict(title='Category'))
fig = dict(data=data, layout=layout)
py.iplot(fig)

## **Brand Name**

In [ ]:
print("There are %d unique brand names in the training dataset." % train['brand_name'].nunique())

In [ ]:
# x = train['brand_name'].value_counts().index.values.astype('str')[:10]
# y = train['brand_name'].value_counts().values[:10]

In [ ]:
# trace1 = go.Bar(x=x, y=y, 
#                 marker=dict(
#                 color = y,colorscale='Portland',showscale=True,
#                 reversescale = False
#                 ))
# layout = dict(title= 'Top 10 Brand by Number of Items',
#               yaxis = dict(title='Brand Name'),
#               xaxis = dict(title='Count'))
# fig=dict(data=[trace1], layout=layout)
# py.iplot(fig)

## **Item Description**

It will be more challenging to parse through this particular item since it's unstructured data. Does it mean a more detailed and lengthy description will result in a higher bidding price? We will strip out all punctuations, remove some english stop words (i.e. redundant words such as "a", "the", etc.) and any other words with a length less than 3: 

In [ ]:
def wordCount(text):
    # convert to lower case and strip regex
    try:
         # convert to lower case and strip regex
        text = text.lower()
        regex = re.compile('[' +re.escape(string.punctuation) + '0-9\\r\\t\\n]')
        txt = regex.sub(" ", text)
        # tokenize
        # words = nltk.word_tokenize(clean_txt)
        # remove words in stop words
        words = [w for w in txt.split(" ") \
                 if not w in stop_words.ENGLISH_STOP_WORDS and len(w)>3]
        return len(words)
    except: 
        return 0

In [ ]:
# add a column of word counts to both the training and test set
train['desc_len'] = train['item_description'].apply(lambda x: wordCount(x))
test['desc_len'] = test['item_description'].apply(lambda x: wordCount(x))

In [ ]:
train.head()

In [ ]:
df = train.groupby('desc_len')['price'].mean().reset_index()

In [ ]:
trace1 = go.Scatter(
    x = df['desc_len'],
    y = np.log(df['price']+1),
    mode = 'lines+markers',
    name = 'lines+markers'
)
layout = dict(title= 'Average Log(Price) by Description Length',
              yaxis = dict(title='Average Log(Price)'),
              xaxis = dict(title='Description Length'))
fig=dict(data=[trace1], layout=layout)
py.iplot(fig)

We also need to check if there are any missing values in the item description (4 observations don't have a description) andl remove those observations from our training set.

In [ ]:
train.item_description.isnull().sum()

In [ ]:
# remove missing values in item description
train = train[pd.notnull(train['item_description'])]

In [ ]:
# create a dictionary of words for each category
# cat_desc = dict()
# for cat in general_cats: 
#     text = " ".join(train.loc[train['general_cat']==cat, 'item_description'].values)
#     cat_desc[cat] = tokenize(text)

In [ ]:
# # flat list of all words combined
# flat_lst = [item for sublist in list(cat_desc.values()) for item in sublist]
# allWordsCount = Counter(flat_lst)
# all_top10 = allWordsCount.most_common(20)
# x = [w[0] for w in all_top10]
# y = [w[1] for w in all_top10]

In [ ]:
# trace1 = go.Bar(x=x, y=y, text=pct)
# layout = dict(title= 'Word Frequency',
#               yaxis = dict(title='Count'),
#               xaxis = dict(title='Word'))
# fig=dict(data=[trace1], layout=layout)
# py.iplot(fig)

If we look at the most common words by category, we could also see that, ***size***, ***free*** and ***shipping*** is very commonly used by the sellers, probably with the intention to attract customers, which is contradictory to what  we have shown previously that there is little correlation between the two variables `price` and `shipping` (or shipping fees do not account for a differentiation in prices). ***Brand names*** also played quite an important role - it's one of the most popular in all four categories.  

# **Text Processing - Item Description**

## **Pre-processing:  tokenization**

Most of the time, the first steps of an NLP project is to **"tokenize"** your documents, which main purpose is to normalize our texts. The three fundamental stages will usually include: 
* break the descriptions into sentences and then break the sentences into tokens
* remove punctuation and stop words
* lowercase the tokens
* herein, I will also only consider words that have length equal to or greater than 3 characters

In [4]:
stop = set(stopwords.words('english'))
def tokenize(text):
    """
    sent_tokenize(): segment text into sentences
    word_tokenize(): break sentences into words
    """
    try: 
        regex = re.compile('[' +re.escape(string.punctuation) + '0-9\\r\\t\\n]')
        text = regex.sub(" ", text) # remove punctuation
        
        tokens_ = [word_tokenize(s) for s in sent_tokenize(text)]
        tokens = []
        for token_by_sent in tokens_:
            tokens += token_by_sent
        tokens = list(filter(lambda t: t.lower() not in stop, tokens))
        filtered_tokens = [w for w in tokens if re.search('[a-zA-Z]', w)]
        filtered_tokens = [w.lower() for w in filtered_tokens if len(w)>=3]
        
        return filtered_tokens
            
    except TypeError as e: print(text,e)

In [ ]:
# apply the tokenizer into the item descriptipn column
train['tokens'] = train['item_description'].map(tokenize)
test['tokens'] = test['item_description'].map(tokenize)

In [ ]:
train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

Let's look at the examples of if the tokenizer did a good job in cleaning up our descriptions

In [ ]:
for description, tokens in zip(train['item_description'].head(),
                              train['tokens'].head()):
    print('description:', description)
    print('tokens:', tokens)
    print()

In [ ]:
def generate_wordcloud(tup):
    wordcloud = WordCloud(background_color='white',
                          max_words=50, max_font_size=40,
                          random_state=42
                         ).generate(str(tup))
    return wordcloud

In [ ]:
# build dictionary with key=category and values as all the descriptions related.
cat_desc = dict()
for cat in general_cats: 
    text = " ".join(train.loc[train['general_cat']==cat, 'item_description'].values)
    cat_desc[cat] = tokenize(text)


# find the most common words for the top 4 categories
women100 = Counter(cat_desc['Women']).most_common(100)
beauty100 = Counter(cat_desc['Beauty']).most_common(100)
kids100 = Counter(cat_desc['Kids']).most_common(100)
electronics100 = Counter(cat_desc['Electronics']).most_common(100)

In [ ]:
# plot word cloud
fig,axes = plt.subplots(2, 2, figsize=(30, 15))

ax = axes[0, 0]
ax.imshow(generate_wordcloud(women100), interpolation="bilinear")
ax.axis('off')
ax.set_title("Women Top 100", fontsize=30)

ax = axes[0, 1]
ax.imshow(generate_wordcloud(beauty100))
ax.axis('off')
ax.set_title("Beauty Top 100", fontsize=30)

ax = axes[1, 0]
ax.imshow(generate_wordcloud(kids100))
ax.axis('off')
ax.set_title("Kids Top 100", fontsize=30)

ax = axes[1, 1]
ax.imshow(generate_wordcloud(electronics100))
ax.axis('off')
ax.set_title("Electronic Top 100", fontsize=30)

## **Pre-processing:  tf-idf**

tf-idf is the acronym for **Term Frequency–inverse Document Frequency**. It quantifies the importance of a particular word in relative to the vocabulary of a collection of documents or corpus. The metric depends on two factors: 
- **Term Frequency**: the occurences of a word in a given document (i.e. bag of words)
- **Inverse Document Frequency**: the reciprocal number of times a word occurs in a corpus of documents

Think about of it this way: If the word is used extensively in all documents, its existence within a specific document will not be able to provide us much specific information about the document itself. So the second term could be seen as a penalty term that penalizes common words such as "a", "the", "and", etc. tf-idf can therefore, be seen as a weighting scheme for words relevancy in a specific document.

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=10,
                             max_features=180000,
                             tokenizer=tokenize,
                             ngram_range=(1, 2))

In [ ]:
all_desc = np.append(train['item_description'].values, test['item_description'].values)
vz = vectorizer.fit_transform(list(all_desc))

vz is a tfidf matrix where:
* the number of rows is the total number of descriptions
* the number of columns is the total number of unique tokens across the descriptions

In [ ]:
#  create a dictionary mapping the tokens to their tfidf values
tfidf = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))
tfidf = pd.DataFrame(columns=['tfidf']).from_dict(
                    dict(tfidf), orient='index')
tfidf.columns = ['tfidf']

Below is the 10 tokens with the lowest tfidf score, which is unsurprisingly, very generic words that we could not use to distinguish one description from another.

In [ ]:
tfidf.sort_values(by=['tfidf'], ascending=True).head(10)

Below is the 10 tokens with the highest tfidf score, which includes words that are a lot specific that by looking at them, we could guess the categories that they belong to: 

In [ ]:
tfidf.sort_values(by=['tfidf'], ascending=False).head(10)

Given the high dimension of our tfidf matrix, we need to reduce their dimension using the Singular Value Decomposition (SVD) technique. And to visualize our vocabulary, we could next use t-SNE to reduce the dimension from 50 to 2. t-SNE is more suitable for dimensionality reduction to 2 or 3. 

### **t-Distributed Stochastic Neighbor Embedding (t-SNE)**

t-SNE is a technique for dimensionality reduction that is particularly well suited for the visualization of high-dimensional datasets. The goal is to take a set of points in a high-dimensional space and find a representation of those points in a lower-dimensional space, typically the 2D plane. It is based on probability distributions with random walk on neighborhood graphs to find the structure within the data.

First, let's take a sample from the both training and testing item's description since t-SNE can take a very long time to execute. We can then reduce the dimension of each vector from to n_components (50) using SVD.

In [ ]:
trn = train.copy()
tst = test.copy()
trn['is_test'] = 1
test['is_test'] = 0

sample_sz = 15000

combined_df = pd.concat([trn, test])
combined_sample = combined_df.sample(n=sample_sz)
vz_sample = vectorizer.fit_transform(list(combined_sample['item_description']))

In [ ]:
from sklearn.decomposition import TruncatedSVD

n_comp=30
svd = TruncatedSVD(n_components=n_comp, random_state=42)
svd_tfidf = svd.fit_transform(vz_sample)

In [ ]:
from sklearn.manifold import TSNE
tsne_model = TSNE(n_components=2, verbose=1, random_state=42, n_iter=500)

In [ ]:
tsne_tfidf = tsne_model.fit_transform(svd_tfidf)

In [ ]:
output_notebook()
plot_tfidf = bp.figure(plot_width=700, plot_height=600,
                       title="tf-idf clustering of the item description",
    tools="pan,wheel_zoom,box_zoom,reset,hover,previewsave",
    x_axis_type=None, y_axis_type=None, min_border=1)

In [ ]:
combined_sample.reset_index(inplace=True, drop=True)

In [ ]:
tfidf_df = pd.DataFrame(tsne_tfidf, columns=['x', 'y'])
tfidf_df['description'] = combined_sample['item_description']
tfidf_df['tokens'] = combined_sample['tokens']
tfidf_df['category'] = combined_sample['general_cat']

In [ ]:
plot_tfidf.scatter(x='x', y='y', source=tfidf_df, alpha=0.7)
hover = plot_tfidf.select(dict(type=HoverTool))
hover.tooltips={"description": "@description", "tokens": "@tokens", "category":"@category"}
show(plot_tfidf)

## K-Means Clustering

In [ ]:
from sklearn.cluster import MiniBatchKMeans

num_clusters = 30 # need to be selected wisely
kmeans_model = MiniBatchKMeans(n_clusters=num_clusters,
                               init='k-means++',
                               n_init=1,
                               init_size=1000, batch_size=1000, verbose=0, max_iter=1000)

In [ ]:
kmeans = kmeans_model.fit(vz)
kmeans_clusters = kmeans.predict(vz)
kmeans_distances = kmeans.transform(vz)

Let's look at the main keywords that describe each clusters: 

In [ ]:
sorted_centroids = kmeans.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()

for i in range(num_clusters-20):
    print("Cluster %d:" % i)
    aux = ''
    for j in sorted_centroids[i, :10]:
        aux += terms[j] + ' | '
    print(aux)
    print() 

In order to plot these clusters, first we will need to reduce the dimension of the distances to 2 using tsne: 

In [ ]:
# repeat the same steps for the sample
kmeans = kmeans_model.fit(vz_sample)
kmeans_clusters = kmeans.predict(vz_sample)
kmeans_distances = kmeans.transform(vz_sample)
# reduce dimension to 2 using tsne
tsne_kmeans = tsne_model.fit_transform(kmeans_distances)

In [ ]:
colormap = np.array(["#6d8dca", "#69de53", "#723bca", "#c3e14c", "#c84dc9", "#68af4e", "#6e6cd5",
"#e3be38", "#4e2d7c", "#5fdfa8", "#d34690", "#3f6d31", "#d44427", "#7fcdd8", "#cb4053", "#5e9981",
"#803a62", "#9b9e39", "#c88cca", "#e1c37b", "#34223b", "#bdd8a3", "#6e3326", "#cfbdce", "#d07d3c",
"#52697d", "#7d6d33", "#d27c88", "#36422b", "#b68f79"])

plot_kmeans = bp.figure(plot_width=700, plot_height=600,
                        title="KMeans clustering of the description",
    tools="pan,wheel_zoom,box_zoom,reset,hover,previewsave",
    x_axis_type=None, y_axis_type=None, min_border=1)

clusters = np.unique(kmeans_clusters) 
cmap = dict()
for i,cluster in enumerate(clusters): 
    cmap[str(cluster)] = colormap[i]

In [ ]:
#combined_sample.reset_index(drop=True, inplace=True)
kmeans_df = pd.DataFrame(tsne_kmeans, columns=['x', 'y'])
kmeans_df['cluster'] = kmeans_clusters
kmeans_df['description'] = combined_sample['item_description']
kmeans_df['category'] = combined_sample['general_cat']

In [ ]:
# plot_kmeans.scatter(x='x', y='y', source=kmeans_df,
#                    color=factor_cmap('cluster',
#                                  palette=list(cmap.values()), factors=list(cmap.keys())))
# hover = plot_kmeans.select(dict(type=HoverTool))
# hover.tooltips={"description": "@description", "category": "@category", "cluster":"@cluster" }
# show(plot_kmeans)

## ** Latent Dirichlet Allocation**

Latent Dirichlet Allocation (LDA) is an algorithms used to discover the topics that are present in a corpus. LDA is based on probabilistic graphical modelling and it is capable of returning the documents that belong to a topic in a corpus and the words that belong to a topic.

Its input is a **bag of words**, i.e. each document represented as a row, with each columns containing the count of words in the corpus. We are going to use a powerful tool called pyLDAvis that gives us an interactive visualization for LDA. 

*You can read more about LDA at: https://medium.com/@aneesha/topic-modeling-with-scikit-learn-e80d33668730*

In [ ]:
cvectorizer = CountVectorizer(min_df=4,
                              max_features=180000,
                              tokenizer=tokenize,
                              ngram_range=(1,2))

cvz = cvectorizer.fit_transform(combined_sample['item_description'])

n_topics = 20
n_iter = 2000
lda_model = lda.LDA(n_topics=n_topics, n_iter=n_iter)
X_topics = lda_model.fit_transform(cvz)

In [ ]:
# reduce dimension to 2 using tsne
tsne_lda = tsne_model.fit_transform(X_topics)

In [ ]:
doc_topic = lda_model.doc_topic_
lda_keys = []
for i, tweet in enumerate(combined_sample['item_description']):
    lda_keys += [doc_topic[i].argmax()]

lda_df = pd.DataFrame(tsne_lda, columns=['x','y'])
lda_df['description'] = combined_sample['item_description']
lda_df['category'] = combined_sample['general_cat']
lda_df['topic'] = lda_keys
lda_df['topic'] = lda_df['topic'].map(int)

In [ ]:
plot_lda = bp.figure(plot_width=700,
                     plot_height=600,
                     title="LDA topic visualization",
    tools="pan,wheel_zoom,box_zoom,reset,hover,previewsave",
    x_axis_type=None, y_axis_type=None, min_border=1)

keys = np.unique(lda_df['topic'].values)
cmap = dict()
for i,k in enumerate(keys): 
    cmap[str(k)] = colormap[i]

In [ ]:
def prepareLDAData():
    data = {
        'vocab': vocab,
        'doc_topic_dists': lda_model.doc_topic_,
        'doc_lengths': list(lda_df['len_docs']),
        'term_frequency':cvectorizer.vocabulary_,
        'topic_term_dists': lda_model.components_
    } 
    return data

In [ ]:
import pyLDAvis

lda_df['len_docs'] = combined_sample['tokens'].map(len)
ldadata = prepareLDAData()
pyLDAvis.enable_notebook()
prepared_data = pyLDAvis.prepare(**ldadata)

In [ ]:
html_string="""
\<!DOCTYPE html>
<meta charset="utf-8">

<link rel="stylesheet" type="text/css" href="https://cdn.rawgit.com/bmabey/pyLDAvis/files/ldavis.v1.0.0.css">


<div id="ldavis_el499021163441769524580143789"></div>
<script type="text/javascript">

var ldavis_el499021163441769524580143789_data = {"mdsDat": {"Freq": [6.703822024464648, 6.636094279002835, 6.509919259272986, 6.082364273717624, 5.895533562705117, 5.824908981178469, 5.583948795809036, 5.490098912895271, 5.280251573593505, 5.277217422150439, 5.083975759014407, 4.7944149778014795, 4.657165840290482, 4.6097690625458, 4.445775444070983, 4.200302602665383, 4.098756273729539, 3.9823923027883414, 3.354304467524888, 1.48898418477877], "cluster": [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], "topics": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], "x": [0.27774841177382004, 0.08017133790963385, -0.18577240386914512, 0.11734621876253024, 0.07774021522283102, 0.2673622077418905, 0.02310745942456076, -0.032338232983442974, -0.18650998094142093, -0.0558191934074682, -0.2452966101808272, -0.16789378445399794, -0.02518049120678609, 0.06166691775146207, -0.023213416546706653, 0.21060728634287393, -0.1682679421991669, -0.14240865844871023, 0.0851481937848881, 0.03180246552318101], "y": [-0.034528826013619375, 0.09640670034003634, -0.02138796085228505, 0.2096907180886993, 0.018028156008942663, -0.07231908609787706, 0.0009013862467281812, -0.011285708215164872, -0.15676120311600966, 0.3105069523981808, -0.07987789831838293, 0.1633256938796853, -0.06900606330846362, 0.1694633947079834, -0.07349646611854674, -0.1749589828158121, -0.04277917006766046, -0.10033432635662266, -0.14240001465298327, 0.010812704263173264]}, "tinfo": {"Category": ["Default", "Default", "Default", "Default", "Default", "Default", "Default", "Default", "Default", "Default", "Default", "Default", "Default", "Default", "Default", "Default", "Default", "Default", "Default", "Default", "Default", "Default", "Default", "Default", "Default", "Default", "Default", "Default", "Default", "Default", "Topic1", "Topic1", "Topic1", "Topic1", "Topic1", "Topic1", "Topic1", "Topic1", "Topic1", "Topic1", "Topic1", "Topic1", "Topic1", "Topic1", "Topic1", "Topic1", "Topic1", "Topic1", "Topic1", "Topic1", "Topic1", "Topic1", "Topic1", "Topic1", "Topic1", "Topic1", "Topic1", "Topic1", "Topic1", "Topic1", "Topic1", "Topic1", "Topic1", "Topic1", "Topic1", "Topic1", "Topic1", "Topic1", "Topic1", "Topic1", "Topic1", "Topic1", "Topic1", "Topic1", "Topic1", "Topic1", "Topic1", "Topic1", "Topic1", "Topic1", "Topic1", "Topic1", "Topic1", "Topic1", "Topic1", "Topic1", "Topic1", "Topic2", "Topic2", "Topic2", "Topic2", "Topic2", "Topic2", "Topic2", "Topic2", "Topic2", "Topic2", "Topic2", "Topic2", "Topic2", "Topic2", "Topic2", "Topic2", "Topic2", "Topic2", "Topic2", "Topic2", "Topic2", "Topic2", "Topic2", "Topic2", "Topic2", "Topic2", "Topic2", "Topic2", "Topic2", "Topic2", "Topic2", "Topic2", "Topic2", "Topic2", "Topic2", "Topic2", "Topic2", "Topic2", "Topic2", "Topic2", "Topic2", "Topic2", "Topic2", "Topic2", "Topic2", "Topic2", "Topic2", "Topic2", "Topic2", "Topic2", "Topic2", "Topic2", "Topic2", "Topic2", "Topic2", "Topic2", "Topic2", "Topic2", "Topic2", "Topic2", "Topic2", "Topic2", "Topic2", "Topic2", "Topic2", "Topic2", "Topic2", "Topic2", "Topic2", "Topic2", "Topic2", "Topic2", "Topic3", "Topic3", "Topic3", "Topic3", "Topic3", "Topic3", "Topic3", "Topic3", "Topic3", "Topic3", "Topic3", "Topic3", "Topic3", "Topic3", "Topic3", "Topic3", "Topic3", "Topic3", "Topic3", "Topic3", "Topic3", "Topic3", "Topic3", "Topic3", "Topic3", "Topic3", "Topic3", "Topic3", "Topic3", "Topic3", "Topic3", "Topic3", "Topic3", "Topic3", "Topic3", "Topic3", "Topic3", "Topic3", "Topic3", "Topic3", "Topic3", "Topic3", "Topic3", "Topic3", "Topic3", "Topic3", "Topic3", "Topic3", "Topic3", "Topic3", "Topic3", "Topic3", "Topic3", "Topic3", "Topic3", "Topic3", "Topic3", "Topic3", "Topic3", "Topic3", "Topic3", "Topic3", "Topic3", "Topic3", "Topic3", "Topic3", "Topic3", "Topic3", "Topic3", "Topic4", "Topic4", "Topic4", "Topic4", "Topic4", "Topic4", "Topic4", "Topic4", "Topic4", "Topic4", "Topic4", "Topic4", "Topic4", "Topic4", "Topic4", "Topic4", "Topic4", "Topic4", "Topic4", "Topic4", "Topic4", "Topic4", "Topic4", "Topic4", "Topic4", "Topic4", "Topic4", "Topic4", "Topic4", "Topic4", "Topic4", "Topic4", "Topic4", "Topic4", "Topic4", "Topic4", "Topic4", "Topic4", "Topic4", "Topic4", "Topic4", "Topic4", "Topic4", "Topic4", "Topic4", "Topic4", "Topic4", "Topic4", "Topic4", "Topic4", "Topic4", "Topic4", "Topic4", "Topic4", "Topic4", "Topic4", "Topic4", "Topic4", "Topic4", "Topic4", "Topic4", "Topic4", "Topic4", "Topic4", "Topic4", "Topic4", "Topic4", "Topic4", "Topic4", "Topic4", "Topic4", "Topic4", "Topic4", "Topic4", "Topic5", "Topic5", "Topic5", "Topic5", "Topic5", "Topic5", "Topic5", "Topic5", "Topic5", "Topic5", "Topic5", "Topic5", "Topic5", "Topic5", "Topic5", "Topic5", "Topic5", "Topic5", "Topic5", "Topic5", "Topic5", "Topic5", "Topic5", "Topic5", "Topic5", "Topic5", "Topic5", "Topic5", "Topic5", "Topic5", "Topic5", "Topic5", "Topic5", "Topic5", "Topic5", "Topic5", "Topic5", "Topic5", "Topic5", "Topic5", "Topic5", "Topic5", "Topic5", "Topic5", "Topic5", "Topic5", "Topic5", "Topic5", "Topic5", "Topic5", "Topic5", "Topic5", "Topic5", "Topic5", "Topic5", "Topic5", "Topic6", "Topic6", "Topic6", "Topic6", "Topic6", "Topic6", "Topic6", "Topic6", "Topic6", "Topic6", "Topic6", "Topic6", "Topic6", "Topic6", "Topic6", "Topic6", "Topic6", "Topic6", "Topic6", "Topic6", "Topic6", "Topic6", "Topic6", "Topic6", "Topic6", "Topic6", "Topic6", "Topic6", "Topic6", "Topic6", "Topic6", "Topic6", "Topic6", "Topic6", "Topic6", "Topic6", "Topic6", "Topic6", "Topic6", "Topic6", "Topic6", "Topic6", "Topic6", "Topic6", "Topic6", "Topic6", "Topic6", "Topic6", "Topic6", "Topic6", "Topic6", "Topic6", "Topic6", "Topic6", "Topic6", "Topic6", "Topic6", "Topic6", "Topic6", "Topic6", "Topic6", "Topic6", "Topic6", "Topic6", "Topic6", "Topic6", "Topic7", "Topic7", "Topic7", "Topic7", "Topic7", "Topic7", "Topic7", "Topic7", "Topic7", "Topic7", "Topic7", "Topic7", "Topic7", "Topic7", "Topic7", "Topic7", "Topic7", "Topic7", "Topic7", "Topic7", "Topic7", "Topic7", "Topic7", "Topic7", "Topic7", "Topic7", "Topic7", "Topic7", "Topic7", "Topic7", "Topic7", "Topic7", "Topic7", "Topic7", "Topic7", "Topic7", "Topic7", "Topic7", "Topic7", "Topic7", "Topic7", "Topic7", "Topic7", "Topic7", "Topic7", "Topic7", "Topic7", "Topic7", "Topic7", "Topic7", "Topic8", "Topic8", "Topic8", "Topic8", "Topic8", "Topic8", "Topic8", "Topic8", "Topic8", "Topic8", "Topic8", "Topic8", "Topic8", "Topic8", "Topic8", "Topic8", "Topic8", "Topic8", "Topic8", "Topic8", "Topic8", "Topic8", "Topic8", "Topic8", "Topic8", "Topic8", "Topic8", "Topic8", "Topic8", "Topic8", "Topic8", "Topic8", "Topic8", "Topic8", "Topic8", "Topic8", "Topic8", "Topic8", "Topic8", "Topic8", "Topic8", "Topic8", "Topic8", "Topic8", "Topic8", "Topic8", "Topic8", "Topic8", "Topic8", "Topic8", "Topic8", "Topic8", "Topic8", "Topic8", "Topic8", "Topic8", "Topic8", "Topic8", "Topic8", "Topic8", "Topic8", "Topic8", "Topic8", "Topic9", "Topic9", "Topic9", "Topic9", "Topic9", "Topic9", "Topic9", "Topic9", "Topic9", "Topic9", "Topic9", "Topic9", "Topic9", "Topic9", "Topic9", "Topic9", "Topic9", "Topic9", "Topic9", "Topic9", "Topic9", "Topic9", "Topic9", "Topic9", "Topic9", "Topic9", "Topic9", "Topic9", "Topic9", "Topic9", "Topic9", "Topic9", "Topic9", "Topic9", "Topic9", "Topic9", "Topic9", "Topic9", "Topic9", "Topic9", "Topic9", "Topic9", "Topic9", "Topic9", "Topic9", "Topic9", "Topic9", "Topic9", "Topic9", "Topic9", "Topic9", "Topic9", "Topic9", "Topic9", "Topic9", "Topic9", "Topic9", "Topic9", "Topic9", "Topic9", "Topic9", "Topic9", "Topic10", "Topic10", "Topic10", "Topic10", "Topic10", "Topic10", "Topic10", "Topic10", "Topic10", "Topic10", "Topic10", "Topic10", "Topic10", "Topic10", "Topic10", "Topic10", "Topic10", "Topic10", "Topic10", "Topic10", "Topic10", "Topic10", "Topic10", "Topic10", "Topic10", "Topic10", "Topic10", "Topic10", "Topic10", "Topic10", "Topic10", "Topic10", "Topic10", "Topic10", "Topic10", "Topic10", "Topic10", "Topic10", "Topic10", "Topic10", "Topic10", "Topic10", "Topic10", "Topic10", "Topic10", "Topic10", "Topic10", "Topic10", "Topic10", "Topic10", "Topic10", "Topic10", "Topic10", "Topic10", "Topic10", "Topic10", "Topic10", "Topic10", "Topic10", "Topic10", "Topic10", "Topic10", "Topic10", "Topic11", "Topic11", "Topic11", "Topic11", "Topic11", "Topic11", "Topic11", "Topic11", "Topic11", "Topic11", "Topic11", "Topic11", "Topic11", "Topic11", "Topic11", "Topic11", "Topic11", "Topic11", "Topic11", "Topic11", "Topic11", "Topic11", "Topic11", "Topic11", "Topic11", "Topic11", "Topic11", "Topic11", "Topic11", "Topic11", "Topic11", "Topic11", "Topic11", "Topic11", "Topic11", "Topic11", "Topic11", "Topic11", "Topic11", "Topic11", "Topic11", "Topic11", "Topic11", "Topic11", "Topic11", "Topic11", "Topic11", "Topic11", "Topic11", "Topic11", "Topic11", "Topic11", "Topic11", "Topic11", "Topic11", "Topic11", "Topic11", "Topic11", "Topic11", "Topic11", "Topic11", "Topic11", "Topic11", "Topic11", "Topic11", "Topic11", "Topic11", "Topic12", "Topic12", "Topic12", "Topic12", "Topic12", "Topic12", "Topic12", "Topic12", "Topic12", "Topic12", "Topic12", "Topic12", "Topic12", "Topic12", "Topic12", "Topic12", "Topic12", "Topic12", "Topic12", "Topic12", "Topic12", "Topic12", "Topic12", "Topic12", "Topic12", "Topic12", "Topic12", "Topic12", "Topic12", "Topic12", "Topic12", "Topic12", "Topic12", "Topic12", "Topic12", "Topic12", "Topic12", "Topic12", "Topic12", "Topic12", "Topic12", "Topic12", "Topic12", "Topic12", "Topic12", "Topic12", "Topic12", "Topic12", "Topic12", "Topic12", "Topic12", "Topic12", "Topic12", "Topic12", "Topic12", "Topic12", "Topic13", "Topic13", "Topic13", "Topic13", "Topic13", "Topic13", "Topic13", "Topic13", "Topic13", "Topic13", "Topic13", "Topic13", "Topic13", "Topic13", "Topic13", "Topic13", "Topic13", "Topic13", "Topic13", "Topic13", "Topic13", "Topic13", "Topic13", "Topic13", "Topic13", "Topic13", "Topic13", "Topic13", "Topic13", "Topic13", "Topic13", "Topic13", "Topic13", "Topic13", "Topic13", "Topic13", "Topic13", "Topic13", "Topic13", "Topic13", "Topic13", "Topic13", "Topic13", "Topic13", "Topic13", "Topic13", "Topic13", "Topic13", "Topic13", "Topic13", "Topic13", "Topic13", "Topic13", "Topic13", "Topic13", "Topic13", "Topic13", "Topic13", "Topic13", "Topic13", "Topic13", "Topic13", "Topic14", "Topic14", "Topic14", "Topic14", "Topic14", "Topic14", "Topic14", "Topic14", "Topic14", "Topic14", "Topic14", "Topic14", "Topic14", "Topic14", "Topic14", "Topic14", "Topic14", "Topic14", "Topic14", "Topic14", "Topic14", "Topic14", "Topic14", "Topic14", "Topic14", "Topic14", "Topic14", "Topic14", "Topic14", "Topic14", "Topic14", "Topic14", "Topic14", "Topic14", "Topic14", "Topic14", "Topic14", "Topic14", "Topic14", "Topic14", "Topic14", "Topic14", "Topic14", "Topic14", "Topic14", "Topic14", "Topic14", "Topic14", "Topic14", "Topic14", "Topic14", "Topic14", "Topic14", "Topic14", "Topic14", "Topic14", "Topic14", "Topic15", "Topic15", "Topic15", "Topic15", "Topic15", "Topic15", "Topic15", "Topic15", "Topic15", "Topic15", "Topic15", "Topic15", "Topic15", "Topic15", "Topic15", "Topic15", "Topic15", "Topic15", "Topic15", "Topic15", "Topic15", "Topic15", "Topic15", "Topic15", "Topic15", "Topic15", "Topic15", "Topic15", "Topic15", "Topic15", "Topic15", "Topic15", "Topic15", "Topic15", "Topic15", "Topic15", "Topic15", "Topic15", "Topic15", "Topic15", "Topic15", "Topic15", "Topic15", "Topic15", "Topic15", "Topic15", "Topic15", "Topic15", "Topic15", "Topic15", "Topic15", "Topic15", "Topic16", "Topic16", "Topic16", "Topic16", "Topic16", "Topic16", "Topic16", "Topic16", "Topic16", "Topic16", "Topic16", "Topic16", "Topic16", "Topic16", "Topic16", "Topic16", "Topic16", "Topic16", "Topic16", "Topic16", "Topic16", "Topic16", "Topic16", "Topic16", "Topic16", "Topic16", "Topic16", "Topic16", "Topic16", "Topic16", "Topic16", "Topic16", "Topic16", "Topic16", "Topic16", "Topic16", "Topic16", "Topic16", "Topic16", "Topic16", "Topic16", "Topic16", "Topic16", "Topic16", "Topic16", "Topic16", "Topic16", "Topic16", "Topic16", "Topic16", "Topic16", "Topic16", "Topic16", "Topic16", "Topic16", "Topic16", "Topic16", "Topic16", "Topic16", "Topic16", "Topic16", "Topic16", "Topic16", "Topic16", "Topic17", "Topic17", "Topic17", "Topic17", "Topic17", "Topic17", "Topic17", "Topic17", "Topic17", "Topic17", "Topic17", "Topic17", "Topic17", "Topic17", "Topic17", "Topic17", "Topic17", "Topic17", "Topic17", "Topic17", "Topic17", "Topic17", "Topic17", "Topic17", "Topic17", "Topic17", "Topic17", "Topic17", "Topic17", "Topic17", "Topic17", "Topic17", "Topic17", "Topic17", "Topic17", "Topic17", "Topic17", "Topic17", "Topic17", "Topic17", "Topic17", "Topic17", "Topic17", "Topic17", "Topic17", "Topic17", "Topic17", "Topic17", "Topic17", "Topic17", "Topic18", "Topic18", "Topic18", "Topic18", "Topic18", "Topic18", "Topic18", "Topic18", "Topic18", "Topic18", "Topic18", "Topic18", "Topic18", "Topic18", "Topic18", "Topic18", "Topic18", "Topic18", "Topic18", "Topic18", "Topic18", "Topic18", "Topic18", "Topic18", "Topic18", "Topic18", "Topic18", "Topic18", "Topic18", "Topic18", "Topic18", "Topic18", "Topic18", "Topic18", "Topic18", "Topic18", "Topic18", "Topic18", "Topic18", "Topic18", "Topic18", "Topic18", "Topic18", "Topic18", "Topic18", "Topic18", "Topic18", "Topic18", "Topic18", "Topic18", "Topic18", "Topic18", "Topic18", "Topic18", "Topic18", "Topic18", "Topic18", "Topic18", "Topic18", "Topic18", "Topic18", "Topic18", "Topic18", "Topic19", "Topic19", "Topic19", "Topic19", "Topic19", "Topic19", "Topic19", "Topic19", "Topic19", "Topic19", "Topic19", "Topic19", "Topic19", "Topic19", "Topic19", "Topic19", "Topic19", "Topic19", "Topic19", "Topic19", "Topic19", "Topic19", "Topic19", "Topic19", "Topic19", "Topic19", "Topic19", "Topic19", "Topic19", "Topic19", "Topic19", "Topic19", "Topic19", "Topic19", "Topic19", "Topic19", "Topic19", "Topic19", "Topic19", "Topic19", "Topic19", "Topic19", "Topic19", "Topic19", "Topic19", "Topic19", "Topic19", "Topic19", "Topic19", "Topic19", "Topic19", "Topic19", "Topic19", "Topic19", "Topic19", "Topic19", "Topic19", "Topic19", "Topic19", "Topic19", "Topic19", "Topic19", "Topic19", "Topic19", "Topic19", "Topic19", "Topic19", "Topic19", "Topic19", "Topic19", "Topic19", "Topic20", "Topic20", "Topic20", "Topic20", "Topic20", "Topic20", "Topic20", "Topic20", "Topic20", "Topic20", "Topic20", "Topic20", "Topic20", "Topic20", "Topic20", "Topic20", "Topic20", "Topic20", "Topic20", "Topic20", "Topic20", "Topic20", "Topic20", "Topic20", "Topic20", "Topic20", "Topic20", "Topic20", "Topic20", "Topic20", "Topic20", "Topic20", "Topic20", "Topic20", "Topic20", "Topic20", "Topic20", "Topic20", "Topic20", "Topic20", "Topic20", "Topic20", "Topic20", "Topic20", "Topic20", "Topic20", "Topic20", "Topic20", "Topic20", "Topic20", "Topic20", "Topic20", "Topic20"], "Freq": [3861.0, 1894.0, 2112.0, 3467.0, 1676.0, 2078.0, 707.0, 1796.0, 670.0, 649.0, 1382.0, 1207.0, 1097.0, 821.0, 1451.0, 612.0, 755.0, 937.0, 848.0, 914.0, 609.0, 991.0, 898.0, 540.0, 769.0, 478.0, 469.0, 492.0, 1215.0, 475.0, 153.21086753360495, 152.44100236334413, 140.8930248094319, 113.17787868004251, 102.3997662963911, 97.78057527482622, 96.24084493430458, 81.61340669934908, 56.977721251002976, 55.43799091048135, 55.43799091048135, 43.120148186308285, 40.040687505265026, 37.731091994482576, 33.11190097291768, 30.03244029187442, 30.03244029187442, 26.183114440570343, 24.643384100048713, 23.8735189297879, 23.10365375952708, 20.794058248744633, 20.794058248744633, 20.794058248744633, 20.794058248744633, 20.794058248744633, 19.254327908223004, 18.484462737962186, 18.484462737962186, 18.484462737962186, 316.42228362889784, 231.73711490020816, 363.38405901480763, 384.9402837821105, 336.4387780556791, 67.7558336346544, 107.018957317956, 97.0107101045654, 66.21610329413276, 697.5055429080016, 253.293339667511, 435.75138501932435, 172.45749679012533, 64.67637295361114, 496.57073346992877, 73.14488982648011, 578.9463066878361, 88.54219323169643, 471.16518285132184, 116.25733936108578, 220.18913734629592, 95.47097976404378, 90.85178874247887, 117.79706970160741, 105.47922697743435, 93.16138425326133, 103.16963146665191, 139.0300982926889, 47.67398770824936, 46.68098650624459, 36.75097448619681, 31.785968476172926, 31.785968476172926, 30.79296727416815, 29.79996607216337, 29.79996607216337, 28.806964870158595, 27.813963668153818, 27.813963668153818, 27.813963668153818, 27.813963668153818, 26.82096246614904, 25.82796126414426, 25.82796126414426, 24.834960062139483, 21.855956456125153, 20.862955254120376, 20.862955254120376, 19.8699540521156, 19.8699540521156, 18.876952850110822, 18.876952850110822, 18.876952850110822, 208.5401824330233, 17.88395164810604, 17.88395164810604, 17.88395164810604, 17.88395164810604, 17.88395164810604, 38.73697689020637, 35.75797328419203, 76.47102256638793, 262.1622473412813, 130.09308747464587, 27.813963668153818, 26.82096246614904, 26.82096246614904, 46.68098650624459, 35.75797328419203, 127.11408386863154, 105.26805742452646, 31.785968476172926, 42.70898169822548, 53.63199492027803, 110.23306343455036, 106.26105862653125, 76.47102256638793, 63.562006940325816, 160.87612473679397, 42.70898169822548, 52.63899371827325, 70.51301535435925, 52.63899371827325, 46.68098650624459, 72.49901775836881, 41.7159804962207, 66.54101054634015, 67.53401174834492, 58.59700093030192, 71.50601655636403, 67.53401174834492, 59.590002132306694, 70.51301535435925, 82.42902977841659, 69.52001415235448, 52.63899371827325, 55.617997324287586, 48.66698891025414, 49.65999011225892, 85.92613230271962, 59.36956018843486, 59.36956018843486, 56.24525758675429, 41.404820228771634, 40.623744578351484, 38.280517627091065, 35.937290675830646, 28.126534171629245, 27.345458521209103, 27.345458521209103, 25.002231569948687, 24.22115591952855, 24.22115591952855, 23.440080269108407, 22.659004618688268, 21.87792896826813, 20.315777667427845, 19.534702017007707, 17.191475065747287, 16.410399415327145, 16.410399415327145, 16.410399415327145, 15.629323764907006, 15.629323764907006, 15.629323764907006, 14.848248114486866, 14.067172464066726, 14.067172464066726, 14.067172464066726, 242.14126238674763, 146.0689573850704, 128.88529307582735, 95.2990401077613, 28.907609822049388, 145.28788173465026, 442.87770454472366, 34.37513937499037, 457.7181419027063, 101.54764531112241, 84.36398100187935, 214.0225389716226, 73.42892189599738, 96.8611914086016, 57.02633323717444, 174.9687564506156, 55.46418193633416, 128.88529307582735, 250.73309454136918, 128.88529307582735, 257.76277539515047, 60.93171148927514, 182.779512954817, 65.61816539179597, 109.35840181532383, 120.29346092120578, 365.5512151531298, 146.0689573850704, 99.204418359862, 171.0633781985149, 132.0095956775079, 74.20999754641753, 76.55322449767795, 71.8667705951571, 100.76656966070227, 79.6775270993585, 83.5829053514592, 83.5829053514592, 78.89645144893836, 55.683846184457124, 48.09174475043024, 36.281809186388436, 35.438242360385445, 33.75110870837947, 27.002574100355584, 26.1590072743526, 25.315440448349616, 21.097606318334684, 21.097606318334684, 21.097606318334684, 19.41047266632871, 19.41047266632871, 19.41047266632871, 18.566905840325727, 17.72333901432274, 17.72333901432274, 17.72333901432274, 16.879772188319755, 13.50550488430781, 13.50550488430781, 13.50550488430781, 13.50550488430781, 12.661938058304822, 12.661938058304822, 12.661938058304822, 12.661938058304822, 11.818371232301836, 11.818371232301836, 11.818371232301836, 11.818371232301836, 122.325625438693, 34.59467553438246, 11.818371232301836, 27.84614092635857, 134.13556100273482, 33.75110870837947, 107.14142257063928, 146.7890633927796, 173.78320182487516, 37.968942838394405, 33.75110870837947, 54.84027935845413, 41.34321014240635, 164.50396673884234, 91.11365287658253, 175.47033547688116, 63.275947618484, 36.281809186388436, 111.3592567006542, 124.01275909069899, 131.60486052472586, 48.93531157643323, 30.37684140436753, 103.76715526662734, 310.4410276373589, 70.86804905251087, 80.99085096454671, 106.2978557446363, 249.70421616514395, 88.58295239857358, 148.47619704478558, 90.27008605057955, 187.28027104092294, 108.82855622264523, 85.20868509456163, 106.2978557446363, 178.0010359548901, 90.27008605057955, 84.36511826855865, 85.20868509456163, 112.20282352665717, 91.95721970258552, 84.36511826855865, 164.79161558611437, 119.36503110477915, 116.6928790764653, 97.09709753549717, 94.42494550718334, 92.64351082164079, 70.3755772523588, 61.468403824645996, 50.77979571139064, 50.77979571139064, 50.77979571139064, 44.54477431199169, 39.200470255364, 37.419035569821446, 36.528318227050164, 35.63760088427889, 35.63760088427889, 33.85616619873633, 32.074731513193775, 32.074731513193775, 32.074731513193775, 29.40257948487993, 28.51186214210865, 28.51186214210865, 25.839710113794816, 24.948992771023537, 24.948992771023537, 24.948992771023537, 24.05827542825226, 24.05827542825226, 24.05827542825226, 41.87262228367784, 448.9304479301526, 236.04900300781676, 57.90553445356088, 364.312300366881, 130.0536392180345, 130.94435656080577, 67.70342522404496, 221.79752552347628, 62.35912116741727, 73.93844662344391, 71.26629459513008, 52.5612303969332, 64.14055585295984, 50.77979571139064, 63.24983851018855, 85.51777207947053, 65.03127319573112, 75.71988130898646, 68.59414256681623, 80.17346802284287, 146.97726873068882, 78.3920333373003, 86.40848942224183, 63.24983851018855, 140.973935345512, 100.4665118222915, 94.79547252904062, 94.79547252904062, 93.98532405857621, 48.61700971256925, 47.80686124210484, 43.75611888978278, 41.325673478389554, 41.325673478389554, 39.70537653746073, 38.895228066996324, 34.034337244209866, 33.22418877374545, 31.603891832816633, 30.793743362352224, 29.983594891887815, 25.93285253956576, 25.122704069101356, 25.122704069101356, 24.312555598636944, 24.312555598636944, 23.502407128172536, 23.502407128172536, 22.692258657708127, 21.882110187243715, 21.071961716779306, 20.261813246314897, 20.261813246314897, 18.641516305386077, 57.52864288767776, 153.93631087294256, 32.41404030328104, 53.47790053535571, 29.983594891887815, 91.55487864718299, 136.11304452272552, 121.53037205436615, 320.0167473181466, 55.90834594674894, 191.2031405143054, 33.22418877374545, 114.23903582018646, 54.28804900582012, 282.74991767678375, 166.08853792990868, 110.99844193832882, 83.45339394253888, 204.97566451220038, 328.1182320227907, 46.996712771640425, 54.28804900582012, 65.63012759232186, 324.06748967046866, 177.43061651641042, 76.9722061788236, 63.199682180928626, 168.51898334130192, 68.06057300371509, 89.93458170625416, 79.40265159021682, 76.9722061788236, 64.00983065139305, 120.72022358390174, 80.21280006068123, 67.25042453325068, 249.40032721911336, 218.11775246601042, 189.4420589423327, 133.8285927145941, 125.13898861650993, 109.49770123995846, 104.28393878110798, 83.42888894570599, 79.08408689666392, 77.34616607704709, 75.60824525743026, 70.39448279857976, 67.7876015691545, 66.9186411593461, 63.44279952011244, 62.57383911030402, 60.83591829068719, 57.360076651453525, 53.01527460241144, 52.14631419260303, 52.14631419260303, 50.40839337298619, 46.06359132394412, 45.1946309141357, 43.45671009451887, 40.84982886509362, 40.84982886509362, 33.89814558662629, 33.89814558662629, 33.02918517681788, 189.4420589423327, 48.67047255336936, 144.2561176322951, 97.33225550264063, 143.38715722248665, 67.7876015691545, 102.54601796149115, 77.34616607704709, 78.2151264868555, 65.18072033972926, 119.05626574785103, 165.11116746769704, 268.5174562348985, 95.59433468302382, 163.37324664808023, 106.02185960072481, 97.33225550264063, 109.49770123995846, 98.20121591244906, 84.2978493555144, 143.7211448895948, 133.14109618905408, 104.04596226256723, 88.17588921175619, 57.31741383517918, 48.500706584728604, 48.500706584728604, 42.3290115094132, 41.447340784368144, 40.56567005932309, 37.92065788418791, 37.038987159142856, 35.275645709052746, 28.222279908692283, 28.222279908692283, 27.34060918364723, 26.458938458602173, 23.813926283467, 22.932255558421943, 22.050584833376885, 22.050584833376885, 21.168914108331826, 19.405572658241713, 19.405572658241713, 18.523901933196655, 18.523901933196655, 17.6422312081516, 17.6422312081516, 16.76056048310654, 15.878889758061483, 54.672401660044, 83.7675355865309, 43.21068223445826, 52.02738948490883, 59.962426010314346, 216.89981506833456, 71.4241454359001, 332.39868004923704, 201.02974201752352, 60.844096735359415, 80.24085268635066, 168.4079251908564, 59.962426010314346, 57.31741383517918, 60.844096735359415, 38.80232860923297, 46.73736513463849, 103.16429153752216, 68.77913326076494, 64.37077963553965, 59.0807552852693, 97.87426718725182, 205.4380956427488, 74.95082833608033, 74.06915761103527, 75.83249906112539, 92.58424283698147, 60.844096735359415, 105.80930371265733, 66.13412108562974, 70.54247471085505, 64.37077963553965, 59.962426010314346, 475.4371790742906, 312.7155660222723, 302.81077218432335, 181.83079030651842, 131.5993358426345, 82.07536665288978, 46.70110294592927, 43.16367657523322, 35.3813385597019, 34.67385328556269, 31.843912189005852, 30.42894164072743, 29.721456366588225, 29.721456366588225, 26.891515270031384, 324.7428156826389, 26.18402999589217, 24.76905944761375, 22.646603625196118, 21.231633076917703, 20.52414780277849, 19.816662528639277, 19.816662528639277, 19.10917725450007, 17.69420670622165, 16.98672143208244, 16.27923615794323, 16.27923615794323, 14.864265609664809, 14.864265609664809, 14.864265609664809, 47.40858822006848, 55.1909262355998, 59.43583788043506, 316.96047766710757, 563.1653530675527, 93.39513103911715, 201.6403779824163, 62.2657789769919, 43.16367657523322, 26.18402999589217, 569.5327205348057, 511.51892805539035, 99.05501323223082, 119.57208618226791, 827.7648455956174, 478.2671201708475, 427.3281804328244, 66.51069062182717, 232.76973004454155, 467.6548410587594, 53.77595568732138, 46.70110294592927, 90.5651899425603, 107.54483652190135, 77.12296973391531, 105.42238069948372, 93.39513103911715, 69.340631718384, 101.88495432878766, 85.61279302358584, 66.51069062182717, 492.05675959915135, 88.63760606372193, 73.35657752071323, 64.95201182205845, 58.075548977704536, 57.31149755055411, 55.78339469625324, 55.0193432691028, 45.086674716147144, 45.086674716147144, 44.322623288996716, 37.4461604446428, 35.1540061631915, 31.333749027439325, 31.333749027439325, 30.569697600288887, 30.569697600288887, 30.569697600288887, 30.569697600288887, 27.513491891687153, 27.513491891687153, 27.513491891687153, 25.985389037386277, 23.69323475593498, 23.69323475593498, 22.165131901634112, 21.401080474483674, 20.63702904733324, 18.344874765881936, 126.07612599409322, 750.3061419759983, 38.97426329894367, 105.44673746103146, 116.90750886828799, 239.919788639508, 106.21078888818191, 38.97426329894367, 106.21078888818191, 37.4461604446428, 906.172633114687, 58.075548977704536, 288.05502854998537, 90.92976034517324, 50.43503470620019, 43.558571861846275, 106.21078888818191, 94.75001748092541, 171.15516019596885, 91.69381177232366, 155.11008022580972, 140.59310310995147, 343.8307827319671, 114.61535458683669, 67.24416610350976, 90.16570891802279, 75.64873180216453, 156.6381830801106, 100.09837747097843, 97.80622318952713, 81.76114321936802, 91.69381177232366, 81.76114321936802, 89.40165749087237, 107.88459538152185, 76.42044007269665, 47.95287098375956, 42.70884509895536, 36.71567265917913, 35.21737954923506, 33.71908643929101, 30.722500219402892, 29.973353664430867, 29.973353664430867, 29.973353664430867, 25.478474334598694, 23.980181224654636, 23.980181224654636, 23.980181224654636, 23.23103466968261, 22.481888114710582, 20.983595004766524, 20.234448449794495, 20.234448449794495, 19.485301894822467, 17.98700878487841, 17.23786222990638, 17.23786222990638, 17.23786222990638, 16.48871567493435, 16.48871567493435, 16.48871567493435, 15.739569119962322, 14.990422564990295, 14.990422564990295, 14.990422564990295, 135.6030179154869, 171.5620525541443, 74.17300040778056, 40.461405434039264, 43.45799165392738, 234.49036317179468, 192.53815609336107, 409.04151048027734, 197.78218197816528, 348.360639527543, 152.83338867984355, 478.71214009267607, 62.1866555282281, 52.44775031359173, 199.2804750881093, 298.91696689938914, 65.93238830308825, 172.3111991091163, 46.454577873815495, 888.4953056623757, 113.87776782129806, 38.96311232409521, 68.92897452297636, 124.36581959090647, 95.89825050196939, 65.93238830308825, 147.58936279503936, 62.1866555282281, 58.440922753367964, 98.14569016688547, 68.17982796800433, 77.16958662766868, 107.1354488265498, 83.16275906744491, 65.18324174811622, 140.83070912862672, 127.96358175278142, 122.95969888439714, 75.78023183963109, 69.34666815170843, 62.913104463785785, 53.62017913678641, 46.47177503909458, 44.327253809787024, 42.18273258047948, 33.60464766324929, 32.17496684371092, 30.74528602417255, 30.74528602417255, 30.03044561440337, 30.03044561440337, 27.885924385095823, 27.885924385095823, 26.456243565557457, 25.741403155788273, 25.026562746019092, 25.026562746019092, 25.026562746019092, 22.167201106942365, 20.022679877634815, 19.30783946786563, 18.592999058096446, 18.592999058096446, 17.16331823855808, 17.16331823855808, 307.38852460484634, 358.1421936984583, 356.71251287891994, 71.49118938101599, 134.39714544070407, 205.16634600785318, 252.34581305261923, 135.82682626024243, 45.75693462932539, 946.4558509384958, 660.5196870308228, 89.36219962524555, 456.0753298368364, 348.13442796168977, 72.92087020055435, 42.18273258047948, 142.26038994816506, 67.20214692240089, 108.6628906890135, 561.8717104826754, 75.78023183963109, 242.3380473158507, 147.26427281654935, 278.794908214079, 70.06150856147762, 67.20214692240089, 101.48420480059177, 61.8091529275689, 41.97162699105746, 38.91969992390186, 35.86777285674625, 35.86777285674625, 32.81584578959065, 29.76391872243504, 189.2271079813154, 26.711991655279437, 26.711991655279437, 25.949009888490536, 22.89708282133493, 19.845155754179327, 16.79322868702372, 15.267265153445917, 15.267265153445917, 14.504283386657017, 14.504283386657017, 14.504283386657017, 12.978319853079213, 12.978319853079213, 12.215338086290311, 12.215338086290311, 12.215338086290311, 11.45235631950141, 11.45235631950141, 9.926392785923607, 9.926392785923607, 9.926392785923607, 424.2254921522971, 163.28572791049277, 9.926392785923607, 9.926392785923607, 9.926392785923607, 605.8151526480555, 168.62660027801508, 289.1777194306615, 196.8569256492044, 60.2831893939911, 32.81584578959065, 945.3420388691165, 77.06878826334693, 1010.9584708129622, 634.8084597860338, 1453.4878955505249, 103.01016833416956, 33.578827556379544, 527.2280306687987, 171.6785273451707, 71.72791589582461, 40.44566345747966, 107.58805893490297, 34.34180932316845, 39.68268169069076, 74.77984296298023, 60.2831893939911, 61.04617116078, 123.6106760374699, 56.468280560046594, 56.468280560046594, 40.44566345747966, 160.69646769855592, 106.52189713564638, 92.74870122982192, 66.1205224785613, 62.447670237008104, 55.10196575390172, 49.59268739157194, 48.67447433118364, 45.00162208963045, 38.574130666912374, 35.81949148574748, 34.90127842535918, 33.06485230458259, 33.06485230458259, 31.22842618380599, 31.22842618380599, 31.22842618380599, 30.310213123417693, 29.392000063029396, 28.473787002641103, 26.637360881864506, 25.719147821476213, 25.719147821476213, 24.800934761087913, 24.800934761087913, 24.800934761087913, 22.964508640311315, 22.964508640311315, 22.964508640311315, 22.046295579923022, 60.61124411623151, 70.71158778050278, 96.42155347137512, 30.310213123417693, 99.17619265254001, 50.51090045196023, 77.13907920322087, 43.16519596885386, 70.71158778050278, 51.42911351234853, 152.43255015506122, 55.10196575390172, 76.22086614283256, 130.3954367057421, 78.97550532399745, 139.57756730962507, 54.18375269351342, 117.54045386030593, 63.3658832973964, 60.61124411623151, 54.18375269351342, 133.150075886907, 114.78581467914105, 75.30265308244427, 62.447670237008104, 56.02017881429002, 56.02017881429002, 478.1986034482627, 394.79325575957387, 390.82157253630294, 247.0466398538964, 169.995985322441, 122.3357866431902, 79.44160783186452, 77.05859789790198, 75.46992460859362, 65.93788487274345, 53.228498558276584, 51.639825268968224, 37.34176566519299, 37.34176566519299, 33.37008244192209, 32.575745797267906, 31.78140915261373, 31.78140915261373, 30.98707250795955, 29.39839921865119, 28.60406257399701, 26.22105264003447, 25.42671599538029, 25.42671599538029, 25.42671599538029, 23.043706061417755, 21.455032772109398, 21.455032772109398, 20.660696127455214, 20.660696127455214, 510.76640587908406, 33.37008244192209, 70.70390474066853, 313.7709180048475, 62.76053829412675, 69.11523145136019, 74.67558796393944, 108.03772703941497, 129.48481644507783, 149.34323256143233, 52.434161913622404, 592.5830802784645, 73.08691467463107, 92.94533079098557, 61.17186500481838, 62.76053829412675, 124.71879657715274, 81.03028112117288, 61.17186500481838, 62.76053829412675, 61.966201649472566, 60.3775283601642, 52.166895388021125, 52.166895388021125, 49.993576596685706, 47.095818208238484, 44.922499416903065, 33.331465863114175, 27.53594908621973, 26.811509489107923, 26.811509489107923, 26.087069891996116, 24.638190697772504, 20.291553115101674, 20.291553115101674, 19.56711351798987, 19.56711351798987, 19.56711351798987, 18.842673920878063, 18.118234323766256, 15.94491553243084, 15.94491553243084, 15.220475935319037, 15.220475935319037, 14.496036338207231, 14.496036338207231, 13.771596741095426, 13.771596741095426, 13.047157143983618, 13.047157143983618, 13.047157143983618, 13.047157143983618, 462.92414695041487, 586.0788784594218, 13.047157143983618, 81.86891886960515, 94.18439202050584, 757.046623377808, 19.56711351798987, 18.842673920878063, 489.72841204355166, 728.0690394933357, 76.79784168982252, 53.61577458224474, 53.61577458224474, 107.22430476851832, 709.2336099684288, 112.29538194830097, 39.12698264000862, 176.77050609125166, 62.3090497475864, 36.22922425156139, 43.47362022267945, 38.40254304289681, 57.23797256780376, 260.8054993562211, 84.76667725805237, 48.5446974024621, 63.033489344698204, 44.922499416903065, 132.57969066743152, 68.10456652448084, 118.81533832230723, 65.93124773314543, 74.6245228984871, 55.06465377646834, 198.5852375617012, 140.93378357015857, 68.06875144195888, 66.46732216441603, 63.26446360933033, 58.460175776701774, 58.460175776701774, 56.85874649915892, 56.058031860387494, 52.054458666530365, 52.054458666530365, 51.25374402775894, 48.050885472673244, 46.44945619513039, 42.44588300127326, 38.44230980741614, 37.641595168644706, 31.235878058473304, 31.235878058473304, 30.43516341970188, 29.634448780930455, 28.833734142159027, 28.833734142159027, 27.23230486461618, 26.43159022584475, 26.43159022584475, 26.43159022584475, 25.630875587073326, 24.830160948301906, 24.830160948301906, 60.06160505424462, 112.10805657438726, 90.48876132755879, 144.9373567640157, 78.4780417459874, 122.5173468784158, 109.70591265807299, 72.87303927458743, 65.66660752564461, 514.0668052376428, 41.645168362501835, 72.87303927458743, 42.44588300127326, 76.87661246844456, 54.456602582844646, 83.28232957861596, 52.8551733053018, 83.28232957861596, 81.68090030107311, 67.26803680318746, 57.47859151547738, 56.680388605944295, 39.11992459621644, 37.52351877715027, 37.52351877715027, 33.53250422948486, 33.53250422948486, 30.33969259135252, 27.146880953220183, 25.550475134154016, 24.75227222462093, 24.75227222462093, 23.155866405554768, 23.155866405554768, 22.35766349602168, 21.559460586488598, 21.559460586488598, 20.761257676955516, 19.963054767422427, 19.164851857889346, 17.56844603882318, 17.56844603882318, 16.770243129290094, 16.770243129290094, 16.770243129290094, 16.770243129290094, 16.770243129290094, 16.770243129290094, 15.972040219757012, 15.173837310223927, 15.173837310223927, 15.173837310223927, 522.0326848637322, 253.03830435108296, 183.59465122170465, 94.99412826353232, 22.35766349602168, 139.69349119738501, 405.495060071902, 157.25395520711288, 142.88630283551734, 471.745901563148, 86.2138962586684, 466.15848119641635, 175.6126221263738, 490.902771391942, 77.43366425380448, 177.20902794543994, 41.514533324815694, 92.59951953493307, 87.81030207773456, 37.52351877715027, 93.39772244446615, 39.11992459621644, 56.680388605944295, 47.90015660108037, 151.66653484038127, 56.680388605944295, 76.63546134427139, 47.10195369154728, 53.487576967811954, 44.70734496294803, 43.110939143881865, 43.19466372308234, 33.77208665205911, 32.98687189614051, 29.060798116547492, 29.060798116547492, 28.275583360628886, 26.70515384879168, 22.77908006919867, 21.993865313280068, 21.993865313280068, 21.993865313280068, 21.993865313280068, 20.42343580144286, 19.638221045524258, 18.853006289605656, 18.067791533687053, 18.067791533687053, 18.067791533687053, 15.712147265931245, 15.712147265931245, 15.712147265931245, 15.712147265931245, 15.712147265931245, 14.92693251001264, 14.92693251001264, 14.92693251001264, 14.92693251001264, 14.141717754094039, 14.141717754094039, 13.356502998175436, 13.356502998175436, 13.356502998175436, 19.638221045524258, 102.87098517289616, 43.979878479000945, 32.201657140221904, 129.56828687412866, 34.55730140797771, 45.55030799083815, 80.88497200717529, 69.10675066839624, 26.70515384879168, 25.91993909287308, 33.77208665205911, 80.09975725125668, 32.201657140221904, 31.4164423843033, 39.26858994348933, 39.26858994348933, 61.254603109210215, 54.97288506186139, 51.04681128226838, 35.34251616389631, 102.08577041697757, 61.254603109210215, 48.691167014512565, 54.97288506186139, 43.979878479000945, 124.07178358269844, 77.74411298350088, 54.97288506186139, 43.979878479000945, 55.75809981777999, 81.67018676309388, 46.33552274675675, 43.19466372308234, 40.839019455326536, 49.47638177043117, 41.62423421124514, 45.55030799083815, 40.053804699407934, 707.7941454827048, 670.0841880385182, 649.7788263378022, 42.79354978425877, 19.587422126297742, 14.511081701118764, 11.610315743873633, 11.610315743873633, 10.159932765251067, 10.159932765251067, 10.159932765251067, 9.434741275939784, 9.434741275939784, 9.434741275939784, 7.984358297317221, 7.259166808005938, 7.259166808005938, 7.259166808005938, 6.5339753186946545, 6.5339753186946545, 6.5339753186946545, 5.808783829383372, 5.808783829383372, 5.808783829383372, 5.808783829383372, 5.808783829383372, 5.808783829383372, 5.808783829383372, 5.808783829383372, 5.808783829383372, 17.411847658363893, 37.7172093590798, 29.740102976655695, 12.335507233184916, 11.610315743873633, 65.99967744221982, 57.29737957048443, 21.037805104920306, 17.411847658363893, 25.388954040788, 10.88512425456235, 8.709549786628502, 13.060698722496198, 15.236273190430046, 9.434741275939784, 13.060698722496198, 16.68665616905261, 14.511081701118764, 14.511081701118764, 14.511081701118764, 15.236273190430046, 11.610315743873633, 10.88512425456235], "Term": ["new", "condition", "free", "size", "shipping", "brand", "description", "brand new", "yet", "description yet", "worn", "never", "price", "good", "used", "firm", "box", "pink", "free shipping", "bundle", "blue", "color", "please", "home", "tags", "smoke", "price firm", "good condition", "black", "never worn", "save shipping", "feel free", "check listings", "prices", "discount", "slime", "ask questions", "free ask", "deals", "bundle items", "questions please", "please feel", "shipping bundle", "check closet", "check items", "ask bundle", "forget", "see listings", "listings bundle", "borax", "save money", "questions feel", "get one", "multiple items", "understand", "bundle discounts", "get free", "floam", "like bundle", "happy bundle", "listings", "bundle save", "check", "save", "ask", "discounts", "bundles", "please ask", "please check", "bundle", "questions", "items", "feel", "combine", "please", "page", "shipping", "closet", "free", "thank", "free shipping", "thanks", "looking", "get", "make", "buy", "item", "oil", "remove", "oils", "soap", "wig", "teeth", "cleansing", "results", "appearance", "types", "conditioner", "benefits", "step", "acne", "serum", "human", "salt", "ideal", "healthy", "essential oils", "fiber", "spinner", "easy clean", "human hair", "hydrating", "spf", "hair", "alcohol", "powerful", "aging", "vitamin", "packets", "facial", "shampoo", "mask", "skin", "natural", "seat", "peel", "minutes", "essential", "henna", "water", "dry", "lines", "heat", "helps", "easy", "face", "bottle", "smooth", "use", "technology", "safe", "hand", "gel", "fresh", "cream", "head", "product", "clean", "features", "soft", "made", "body", "perfect", "free", "one", "includes", "black", "pack", "high", "spandex", "crop", "mesh", "elastic", "bust", "knit", "waistband", "polyester spandex", "crop top", "strapless", "flowy", "rayon", "ties", "flattering", "front back", "cotton polyester", "super comfy", "perfect fall", "soft material", "neckline", "racerback", "wool", "cute comfy", "super comfortable", "machine wash", "maternity", "cotton spandex", "nursing", "black lace", "hits", "super cute", "lace", "polyester", "comfy", "adjustable straps", "bra", "super", "hem", "cute", "stretchy", "fabric", "dress", "fall", "straps", "super soft", "front", "tie", "cotton", "back", "waist", "top", "sexy", "soft", "neck", "length", "material", "size", "fit", "style", "black", "perfect", "comfortable", "pockets", "adjustable", "small", "fits", "wear", "great", "medium", "reasonable", "price negotiable", "rid", "reasonable offers", "though", "moving", "ago", "get rid", "thought", "given", "lol", "need gone", "money back", "know questions", "keep mind", "friend", "worth", "considered", "ways", "problem", "drawer", "gone asap", "closed", "decided", "negotiate price", "least", "ton", "possibly", "changed", "instead", "know want", "got", "gone", "sold separately", "nothing wrong", "let know", "please let", "much", "let", "know", "years", "else", "anymore", "something", "need", "sell", "bought", "negotiable", "someone", "sold", "would", "want", "nothing", "trying", "selling", "one", "many", "really", "love", "like", "purchased", "get", "well", "price", "make", "picture", "use", "used", "little", "two", "also", "great", "wear", "back", "charger", "screen", "cable", "battery", "usb", "ipad", "compatible", "bluetooth", "charging", "apple iphone", "music", "batteries", "device", "screen protector", "led", "headphones", "generation", "tempered", "headset", "tempered glass", "mobile", "adapter", "phone case", "ipad mini", "devices", "carrier", "compatible iphone", "shock", "plug", "android", "sound", "wireless", "iphone", "phone", "charge", "case", "iphone plus", "apple", "power", "plus", "protector", "iphone iphone", "watch", "cord", "glass", "ipod", "samsung", "quality", "scratches", "works", "design", "use", "new", "included", "comes", "high", "shipped", "next day", "rae", "dunn", "rae dunn", "first class", "within hours", "mail", "request", "sales final", "responsible", "buyer", "tracking number", "ship next", "ship day", "ships next", "priority", "please look", "usps first", "ships within", "day next", "service", "bubble wrap", "hrs", "upon request", "update", "cancel", "via usps", "items shipped", "customer", "class", "next", "ratings", "business day", "delivery", "usps", "ships", "tracking", "day", "final", "purchase", "upon", "within", "day shipping", "ship", "days", "business", "receive", "item", "please", "sales", "make sure", "purchasing", "shipping", "items", "first", "care", "free", "sure", "make", "listing", "package", "read", "new", "free shipping", "brand", "makeup", "lip", "palette", "shade", "lipstick", "eyeshadow", "foundation", "kylie", "lashes", "swatched", "shadow", "gloss", "sephora", "cosmetics", "blush", "contour", "mascara", "lips", "eye shadow", "anastasia", "brushes", "primer", "eyeliner", "concealer", "urban decay", "liquid lipstick", "faced", "eyeshadow palette", "bronzer", "nyx", "matte", "tarte", "eye", "liquid", "brush", "shades", "beauty", "kit", "powder", "glow", "full", "color", "new", "colors", "brand", "authentic", "set", "brand new", "used", "light", "ring", "necklace", "bracelet", "earrings", "beads", "sterling silver", "decor", "approximately", "plated", "great gift", "pillow", "wedding", "pendant", "kitty", "charms", "rhinestone", "bed", "measures inches", "necklaces", "tiffany", "pandora", "hello kitty", "bracelets", "japan", "home decor", "ring size", "mother", "scarf", "butterfly", "bead", "charm", "metal", "width", "stone", "sterling", "inches", "chain", "gold", "silver", "rose gold", "jewelry", "gift", "handmade", "heart", "measures", "diamond", "faux", "made", "rose", "vintage", "tall", "set", "new", "style", "beautiful", "included", "great", "piece", "size", "cute", "brand", "perfect", "length", "never worn", "victoria", "victoria secret", "lularoe", "secret pink", "without tags", "llr", "tags never", "lularoe leggings", "irma", "curvy", "carly", "pink victoria", "tall curvy", "tags attached", "secret", "worn tags", "never wore", "size never", "washed per", "without tag", "panty", "per llr", "xxs", "new victoria", "black background", "paisley", "made china", "boyshorts", "tags victoria", "wrong size", "bikini", "tags size", "background", "new tags", "tags", "new without", "leggings", "unicorn", "worn washed", "htf", "worn", "never", "washed", "without", "new", "brand", "brand new", "one size", "pink", "size", "online", "tried", "tag", "new never", "print", "medium", "large", "size large", "black", "small", "one", "good condition", "tears", "signs wear", "noticeable", "good used", "stain", "pre", "pilling", "rips stains", "still good", "fading", "life left", "good shape", "condition stains", "piling", "used good", "stains tears", "size good", "pre owned", "holes stains", "stains rips", "condition rips", "stains holes", "rips tears", "minimal", "see pic", "condition minor", "see pics", "minor scratches", "rips", "good", "mark", "signs", "holes", "stains", "minor", "owned", "used condition", "flaw", "condition", "marks", "wear", "pic", "shows", "tiny", "shown", "left", "still", "gently", "see", "little", "used", "picture", "gently used", "bottom", "shape", "great", "excellent", "great condition", "inside", "small", "one", "size", "long sleeve", "fits like", "jersey", "oversized", "north", "shirt size", "north face", "dri", "dri fit", "short sleeve", "fit small", "sleeve shirt", "like small", "could fit", "like medium", "medium fits", "fit medium", "pants size", "new york", "fit like", "jacket size", "nike air", "loose fit", "tag says", "team", "size youth", "hoodie size", "baseball", "large fits", "size xxl", "nike dri", "basketball", "jacket", "sleeve", "hoodie", "small medium", "sweatshirt", "shirt", "nike", "medium", "size medium", "large", "size large", "small", "says", "fleece", "size small", "fit", "tee", "fits", "runs", "size", "men", "jordan", "warm", "long", "women", "zip", "like", "pants", "tank", "pink", "tag", "top", "new", "black", "nwt", "twice", "worn times", "new condition", "couple times", "worn twice", "condition worn", "worn couple", "size worn", "size women", "new worn", "barely used", "used twice", "handful", "used great", "handful times", "condition flaws", "barely worn", "size great", "shoes size", "worn great", "gently worn", "size excellent", "times great", "used couple", "upper", "soles", "times still", "worn excellent", "condition never", "worn handful", "like new", "times", "great condition", "barely", "perfect condition", "excellent condition", "excellent", "shoes", "heel", "condition", "worn", "couple", "great", "like", "boots", "women size", "women", "condition size", "flaws", "size", "wore", "used", "perfect", "new", "men", "black", "never opened", "original box", "original packaging", "new unopened", "without box", "box never", "new still", "original price", "new box", "comes original", "factory sealed", "eau", "new original", "used comes", "parfum", "still box", "practically", "new factory", "versace", "loss gain", "eau parfum", "still original", "opened used", "sealed box", "box authentic", "box price", "like brand", "still sealed", "box free", "size box", "never used", "opened", "used price", "france", "sealed packaging", "box", "sealed", "new never", "original", "unopened", "factory", "brand new", "new sealed", "brand", "never", "new", "packaging", "comes box", "used", "authentic", "retails", "new package", "comes", "boxes", "perfume", "still", "retail", "package", "size", "full", "price", "without", "game", "bath", "games", "scent", "xbox", "nintendo", "funko", "bath body", "mist", "body works", "pokemon", "funko pop", "movie", "candle", "wars", "mint condition", "disc", "star wars", "strawberry", "books", "pumpkin", "playstation", "wii", "scented", "body lotion", "bomb", "topic", "hot topic", "mario", "pages", "exclusive", "book", "pop", "toys", "cards", "vanilla", "disney", "played", "rare", "lotion", "body", "fragrance", "edition", "works", "card", "set", "star", "includes", "collection", "mint", "limited", "great", "one", "comes", "included", "pack", "super", "smoke", "free home", "smoke free", "pet", "pet free", "purse", "smoke pet", "free pet", "wallet", "comes smoke", "interior", "tote", "hardware", "dust bag", "crossbody", "condition smoke", "friendly home", "spade", "kate spade", "zippered", "home free", "slots", "exterior", "handbag", "wristlet", "free smoke", "velcro", "bradley", "vera bradley", "shoulder strap", "home", "pet smoke", "kors", "bag", "michael kors", "michael", "coach", "strap", "pocket", "leather", "backpack", "free", "zipper", "pockets", "closure", "shoulder", "comes", "inside", "zip", "authentic", "black", "color", "note note", "firm price", "shipping price", "firm free", "new free", "firm unless", "edge edge", "cases cases", "shipping included", "firm please", "new price", "price includes", "ask lower", "included price", "phone size", "unless bundled", "galaxy note", "firm bundle", "galaxy edge", "shipping firm", "size price", "condition price", "dont ask", "edge samsung", "ship today", "samsung note", "edge plus", "size free", "shipping takes", "note edge", "price firm", "firm", "available phone", "edge", "free ship", "price", "price free", "free fast", "free shipping", "shipping", "cases", "fast shipping", "trades", "fast", "free", "note", "retail price", "ship", "galaxy", "fast free", "samsung galaxy", "today", "samsung", "new", "plus", "holds", "retail", "unless", "brand new", "quality", "brand", "iphone", "size", "bundle", "jeans", "american", "eagle", "american eagle", "skinny", "brandy", "old navy", "outfitters", "denim", "melville", "brandy melville", "urban outfitters", "size months", "gap", "lauren", "ralph", "ralph lauren", "abercrombie", "jeans size", "american apparel", "russe", "charlotte russe", "charlotte", "calvin", "klein", "newborn", "pacsun", "skinny jeans", "calvin klein", "onesie", "socks", "months", "pairs", "shorts", "girl", "baby", "pair", "forever", "old", "size", "polo", "pants", "hollister", "navy", "urban", "dress", "short", "tags", "bundle", "one", "teal", "navy blue", "dot", "stripes", "light blue", "light pink", "blue white", "polka", "white black", "scott", "dark brown", "dark blue", "kendra", "black grey", "kendra scott", "dots", "stars", "color pink", "white blue", "black gray", "green color", "baby blue", "blue color", "lime green", "pink color", "blue black", "mint green", "blue pink", "black color", "grey white", "royal blue", "dark grey", "blue", "green", "grey", "black white", "red white", "gray", "white", "dark", "purple", "pink", "yellow", "color", "red", "black", "orange", "light", "bright", "brown", "navy", "hot pink", "colors", "colored", "hot", "mint", "size", "nwt", "one", "print", "small", "beautiful", "cream", "yeti", "trainer", "days arrive", "take around", "mailers", "days take", "around business", "order ship", "get order", "long take", "waist trainer", "take get", "tumbler", "yeah", "fedex", "chart", "choose color", "clearance", "reading", "pale pink", "purple pink", "postage", "come new", "item come", "new clearance", "vacuum", "rambler", "size chart", "write message", "click", "choice color", "corset", "new item", "order", "choose", "self", "available", "arrive", "cup", "take", "sizes", "choice", "color size", "within days", "days", "stainless", "stainless steel", "stickers", "steel", "waist", "product", "business", "pcs", "color", "high", "come", "quality", "within", "size", "pink", "colors", "around", "item", "new", "buy", "message", "purple", "free", "ship", "shipping", "brand new", "description", "yet", "description yet", "hard find", "nail polish", "body jewelry", "surgical steel", "patches", "alex ani", "ani", "alex", "\u2764\ufe0fplease", "opi", "binding", "contact leaving", "logo waistband", "halloween costume", "needle", "navel", "rare hard", "leaving bad", "glaze", "button ring", "colors brands", "littlest", "great halloween", "littlest pet", "shipping\u2022", "pet shop", "\u2764\ufe0fplease contact", "surgical", "nail", "polish", "belly button", "gloves", "hard", "find", "costume", "belly", "halloween", "piercing", "gauge", "rings", "reserved", "new full", "brands", "jewelry", "steel", "button", "logo", "body", "colors", "listing"], "Total": [3861.0, 1894.0, 2112.0, 3467.0, 1676.0, 2078.0, 707.0, 1796.0, 670.0, 649.0, 1382.0, 1207.0, 1097.0, 821.0, 1451.0, 612.0, 755.0, 937.0, 848.0, 914.0, 609.0, 991.0, 898.0, 540.0, 769.0, 478.0, 469.0, 492.0, 1215.0, 475.0, 153.36400712515524, 152.5941419548944, 141.04616440098218, 113.33101827159277, 102.55290588794136, 97.93371486637648, 96.39398452585483, 81.76654629089934, 57.13086084255323, 55.59113050203161, 55.59113050203161, 43.27328777785854, 40.19382709681528, 37.88423158603283, 33.26504056446794, 30.185579883424676, 30.185579883424676, 26.3362540321206, 24.79652369159897, 24.026658521338156, 23.256793351077338, 20.94719784029489, 20.94719784029489, 20.94719784029489, 20.94719784029489, 20.94719784029489, 19.40746749977326, 18.637602329512443, 18.637602329512443, 18.637602329512443, 325.9980002914713, 244.20572764265913, 395.8975441659419, 421.32357488202166, 392.3523656058288, 72.87397923622854, 119.3243239664724, 108.03044365279274, 72.20141992354763, 914.3861046389974, 320.67258643253007, 636.7573053574574, 226.2574679220263, 75.7723815434981, 898.6806542718721, 97.35099228981616, 1676.7630854057938, 133.80666356685782, 2112.7533534730132, 216.40133429968463, 848.6982517560982, 176.8600988547917, 162.08291092800604, 424.89339577558917, 378.76202862827074, 263.5478050511823, 411.68645260900894, 139.18100652392175, 47.82489593948218, 46.831894737477405, 36.90188271742963, 31.936876707405744, 31.936876707405744, 30.943875505400968, 29.95087430339619, 29.95087430339619, 28.957873101391414, 27.964871899386637, 27.964871899386637, 27.964871899386637, 27.964871899386637, 26.97187069738186, 25.97886949537708, 25.97886949537708, 24.985868293372302, 22.00686468735797, 21.013863485353195, 21.013863485353195, 20.020862283348418, 20.020862283348418, 19.02786108134364, 19.02786108134364, 19.02786108134364, 210.292519941799, 18.03485987933886, 18.03485987933886, 18.03485987933886, 18.03485987933886, 18.03485987933886, 39.769555846484245, 36.67186328221375, 80.29478303917394, 301.4163740138927, 148.49216431185545, 28.76558653815806, 27.735922124532294, 27.741735867642674, 51.17669678651948, 38.51576274485009, 169.7772092791367, 138.93272423877877, 34.22903098885704, 50.35135547917859, 67.91676875804572, 181.49927468390206, 178.81107642972174, 118.11197506290753, 92.05831074122622, 389.95082512280663, 55.3299327282562, 79.79079363038639, 134.18868182870017, 88.05843214827097, 70.34647656999071, 189.1011428994898, 62.438889639061784, 187.08166360313595, 195.6782589190588, 157.26476099666198, 387.9278777447204, 319.142837450007, 287.95364785626987, 625.253059976305, 2112.7533534730132, 1025.0702686644577, 323.2025844876236, 1215.0591908842396, 153.02107868042853, 352.2780062637188, 86.07915978946829, 59.52258767518352, 59.52258767518352, 56.398285073502954, 41.557847715520296, 40.77677206510015, 38.43354511383973, 36.09031816257931, 28.27956165837791, 27.49848600795777, 27.49848600795777, 25.155259056697354, 24.374183406277215, 24.374183406277215, 23.593107755857073, 22.812032105436934, 22.030956455016796, 20.46880515417651, 19.687729503756373, 17.344502552495953, 16.56342690207581, 16.56342690207581, 16.56342690207581, 15.78235125165566, 15.78235125165566, 15.78235125165566, 15.00127560123552, 14.22019995081538, 14.22019995081538, 14.22019995081538, 245.47163645211305, 151.17438179079357, 134.28234644738023, 101.88563128243261, 29.942308033843112, 167.37295271971448, 558.9150243173087, 36.820321143190334, 624.4888232083649, 121.71716722465229, 102.78675131567415, 317.46778297066146, 92.86200837930001, 131.17069461548002, 69.91409565842889, 277.60047742184923, 68.55539160663612, 204.8003604923119, 505.3533182964805, 212.705081409827, 532.6408095462847, 82.33871774272002, 387.9278777447204, 94.23876196748172, 215.173906232107, 274.653568601977, 3467.625687652617, 560.7740440577912, 274.0840581101196, 1215.0591908842396, 625.253059976305, 155.34649153560636, 184.28996862382937, 142.3663945219689, 885.4891775450253, 320.3860073466225, 501.1604102718297, 1173.7430584647977, 623.9124181404517, 55.836248759449965, 48.24414732542308, 36.43421176138128, 35.590644935378286, 33.90351128337231, 27.15497667534842, 26.311409849345434, 25.46784302334245, 21.250008893327518, 21.250008893327518, 21.250008893327518, 19.562875241321546, 19.562875241321546, 19.562875241321546, 18.71930841531856, 17.875741589315574, 17.875741589315574, 17.875741589315574, 17.03217476331259, 13.657907459300636, 13.657907459300636, 13.657907459300636, 13.657907459300636, 12.814340633297649, 12.814340633297649, 12.814340633297649, 12.814340633297649, 11.970773807294663, 11.970773807294663, 11.970773807294663, 11.970773807294663, 127.06909331562731, 35.54528101890838, 11.970773807294663, 28.9167565617397, 147.93562791380813, 35.473940795209515, 120.7651351563002, 173.28473355544926, 213.99714822729396, 41.79419765494043, 36.98297196441557, 63.8283895376839, 46.78563706766953, 226.66378508054274, 119.06831110797003, 255.36878605389137, 78.05578842843232, 41.50528894116391, 167.2260647080536, 200.1959446563676, 248.80952240342774, 65.89684554873564, 34.774155624195615, 204.28319985722482, 1025.0702686644577, 121.47495311322353, 149.54797293914913, 230.51277297392232, 899.2119088076577, 174.89227402476473, 424.89339577558917, 200.69833176117373, 1097.1020653927305, 378.76202862827074, 238.3344055043157, 389.95082512280663, 1451.6013515655395, 314.8773451917966, 317.37190939401955, 362.4550728504096, 1173.7430584647977, 501.1604102718297, 505.3533182964805, 164.94354665593957, 119.51696217460429, 116.84481014629044, 97.24902860532231, 94.57687657700848, 92.79544189146593, 70.52750832218393, 61.62033489447115, 50.931726781215794, 50.931726781215794, 50.931726781215794, 44.69670538181684, 39.352401325189156, 37.5709666396466, 36.68024929687532, 35.78953195410404, 35.78953195410404, 34.008097268561485, 32.22666258301893, 32.22666258301893, 32.22666258301893, 29.554510554705082, 28.663793211933804, 28.663793211933804, 25.99164118361997, 25.10092384084869, 25.10092384084869, 25.10092384084869, 24.21020649807741, 24.21020649807741, 24.21020649807741, 42.78860478065343, 515.0063823371521, 271.7683713836628, 61.29805940524368, 445.92307897059356, 147.59212061854302, 151.14862706904603, 84.13807159501948, 373.6274711492882, 79.0388873242318, 107.41459916041211, 102.31324337687546, 67.70156379252434, 101.32265737467739, 74.83823348590539, 120.63249775184636, 299.55192899635574, 154.40922402588387, 253.6438791312647, 209.6116218188311, 389.95082512280663, 3861.344762129447, 362.6822541605191, 574.8630590814292, 352.2780062637188, 141.12667210406025, 100.61924858083971, 94.94820928758884, 94.94820928758884, 94.13806081712443, 48.76974647111747, 47.95959800065306, 43.908855648331006, 41.478410236937776, 41.478410236937776, 39.85811329600895, 39.04796482554455, 34.18707400275809, 33.376925532293676, 31.756628591364855, 30.946480120900446, 30.136331650436038, 26.085589298113984, 25.27544082764958, 25.27544082764958, 24.465292357185167, 24.465292357185167, 23.655143886720758, 23.655143886720758, 22.84499541625635, 22.034846945791937, 21.22469847532753, 20.41455000486312, 20.41455000486312, 18.7942530639343, 58.57209698899726, 169.02090306484382, 33.45749440460054, 57.55671107349695, 30.906196820696852, 102.70062198859164, 162.23457716359312, 144.14073632338034, 428.36488211212736, 62.58103907930341, 281.7245361079559, 35.36292793630323, 158.36379891017643, 64.0486029228408, 502.1730155370698, 265.4523732361148, 162.19013783158624, 119.47735537053481, 411.68645260900894, 898.6806542718721, 54.8481012327968, 68.3979005418838, 96.74437209078653, 1676.7630854057938, 636.7573053574574, 159.9604772049389, 114.1352307037008, 2112.7533534730132, 146.8467604290663, 378.76202862827074, 254.55400981743065, 248.41109845924825, 120.30647339973399, 3861.344762129447, 848.6982517560982, 2078.403360830447, 249.55247585826817, 218.26990110516522, 189.5942075814875, 133.9807413537489, 125.29113725566472, 109.64984987911325, 104.43608742026277, 83.58103758486078, 79.23623553581871, 77.49831471620188, 75.76039389658504, 70.54663143773455, 67.93975020830929, 67.07078979850088, 63.59494815926721, 62.725987749458795, 60.98806692984196, 57.5122252906083, 53.16742324156621, 52.2984628317578, 52.2984628317578, 50.560542012140964, 46.21573996309889, 45.346779553290474, 43.60885873367364, 41.001977504248394, 41.001977504248394, 34.050294225781066, 34.050294225781066, 33.18133381597266, 192.7350666051619, 49.81562239452891, 161.7918102484482, 106.42141495983842, 175.31534432579434, 75.12357639410705, 127.52319665076536, 99.34434116030698, 106.17130878214405, 80.0242779450968, 330.90103359426746, 991.0312254775216, 3861.344762129447, 310.8713602642306, 2078.403360830447, 469.0416049355142, 430.6214362734546, 1796.067245161999, 1451.6013515655395, 417.9301331266114, 143.87316642559725, 133.29311772505653, 104.19798379856965, 88.32791074775861, 57.46943537118159, 48.65272812073101, 48.65272812073101, 42.48103304541561, 41.59936232037055, 40.7176915953255, 38.07267942019032, 37.191008695145264, 35.427667245055154, 28.3743014446947, 28.3743014446947, 27.49263071964965, 26.610959994604592, 23.96594781946942, 23.084277094424362, 22.202606369379303, 22.202606369379303, 21.320935644334245, 19.557594194244132, 19.557594194244132, 18.675923469199073, 18.675923469199073, 17.79425274415402, 17.79425274415402, 16.91258201910896, 16.030911294063895, 55.61875984070059, 86.59170915084715, 44.125685537249566, 53.799707961840056, 64.97533836910323, 272.298069283895, 81.10820670775269, 453.4574988905805, 275.76688963905, 73.16156400867965, 102.4613346683383, 256.75685714660506, 74.177888865676, 79.68288739007862, 88.79790083425813, 46.40219659248815, 64.18278108591268, 319.142837450007, 143.56518641245677, 131.6350061611176, 115.65514376769922, 430.6214362734546, 3861.344762129447, 274.0840581101196, 287.95317171158695, 362.6822541605191, 1173.7430584647977, 159.49217688617784, 3467.625687652617, 624.4888232083649, 2078.403360830447, 625.253059976305, 215.173906232107, 475.5909424648021, 312.86932941278377, 302.9645355748348, 181.98455369702992, 131.753099233146, 82.22913004340126, 46.854866336440736, 43.31743996574468, 35.53510195021337, 34.82761667607416, 31.997675579517328, 30.582705031238905, 29.8752197570997, 29.8752197570997, 27.04527866054286, 326.63449989276717, 26.337793386403646, 24.922822838125224, 22.800367015707593, 21.385396467429178, 20.677911193289965, 19.970425919150752, 19.970425919150752, 19.262940645011547, 17.847970096733125, 17.140484822593915, 16.432999548454706, 16.432999548454706, 15.018029000176275, 15.018029000176275, 15.018029000176275, 49.158757429646116, 58.547548181196966, 63.580615818611946, 375.15302635748935, 769.1421791914712, 114.91238389971785, 278.75197386947843, 73.41254895036381, 49.751003653667325, 27.958090327332467, 1382.7128326713114, 1207.2103328864825, 160.9285456343162, 229.18804938721854, 3861.344762129447, 2078.403360830447, 1796.067245161999, 108.17202813061587, 937.764309941684, 3467.625687652617, 82.61099116193438, 62.45977021797002, 245.9905184586741, 396.86868952540647, 211.8486549676285, 623.9124181404517, 557.199456906229, 222.3202923231893, 1215.0591908842396, 885.4891775450253, 1025.0702686644577, 492.2099573281327, 88.7908037927033, 73.5097752496946, 65.10520955103982, 58.22874670668589, 57.46469527953546, 55.93659242523459, 55.17254099808415, 45.2398724451285, 45.2398724451285, 44.47582101797807, 37.599358173624154, 35.30720389217285, 31.48694675642069, 31.48694675642069, 30.72289532927025, 30.72289532927025, 30.72289532927025, 30.72289532927025, 27.666689620668517, 27.666689620668517, 27.666689620668517, 26.13858676636764, 23.846432484916345, 23.846432484916345, 22.318329630615477, 21.55427820346504, 20.790226776314604, 18.4980724948633, 127.22232492507936, 821.287170702177, 39.937609498389435, 111.55794240204149, 127.3870829777737, 277.2337116284367, 124.1872529673803, 41.37490069284111, 134.24276259816142, 39.94258512488457, 1894.0941210955395, 68.15875872673367, 501.1604102718297, 122.86186028596063, 58.65967995535709, 49.56750719149972, 177.4339513947157, 156.2908036161367, 375.9731574464539, 157.49569662498868, 365.67732801316197, 314.8773451917966, 1451.6013515655395, 238.3344055043157, 101.7097035014119, 202.26979961670892, 161.67810821629388, 1173.7430584647977, 360.8455044646215, 454.66478539333076, 238.98089527502913, 885.4891775450253, 1025.0702686644577, 3467.625687652617, 108.037942159225, 76.5737868503998, 48.106217761462695, 42.86219187665849, 36.86901943688226, 35.3707263269382, 33.872433216994146, 30.87584699710604, 30.126700442134016, 30.126700442134016, 30.126700442134016, 25.631821112301843, 24.133528002357785, 24.133528002357785, 24.133528002357785, 23.38438144738576, 22.63523489241373, 21.136941782469673, 20.387795227497644, 20.387795227497644, 19.638648672525616, 18.140355562581558, 17.39120900760953, 17.39120900760953, 17.39120900760953, 16.6420624526375, 16.6420624526375, 16.6420624526375, 15.89291589766546, 15.143769342693433, 15.143769342693433, 15.143769342693433, 155.05447210997792, 207.4123532406264, 85.24304643531575, 44.54082599133542, 48.396492281907335, 316.93608296416176, 264.64267157684816, 623.9124181404517, 288.26778440763314, 557.199456906229, 222.3202923231893, 885.4891775450253, 77.63075544256628, 63.38173088908361, 331.6546961642908, 560.7740440577912, 89.73883486488297, 320.3860073466225, 58.15282311793459, 3467.625687652617, 208.7009881957933, 46.97970360925936, 131.9057652709894, 386.581511011328, 309.33131362635345, 142.7239889750042, 899.2119088076577, 157.7237578150131, 133.03212418119986, 937.764309941684, 245.9905184586741, 532.6408095462847, 3861.344762129447, 1215.0591908842396, 274.35741785276326, 140.9843989677819, 128.1172715919366, 123.11338872355232, 75.93392167878626, 69.5003579908636, 63.06679430294095, 53.773868975941575, 46.625464878249744, 44.48094364894219, 42.336422419634644, 33.75833750240445, 32.32865668286608, 30.898975863327728, 30.898975863327728, 30.184135453558547, 30.184135453558547, 28.039614224251, 28.039614224251, 26.609933404712635, 25.89509299494345, 25.18025258517427, 25.18025258517427, 25.18025258517427, 22.320890946097542, 20.176369716789992, 19.461529307020808, 18.746688897251623, 18.746688897251623, 17.317008077713258, 17.317008077713258, 333.9314103734388, 391.19498975172996, 454.66478539333076, 80.81349634597638, 160.52858380297403, 267.972252873344, 360.8455044646215, 179.07245374657063, 51.258984458533604, 1894.0941210955395, 1382.7128326713114, 128.09872754008285, 1173.7430584647977, 899.2119088076577, 104.78892177694598, 49.82788796935493, 309.33131362635345, 103.2662538376265, 224.77414263366973, 3467.625687652617, 140.6736123670963, 1451.6013515655395, 625.253059976305, 3861.344762129447, 208.7009881957933, 1215.0591908842396, 101.63741322617675, 61.962361353153874, 42.12483541664243, 39.07290834948683, 36.02098128233122, 36.02098128233122, 32.96905421517562, 29.91712714802002, 190.2492768167088, 26.865200080864415, 26.865200080864415, 26.102218314075515, 23.050291246919908, 19.998364179764305, 16.9464371126087, 15.420473579030888, 15.420473579030888, 14.657491812241988, 14.657491812241988, 14.657491812241988, 13.131528278664184, 13.131528278664184, 12.368546511875282, 12.368546511875282, 12.368546511875282, 11.605564745086381, 11.605564745086381, 10.079601211508578, 10.079601211508578, 10.079601211508578, 433.7096674101689, 170.39061961454507, 10.079601211508578, 10.079601211508578, 10.079601211508578, 755.547642835084, 200.06238345670303, 396.86868952540647, 274.65152034673986, 74.33976437651073, 37.553362778078224, 1796.067245161999, 100.93200933815186, 2078.403360830447, 1207.2103328864825, 3861.344762129447, 158.38003846382497, 41.59528048942552, 1451.6013515655395, 469.0416049355142, 151.15824105385502, 63.23840065551936, 574.8630590814292, 49.588395130373584, 71.78900885703993, 375.9731574464539, 217.62018568533531, 248.41109845924825, 3467.625687652617, 330.90103359426746, 1097.1020653927305, 229.18804938721854, 160.8481238112049, 106.67355324829536, 92.9003573424709, 66.27217859121028, 62.599326349657076, 55.253621866550695, 49.744343504220915, 48.82613044383261, 45.153278202279424, 38.725786779561346, 35.97114759839645, 35.05293453800815, 33.21650841723156, 33.21650841723156, 31.380082296454976, 31.380082296454976, 31.380082296454976, 30.46186923606668, 29.543656175678382, 28.62544311529009, 26.789016994513492, 25.8708039341252, 25.8708039341252, 24.9525908737369, 24.9525908737369, 24.9525908737369, 23.1161647529603, 23.1161647529603, 23.1161647529603, 22.19795169257201, 61.653617571651765, 76.97159937692987, 110.69414547058739, 31.343539961111734, 118.39192823688931, 58.605923011151, 98.28467571036478, 48.66521207155588, 89.96534629491045, 61.51078164504527, 287.95364785626987, 72.01196334391689, 115.47574069686026, 253.6438791312647, 150.86323588968676, 430.6214362734546, 82.18413658350975, 323.2025844876236, 118.66218619337944, 109.43615045128567, 98.90713817686711, 1173.7430584647977, 1025.0702686644577, 574.8630590814292, 362.6822541605191, 153.02107868042853, 558.9150243173087, 478.35149832506903, 394.9461506363802, 390.97446741310927, 247.19953473070274, 170.14888019924732, 122.48868151999653, 79.59450270867084, 77.2114927747083, 75.62281948539994, 66.09077974954978, 53.3813934350829, 51.79272014577454, 37.4946605419993, 37.4946605419993, 33.5229773187284, 32.72864067407422, 31.934304029420055, 31.934304029420055, 31.139967384765875, 29.551294095457514, 28.756957450803334, 26.373947516840794, 25.579610872186617, 25.579610872186617, 25.579610872186617, 23.19660093822408, 21.607927648915723, 21.607927648915723, 20.81359100426154, 20.81359100426154, 540.9956182024109, 34.287028745878835, 76.46180208887485, 413.55018612398493, 70.92057955864733, 79.6018934827349, 87.07419926555089, 139.53424300765184, 180.10311500437763, 232.24252972714794, 66.65111285330886, 2112.7533534730132, 114.52496113898552, 184.28996862382937, 96.13884927268205, 116.02657739950261, 574.8630590814292, 238.98089527502913, 142.7239889750042, 469.0416049355142, 1215.0591908842396, 991.0312254775216, 52.320489235302865, 52.320489235302865, 50.14717044396745, 47.249412055520224, 45.076093264184806, 33.485059710395916, 27.689542933501478, 26.96510333638967, 26.96510333638967, 26.240663739277863, 24.791784545054252, 20.445146962383422, 20.445146962383422, 19.72070736527162, 19.72070736527162, 19.72070736527162, 18.99626776815981, 18.271828171048003, 16.09850937971259, 16.09850937971259, 15.374069782600781, 15.374069782600781, 14.649630185488975, 14.649630185488975, 13.92519058837717, 13.92519058837717, 13.200750991265362, 13.200750991265362, 13.200750991265362, 13.200750991265362, 469.52695015282507, 612.6652389193375, 13.200750991265362, 88.97352113092035, 112.97140068846903, 1097.1020653927305, 20.483689132060523, 19.796982406931235, 848.6982517560982, 1676.7630854057938, 109.01725987687034, 70.74220300907551, 74.7025568729287, 182.9019306811388, 2112.7533534730132, 210.26662744496375, 53.777230056279485, 502.1730155370698, 112.3428147900598, 52.58578750813133, 73.9116037241847, 59.93627908463102, 120.63249775184636, 3861.344762129447, 373.6274711492882, 108.14824450785987, 217.62018568533531, 93.63582899373066, 1796.067245161999, 299.55192899635574, 2078.403360830447, 515.0063823371521, 3467.625687652617, 914.3861046389974, 198.73806865856636, 141.08661466702372, 68.22158253882404, 66.62015326128119, 63.41729470619547, 58.61300687356692, 58.61300687356692, 57.01157759602406, 56.21086295725264, 52.20728976339551, 52.20728976339551, 51.406575124624084, 48.203716569538386, 46.60228729199553, 42.5987140981384, 38.59514090428128, 37.79442626550985, 31.388709155338457, 31.388709155338457, 30.587994516567033, 29.78727987779561, 28.98656523902418, 28.98656523902418, 27.385135961481332, 26.584421322709904, 26.584421322709904, 26.584421322709904, 25.78370668393848, 24.98299204516706, 24.98299204516706, 65.45846203591398, 142.6777062448374, 113.62891776463222, 211.86239291585295, 101.6682589060278, 203.19053467247895, 218.6500225471937, 123.60787868154068, 110.64678169374426, 3467.625687652617, 53.78434433891944, 157.7237578150131, 56.08335208763492, 179.68103443274765, 97.90333417009136, 317.46778297066146, 105.988201597423, 769.1421791914712, 914.3861046389974, 1025.0702686644577, 57.631447729634914, 56.833244820101825, 39.27278081037397, 37.6763749913078, 37.6763749913078, 33.68536044364239, 33.68536044364239, 30.492548805510054, 27.299737167377717, 25.70333134831155, 24.905128438778465, 24.905128438778465, 23.308722619712302, 23.308722619712302, 22.510519710179214, 21.712316800646132, 21.712316800646132, 20.91411389111305, 20.11591098157996, 19.31770807204688, 17.721302252980713, 17.721302252980713, 16.92309934344763, 16.92309934344763, 16.92309934344763, 16.92309934344763, 16.92309934344763, 16.92309934344763, 16.124896433914547, 15.32669352438146, 15.32669352438146, 15.32669352438146, 609.1141948415454, 295.6834532699872, 213.96003869844287, 109.20634618525237, 23.274571137329648, 184.5022506999924, 634.3613002486028, 226.6040537220862, 204.46830579940902, 937.764309941684, 120.68258878627066, 991.0312254775216, 322.8653425221421, 1215.0591908842396, 115.04288344911116, 417.9301331266114, 55.87710870698273, 193.5315618275885, 179.68103443274765, 50.23981108600545, 310.8713602642306, 59.91634354822028, 133.7871450568507, 109.43615045128567, 3467.625687652617, 274.35741785276326, 1025.0702686644577, 211.8486549676285, 885.4891775450253, 287.95317171158695, 189.1011428994898, 43.34764981877602, 33.925072747752786, 33.139857991834184, 29.21378421224117, 29.21378421224117, 28.428569456322563, 26.85813994448536, 22.932066164892348, 22.146851408973745, 22.146851408973745, 22.146851408973745, 22.146851408973745, 20.576421897136537, 19.791207141217935, 19.005992385299333, 18.22077762938073, 18.22077762938073, 18.22077762938073, 15.865133361624922, 15.865133361624922, 15.865133361624922, 15.865133361624922, 15.865133361624922, 15.079918605706318, 15.079918605706318, 15.079918605706318, 15.079918605706318, 14.294703849787716, 14.294703849787716, 13.509489093869114, 13.509489093869114, 13.509489093869114, 21.289500251161993, 155.76115906935843, 56.04887899875194, 41.171350486366144, 297.7437928811853, 49.29295997203076, 86.71431696688745, 227.5811542425297, 176.42827138507192, 37.7811531665379, 36.449563012496846, 57.419378391220675, 265.4523732361148, 53.514740636996954, 51.8478551560333, 81.75114533798077, 82.1388690268505, 212.705081409827, 187.08166360313595, 162.19013783158624, 71.22559935049657, 991.0312254775216, 352.2780062637188, 197.02171331615875, 299.55192899635574, 158.36379891017643, 3467.625687652617, 937.764309941684, 310.8713602642306, 162.57780281910425, 411.68645260900894, 3861.344762129447, 263.5478050511823, 213.68844870305543, 204.46830579940902, 2112.7533534730132, 502.1730155370698, 1676.7630854057938, 1796.067245161999, 707.9477318110646, 670.2377743668779, 649.932412666162, 42.94713611261852, 19.741008454657493, 14.664668029478515, 11.763902072233384, 11.763902072233384, 10.313519093610818, 10.313519093610818, 10.313519093610818, 9.588327604299534, 9.588327604299534, 9.588327604299534, 8.13794462567697, 7.412753136365689, 7.412753136365689, 7.412753136365689, 6.687561647054405, 6.687561647054405, 6.687561647054405, 5.9623701577431225, 5.9623701577431225, 5.9623701577431225, 5.9623701577431225, 5.9623701577431225, 5.9623701577431225, 5.9623701577431225, 5.9623701577431225, 5.9623701577431225, 18.55843518872842, 44.73873331201707, 35.40637460920523, 14.059523073381872, 13.365331349776234, 106.54829893410404, 120.07587187739088, 35.26385621186365, 27.773225813665483, 50.01884957531889, 17.2104056582375, 11.53528814330209, 26.457439161984247, 37.04903432269211, 14.16621820503294, 36.741348033850436, 102.4613346683383, 82.1388690268505, 132.65825003129632, 142.89930200084305, 287.95364785626987, 310.8713602642306, 254.55400981743065], "loglift": [30.0, 29.0, 28.0, 27.0, 26.0, 25.0, 24.0, 23.0, 22.0, 21.0, 20.0, 19.0, 18.0, 17.0, 16.0, 15.0, 14.0, 13.0, 12.0, 11.0, 10.0, 9.0, 8.0, 7.0, 6.0, 5.0, 4.0, 3.0, 2.0, 1.0, 2.7015, 2.7015, 2.7014, 2.7011, 2.701, 2.7009, 2.7009, 2.7006, 2.6998, 2.6997, 2.6997, 2.6989, 2.6987, 2.6984, 2.6979, 2.6974, 2.6974, 2.6967, 2.6963, 2.6961, 2.6959, 2.6952, 2.6952, 2.6952, 2.6952, 2.6952, 2.6946, 2.6942, 2.6942, 2.6942, 2.6727, 2.6501, 2.6168, 2.6122, 2.5487, 2.6297, 2.5937, 2.5949, 2.616, 2.4317, 2.4666, 2.3232, 2.431, 2.5442, 2.1093, 2.4166, 1.6391, 2.2896, 1.202, 2.0812, 1.3533, 2.086, 2.1236, 1.4196, 1.4241, 1.6626, 1.3186, 2.7116, 2.7095, 2.7094, 2.7085, 2.7079, 2.7079, 2.7078, 2.7076, 2.7076, 2.7074, 2.7072, 2.7072, 2.7072, 2.7072, 2.707, 2.7068, 2.7068, 2.7066, 2.7058, 2.7054, 2.7054, 2.7051, 2.7051, 2.7047, 2.7047, 2.7047, 2.7043, 2.7042, 2.7042, 2.7042, 2.7042, 2.7042, 2.6863, 2.6874, 2.6639, 2.5731, 2.5804, 2.679, 2.6791, 2.6789, 2.6207, 2.6384, 2.4232, 2.4352, 2.6386, 2.548, 2.4765, 2.214, 2.1922, 2.2779, 2.3422, 1.8273, 2.4537, 2.2967, 2.0692, 2.1981, 2.3026, 1.7539, 2.3093, 1.6789, 1.6488, 1.7254, 1.0216, 1.1596, 1.1373, 0.5303, -0.5312, 0.0217, 0.8978, -0.3714, 1.5671, 0.7534, 2.7301, 2.7293, 2.7293, 2.7291, 2.7282, 2.7281, 2.7279, 2.7276, 2.7264, 2.7263, 2.7263, 2.7257, 2.7255, 2.7255, 2.7253, 2.7251, 2.7249, 2.7243, 2.724, 2.723, 2.7226, 2.7226, 2.7226, 2.7221, 2.7221, 2.7221, 2.7216, 2.721, 2.721, 2.721, 2.7182, 2.6975, 2.6908, 2.665, 2.6967, 2.5903, 2.4991, 2.6631, 2.4212, 2.5507, 2.5343, 2.3375, 2.497, 2.4286, 2.5281, 2.2703, 2.5199, 2.2687, 2.031, 2.2309, 2.006, 2.4308, 1.9793, 2.3699, 2.055, 1.9063, 0.482, 1.3866, 1.7156, 0.7713, 1.1766, 1.9931, 1.8533, 2.0483, 0.5585, 1.3403, 0.9408, 0.0897, 0.664, 2.797, 2.7966, 2.7956, 2.7955, 2.7953, 2.7941, 2.794, 2.7938, 2.7926, 2.7926, 2.7926, 2.792, 2.792, 2.792, 2.7916, 2.7912, 2.7912, 2.7912, 2.7908, 2.7886, 2.7886, 2.7886, 2.7886, 2.7878, 2.7878, 2.7878, 2.7878, 2.787, 2.787, 2.787, 2.787, 2.7617, 2.7727, 2.787, 2.762, 2.7019, 2.75, 2.6801, 2.6338, 2.5916, 2.7038, 2.7083, 2.648, 2.6761, 2.4792, 2.5322, 2.4245, 2.5899, 2.6653, 2.3932, 2.3209, 2.1629, 2.5022, 2.6646, 2.1224, 1.6053, 2.2609, 2.1865, 2.0257, 1.5185, 2.1195, 1.7484, 2.0008, 1.032, 1.5526, 1.7712, 1.5, 0.7011, 1.5504, 1.4749, 1.352, 0.4521, 1.1042, 1.0097, 2.8301, 2.8297, 2.8297, 2.8294, 2.8294, 2.8293, 2.8288, 2.8285, 2.828, 2.828, 2.828, 2.8276, 2.8271, 2.8269, 2.8268, 2.8267, 2.8267, 2.8265, 2.8262, 2.8262, 2.8262, 2.8258, 2.8257, 2.8257, 2.8251, 2.8249, 2.8249, 2.8249, 2.8247, 2.8247, 2.8247, 2.8093, 2.6937, 2.6901, 2.774, 2.6288, 2.7045, 2.6875, 2.6137, 2.3095, 2.5939, 2.4575, 2.4694, 2.5778, 2.3737, 2.4431, 2.1853, 1.5774, 1.9662, 1.6221, 1.7139, 1.2491, -0.4375, 1.2992, 0.9359, 1.1136, 2.8419, 2.8415, 2.8414, 2.8414, 2.8414, 2.8399, 2.8398, 2.8395, 2.8393, 2.8393, 2.8392, 2.8391, 2.8385, 2.8384, 2.8382, 2.8381, 2.8379, 2.8372, 2.837, 2.837, 2.8368, 2.8368, 2.8365, 2.8365, 2.8363, 2.8361, 2.8358, 2.8355, 2.8355, 2.8349, 2.8251, 2.7495, 2.8113, 2.7695, 2.8127, 2.7281, 2.6675, 2.6724, 2.5514, 2.7303, 2.4554, 2.7806, 2.5164, 2.6777, 2.2686, 2.3741, 2.4638, 2.4842, 2.1457, 1.8355, 2.6885, 2.612, 2.455, 1.1994, 1.5652, 2.1115, 2.2519, 0.3143, 2.074, 1.4052, 1.678, 1.6714, 2.212, -0.6223, 0.484, -0.5879, 2.8847, 2.8846, 2.8845, 2.8841, 2.8841, 2.8839, 2.8838, 2.8835, 2.8834, 2.8833, 2.8833, 2.8831, 2.883, 2.883, 2.8829, 2.8828, 2.8828, 2.8826, 2.8824, 2.8824, 2.8824, 2.8823, 2.882, 2.8819, 2.8818, 2.8816, 2.8816, 2.8808, 2.8808, 2.8807, 2.868, 2.862, 2.7706, 2.796, 2.6842, 2.7825, 2.6673, 2.635, 2.5797, 2.6801, 1.8631, 1.0931, 0.2194, 1.706, 0.342, 1.3982, 1.3982, 0.0878, 0.1919, 1.2843, 2.9012, 2.9011, 2.9008, 2.9005, 2.8996, 2.8991, 2.8991, 2.8986, 2.8986, 2.8985, 2.8982, 2.8981, 2.8979, 2.8969, 2.8969, 2.8967, 2.8965, 2.8959, 2.8956, 2.8954, 2.8954, 2.8951, 2.8944, 2.8944, 2.8941, 2.8941, 2.8936, 2.8936, 2.8932, 2.8927, 2.8851, 2.8691, 2.8813, 2.8687, 2.8219, 2.6748, 2.7751, 2.5917, 2.5861, 2.7179, 2.6578, 2.4805, 2.6895, 2.5728, 2.5242, 2.7234, 2.585, 1.7729, 2.1663, 2.1869, 2.2305, 1.4207, -0.0314, 1.6056, 1.5444, 1.3372, 0.3624, 1.9385, -0.5874, 0.657, -0.4809, 0.6287, 1.6245, 2.9409, 2.9407, 2.9407, 2.9404, 2.94, 2.9393, 2.9379, 2.9376, 2.9369, 2.9368, 2.9364, 2.9362, 2.936, 2.936, 2.9355, 2.9354, 2.9353, 2.935, 2.9344, 2.934, 2.9337, 2.9335, 2.9335, 2.9332, 2.9325, 2.9322, 2.9318, 2.9318, 2.9309, 2.9309, 2.9309, 2.9049, 2.8822, 2.8738, 2.7726, 2.6295, 2.7339, 2.6174, 2.7765, 2.7992, 2.8756, 2.0542, 2.0825, 2.4559, 2.2906, 1.4012, 1.472, 1.5054, 2.4548, 1.5477, 0.9377, 2.5119, 2.6504, 1.942, 1.6355, 1.9307, 1.1632, 1.1551, 1.7761, 0.4625, 0.6049, 0.206, 2.9415, 2.94, 2.9397, 2.9394, 2.9391, 2.9391, 2.939, 2.939, 2.9384, 2.9384, 2.9383, 2.9377, 2.9374, 2.9369, 2.9369, 2.9368, 2.9368, 2.9368, 2.9368, 2.9362, 2.9362, 2.9362, 2.9359, 2.9353, 2.9353, 2.9349, 2.9346, 2.9344, 2.9335, 2.9327, 2.8514, 2.9174, 2.8854, 2.8559, 2.7972, 2.7854, 2.882, 2.7075, 2.8772, 2.2045, 2.7817, 2.388, 2.6408, 2.7907, 2.8125, 2.4286, 2.4413, 2.1548, 2.4008, 2.0842, 2.1355, 1.5015, 2.2097, 2.528, 2.1338, 2.1823, 0.9278, 1.6595, 1.4052, 1.8692, 0.6741, 0.4131, -0.7163, 2.9777, 2.9771, 2.9759, 2.9755, 2.9749, 2.9747, 2.9745, 2.9741, 2.974, 2.974, 2.974, 2.9731, 2.9727, 2.9727, 2.9727, 2.9725, 2.9723, 2.9718, 2.9715, 2.9715, 2.9712, 2.9706, 2.9702, 2.9702, 2.9702, 2.9698, 2.9698, 2.9698, 2.9694, 2.9689, 2.9689, 2.9689, 2.845, 2.7893, 2.84, 2.883, 2.8714, 2.6778, 2.661, 2.5569, 2.6024, 2.5094, 2.6043, 2.364, 2.7573, 2.7897, 2.4697, 2.3499, 2.6708, 2.3589, 2.7545, 1.6174, 2.3733, 2.792, 2.3301, 1.845, 1.808, 2.2068, 1.172, 2.0484, 2.1565, 0.722, 1.6959, 1.0472, -0.6056, 0.2973, 1.5418, 3.0366, 3.0365, 3.0365, 3.0357, 3.0355, 3.0353, 3.0349, 3.0344, 3.0343, 3.0341, 3.0332, 3.033, 3.0327, 3.0327, 3.0326, 3.0326, 3.0322, 3.0322, 3.0319, 3.0318, 3.0316, 3.0316, 3.0316, 3.0308, 3.0301, 3.0298, 3.0295, 3.0295, 3.0288, 3.0288, 2.9549, 2.9494, 2.7951, 2.9151, 2.86, 2.7707, 2.6801, 2.7613, 2.9242, 2.3439, 2.2989, 2.6776, 2.0924, 2.0888, 2.6751, 2.8712, 2.261, 2.6081, 2.3109, 1.2178, 2.4191, 1.2476, 1.5918, 0.4094, 1.9462, 0.1429, 3.0653, 3.0643, 3.0631, 3.0628, 3.0625, 3.0625, 3.0621, 3.0616, 3.0614, 3.061, 3.061, 3.0609, 3.0601, 3.0591, 3.0577, 3.0568, 3.0568, 3.0563, 3.0563, 3.0563, 3.055, 3.055, 3.0543, 3.0543, 3.0543, 3.0535, 3.0535, 3.0514, 3.0514, 3.0514, 3.0447, 3.0242, 3.0514, 3.0514, 3.0514, 2.8459, 2.8958, 2.7502, 2.7337, 2.8572, 2.9319, 2.425, 2.797, 2.3461, 2.424, 2.0897, 2.6366, 2.8527, 2.054, 2.0617, 2.3213, 2.6198, 1.3909, 2.6994, 2.4739, 1.4518, 1.7831, 1.6633, -0.2673, 1.2986, 0.1, 1.3322, 3.076, 3.0756, 3.0754, 3.0747, 3.0746, 3.0742, 3.0739, 3.0739, 3.0736, 3.0731, 3.0728, 3.0727, 3.0724, 3.0724, 3.0721, 3.0721, 3.0721, 3.072, 3.0718, 3.0717, 3.0713, 3.0711, 3.0711, 3.0709, 3.0709, 3.0709, 3.0704, 3.0704, 3.0704, 3.0701, 3.0599, 2.9922, 2.939, 3.0435, 2.8999, 2.9283, 2.8347, 2.9571, 2.8362, 2.898, 2.4409, 2.8093, 2.6616, 2.4116, 2.4298, 1.9504, 2.6604, 2.0655, 2.4496, 2.4861, 2.4752, 0.9005, 0.8875, 1.0444, 1.3178, 2.0721, 0.7767, 3.1129, 3.1128, 3.1128, 3.1126, 3.1123, 3.112, 3.1113, 3.1112, 3.1112, 3.1109, 3.1103, 3.1103, 3.1091, 3.1091, 3.1086, 3.1085, 3.1084, 3.1084, 3.1083, 3.108, 3.1079, 3.1074, 3.1072, 3.1072, 3.1072, 3.1066, 3.1061, 3.1061, 3.1058, 3.1058, 3.0557, 3.0861, 3.0349, 2.8371, 2.991, 2.972, 2.9596, 2.8574, 2.7832, 2.6717, 2.8733, 1.842, 2.6641, 2.4287, 2.6611, 2.4987, 1.5851, 2.0317, 2.266, 1.1019, 0.1373, 0.3151, 3.1671, 3.1671, 3.1669, 3.1668, 3.1666, 3.1654, 3.1645, 3.1643, 3.1643, 3.1641, 3.1638, 3.1625, 3.1625, 3.1622, 3.1622, 3.1622, 3.1619, 3.1616, 3.1604, 3.1604, 3.16, 3.16, 3.1595, 3.1595, 3.1589, 3.1589, 3.1583, 3.1583, 3.1583, 3.1583, 3.1559, 3.1256, 3.1583, 3.0868, 2.9881, 2.799, 3.1242, 3.1206, 2.6202, 2.3358, 2.8197, 2.8928, 2.8383, 2.636, 2.0785, 2.5428, 2.852, 2.1259, 2.5806, 2.7974, 2.6393, 2.7249, 2.4245, 0.475, 1.6867, 2.369, 1.9309, 2.4355, 0.5638, 1.6888, 0.3082, 1.1144, -0.6687, 0.3603, 3.1937, 3.1934, 3.1922, 3.1922, 3.1921, 3.1919, 3.1919, 3.1918, 3.1918, 3.1916, 3.1916, 3.1915, 3.1913, 3.1912, 3.1909, 3.1905, 3.1904, 3.1896, 3.1896, 3.1895, 3.1893, 3.1892, 3.1892, 3.1889, 3.1887, 3.1887, 3.1887, 3.1885, 3.1884, 3.1884, 3.1084, 2.9534, 2.9668, 2.8149, 2.9356, 2.6886, 2.5048, 2.6661, 2.6727, 1.2856, 2.9387, 2.4224, 2.9159, 2.3455, 2.6079, 1.8563, 2.4987, 0.9714, 0.7791, 0.4707, 3.2206, 3.2206, 3.2194, 3.2192, 3.2192, 3.2187, 3.2187, 3.2183, 3.2177, 3.2173, 3.2171, 3.2171, 3.2167, 3.2167, 3.2165, 3.2162, 3.2162, 3.216, 3.2157, 3.2153, 3.2146, 3.2146, 3.2142, 3.2142, 3.2142, 3.2142, 3.2142, 3.2142, 3.2138, 3.2133, 3.2133, 3.2133, 3.069, 3.0675, 3.0702, 3.0839, 3.1831, 2.9451, 2.7758, 2.8579, 2.8649, 2.5362, 2.887, 2.4691, 2.6143, 2.317, 2.8274, 2.3653, 2.9262, 2.4861, 2.5073, 2.9314, 2.0208, 2.797, 2.3645, 2.3971, 0.0937, 1.6463, 0.6298, 1.7197, 0.4166, 1.3606, 1.7448, 3.3914, 3.3904, 3.3903, 3.3897, 3.3897, 3.3895, 3.3892, 3.3882, 3.388, 3.388, 3.388, 3.388, 3.3875, 3.3872, 3.3868, 3.3865, 3.3865, 3.3865, 3.3852, 3.3852, 3.3852, 3.3852, 3.3852, 3.3847, 3.3847, 3.3847, 3.3847, 3.3842, 3.3842, 3.3835, 3.3835, 3.3835, 3.3142, 2.9801, 3.1524, 3.1492, 2.5629, 3.0398, 2.7511, 2.3604, 2.4577, 3.048, 3.054, 2.8642, 2.1968, 2.887, 2.8939, 2.6617, 2.6569, 2.1501, 2.1702, 2.2389, 2.6942, 1.122, 1.6455, 1.9971, 1.6995, 2.1138, 0.0646, 0.9048, 1.6624, 2.0875, 1.3957, -0.4612, 1.6566, 1.7961, 1.7842, -0.3593, 0.9047, -0.2109, -0.4082, 4.2069, 4.2068, 4.2068, 4.2035, 4.1993, 4.1965, 4.1939, 4.1939, 4.1921, 4.1921, 4.1921, 4.1909, 4.1909, 4.1909, 4.188, 4.1861, 4.1861, 4.1861, 4.1838, 4.1838, 4.1838, 4.181, 4.181, 4.181, 4.181, 4.181, 4.181, 4.181, 4.181, 4.181, 4.1433, 4.0364, 4.0327, 4.0763, 4.0663, 3.7281, 3.4672, 3.6905, 3.7402, 3.529, 3.749, 3.9261, 3.5011, 3.3185, 3.8006, 3.1728, 2.3922, 2.4736, 1.9942, 1.9198, 1.268, 0.9196, 1.055], "logprob": [30.0, 29.0, 28.0, 27.0, 26.0, 25.0, 24.0, 23.0, 22.0, 21.0, 20.0, 19.0, 18.0, 17.0, 16.0, 15.0, 14.0, 13.0, 12.0, 11.0, 10.0, 9.0, 8.0, 7.0, 6.0, 5.0, 4.0, 3.0, 2.0, 1.0, -4.6566, -4.6616, -4.7404, -4.9594, -5.0595, -5.1057, -5.1216, -5.2864, -5.6457, -5.6731, -5.6731, -5.9244, -5.9985, -6.0579, -6.1885, -6.2861, -6.2861, -6.4233, -6.4839, -6.5156, -6.5484, -6.6537, -6.6537, -6.6537, -6.6537, -6.6537, -6.7307, -6.7715, -6.7715, -6.7715, -3.9313, -4.2428, -3.7929, -3.7353, -3.87, -5.4725, -5.0154, -5.1136, -5.4955, -3.1409, -4.1539, -3.6113, -4.5383, -5.519, -3.4807, -5.396, -3.3272, -5.2049, -3.5332, -4.9326, -4.2939, -5.1296, -5.1792, -4.9194, -5.0299, -5.1541, -5.052, -4.7436, -5.8139, -5.8349, -6.0741, -6.2192, -6.2192, -6.251, -6.2837, -6.2837, -6.3176, -6.3527, -6.3527, -6.3527, -6.3527, -6.3891, -6.4268, -6.4268, -6.466, -6.5938, -6.6403, -6.6403, -6.689, -6.689, -6.7403, -6.7403, -6.7403, -4.3381, -6.7943, -6.7943, -6.7943, -6.7943, -6.7943, -6.0215, -6.1015, -5.3413, -4.1093, -4.81, -6.3527, -6.3891, -6.3891, -5.8349, -6.1015, -4.8332, -5.0217, -6.2192, -5.9238, -5.6961, -4.9757, -5.0124, -5.3413, -5.5262, -4.5976, -5.9238, -5.7148, -5.4225, -5.7148, -5.8349, -5.3947, -5.9474, -5.4804, -5.4656, -5.6076, -5.4085, -5.4656, -5.5908, -5.4225, -5.2663, -5.4366, -5.7148, -5.6597, -5.7933, -5.7731, -5.2056, -5.5753, -5.5753, -5.6293, -5.9357, -5.9547, -6.0141, -6.0773, -6.3223, -6.3505, -6.3505, -6.4401, -6.4718, -6.4718, -6.5046, -6.5385, -6.5736, -6.6477, -6.6869, -6.8146, -6.8611, -6.8611, -6.8611, -6.9099, -6.9099, -6.9099, -6.9612, -7.0152, -7.0152, -7.0152, -4.1695, -4.675, -4.8001, -5.102, -6.295, -4.6803, -3.5658, -6.1217, -3.5328, -5.0385, -5.2239, -4.293, -5.3627, -5.0858, -5.6155, -4.4944, -5.6433, -4.8001, -4.1347, -4.8001, -4.107, -5.5493, -4.4508, -5.4752, -4.9644, -4.8691, -3.7576, -4.675, -5.0619, -4.517, -4.7762, -5.3522, -5.3211, -5.3842, -5.0462, -5.2811, -5.2332, -5.2332, -5.2909, -5.5714, -5.718, -5.9998, -6.0233, -6.0721, -6.2952, -6.3269, -6.3597, -6.542, -6.542, -6.542, -6.6253, -6.6253, -6.6253, -6.6697, -6.7162, -6.7162, -6.7162, -6.765, -6.988, -6.988, -6.988, -6.988, -7.0525, -7.0525, -7.0525, -7.0525, -7.1215, -7.1215, -7.1215, -7.1215, -4.7844, -6.0474, -7.1215, -6.2644, -4.6923, -6.0721, -4.917, -4.6021, -4.4333, -5.9544, -6.0721, -5.5867, -5.8692, -4.4882, -5.079, -4.4237, -5.4436, -5.9998, -4.8784, -4.7707, -4.7113, -5.7006, -6.1774, -4.949, -3.8531, -5.3303, -5.1968, -4.9249, -4.0708, -5.1072, -4.5907, -5.0883, -4.3585, -4.9013, -5.146, -4.9249, -4.4093, -5.0883, -5.156, -5.146, -4.8708, -5.0698, -5.156, -4.4552, -4.7777, -4.8004, -4.9842, -5.0121, -5.0312, -5.3061, -5.4414, -5.6324, -5.6324, -5.6324, -5.7634, -5.8912, -5.9377, -5.9618, -5.9865, -5.9865, -6.0378, -6.0919, -6.0919, -6.0919, -6.1788, -6.2096, -6.2096, -6.308, -6.3431, -6.3431, -6.3431, -6.3794, -6.3794, -6.3794, -5.8253, -3.4531, -4.0959, -5.5011, -3.6619, -4.692, -4.6852, -5.3448, -4.1582, -5.427, -5.2567, -5.2935, -5.5979, -5.3988, -5.6324, -5.4128, -5.1112, -5.3851, -5.2329, -5.3317, -5.1757, -4.5696, -5.1982, -5.1008, -5.4128, -4.5993, -4.938, -4.9962, -4.9962, -5.0047, -5.6639, -5.6807, -5.7692, -5.8264, -5.8264, -5.8664, -5.887, -6.0205, -6.0446, -6.0946, -6.1206, -6.1472, -6.2924, -6.3241, -6.3241, -6.3569, -6.3569, -6.3908, -6.3908, -6.4258, -6.4622, -6.4999, -6.5391, -6.5391, -6.6225, -5.4956, -4.5113, -6.0693, -5.5686, -6.1472, -5.0309, -4.6344, -4.7477, -3.7795, -5.5242, -4.2945, -6.0446, -4.8096, -5.5536, -3.9033, -4.4354, -4.8384, -5.1236, -4.225, -3.7545, -5.6978, -5.5536, -5.3638, -3.7669, -4.3693, -5.2044, -5.4016, -4.4208, -5.3275, -5.0488, -5.1733, -5.2044, -5.3888, -4.7544, -5.1632, -5.3394, -3.9866, -4.1206, -4.2615, -4.6091, -4.6762, -4.8097, -4.8585, -5.0816, -5.1351, -5.1573, -5.1801, -5.2515, -5.2892, -5.3021, -5.3555, -5.3693, -5.3974, -5.4563, -5.535, -5.5516, -5.5516, -5.5855, -5.6756, -5.6946, -5.7339, -5.7957, -5.7957, -5.9823, -5.9823, -6.0082, -4.2615, -5.6206, -4.534, -4.9275, -4.5401, -5.2892, -4.8753, -5.1573, -5.1462, -5.3285, -4.726, -4.399, -3.9127, -4.9455, -4.4096, -4.842, -4.9275, -4.8097, -4.9186, -5.0713, -4.5208, -4.5973, -4.8438, -5.0093, -5.4401, -5.6071, -5.6071, -5.7432, -5.7643, -5.7858, -5.8532, -5.8767, -5.9255, -6.1486, -6.1486, -6.1803, -6.2131, -6.3184, -6.3561, -6.3953, -6.3953, -6.4361, -6.5231, -6.5231, -6.5696, -6.5696, -6.6184, -6.6184, -6.6696, -6.7237, -5.4873, -5.0606, -5.7226, -5.5369, -5.395, -4.1092, -5.22, -3.6823, -4.1852, -5.3804, -5.1036, -4.3623, -5.395, -5.4401, -5.3804, -5.8302, -5.6441, -4.8524, -5.2578, -5.324, -5.4098, -4.905, -4.1635, -5.1718, -5.1837, -5.1601, -4.9606, -5.3804, -4.827, -5.297, -5.2325, -5.324, -5.395, -3.2855, -3.7044, -3.7366, -4.2466, -4.5699, -5.0421, -5.6059, -5.6847, -5.8835, -5.9037, -5.9889, -6.0343, -6.0578, -6.0578, -6.1579, -3.6667, -6.1846, -6.2401, -6.3297, -6.3942, -6.4281, -6.4632, -6.4632, -6.4995, -6.5765, -6.6173, -6.6598, -6.6598, -6.7507, -6.7507, -6.7507, -5.5909, -5.4389, -5.3648, -3.6909, -3.1161, -4.9129, -4.1432, -5.3183, -5.6847, -6.1846, -3.1049, -3.2123, -4.854, -4.6658, -2.731, -3.2795, -3.3922, -5.2523, -3.9997, -3.302, -5.4649, -5.6059, -4.9436, -4.7718, -5.1043, -4.7917, -4.9129, -5.2107, -4.8259, -4.9999, -5.2523, -3.2505, -4.9646, -5.1538, -5.2755, -5.3874, -5.4006, -5.4277, -5.4414, -5.6405, -5.6405, -5.6576, -5.8262, -5.8894, -6.0044, -6.0044, -6.0291, -6.0291, -6.0291, -6.0291, -6.1345, -6.1345, -6.1345, -6.1916, -6.2839, -6.2839, -6.3506, -6.3857, -6.422, -6.5398, -4.6122, -2.8286, -5.7862, -4.7909, -4.6877, -3.9688, -4.7837, -5.7862, -4.7837, -5.8262, -2.6399, -5.3874, -3.786, -4.939, -5.5284, -5.675, -4.7837, -4.8979, -4.3066, -4.9307, -4.405, -4.5033, -3.609, -4.7075, -5.2408, -4.9475, -5.123, -4.3952, -4.843, -4.8661, -5.0453, -4.9307, -5.0453, -4.956, -4.7308, -5.0756, -5.5416, -5.6574, -5.8086, -5.8503, -5.8938, -5.9868, -6.0115, -6.0115, -6.0115, -6.174, -6.2346, -6.2346, -6.2346, -6.2663, -6.2991, -6.3681, -6.4044, -6.4044, -6.4422, -6.5222, -6.5647, -6.5647, -6.5647, -6.6091, -6.6091, -6.6091, -6.6556, -6.7044, -6.7044, -6.7044, -4.5021, -4.2669, -5.1054, -5.7115, -5.64, -3.9544, -4.1515, -3.398, -4.1247, -3.5586, -4.3825, -3.2407, -5.2817, -5.452, -4.1171, -3.7117, -5.2232, -4.2625, -5.5733, -2.6223, -4.6767, -5.7492, -5.1787, -4.5886, -4.8485, -5.2232, -4.4174, -5.2817, -5.3438, -4.8254, -5.1897, -5.0658, -4.7377, -4.991, -5.2346, -4.4056, -4.5014, -4.5413, -5.0253, -5.1141, -5.2114, -5.3713, -5.5143, -5.5616, -5.6112, -5.8385, -5.882, -5.9274, -5.9274, -5.951, -5.951, -6.0251, -6.0251, -6.0777, -6.1051, -6.1332, -6.1332, -6.1332, -6.2546, -6.3563, -6.3927, -6.4304, -6.4304, -6.5104, -6.5104, -3.6251, -3.4723, -3.4763, -5.0836, -4.4524, -4.0294, -3.8224, -4.4418, -5.5298, -2.5005, -2.8602, -4.8605, -3.2305, -3.5006, -5.0638, -5.6112, -4.3955, -5.1455, -4.6649, -3.0219, -5.0253, -3.8628, -4.361, -3.7227, -5.1038, -5.1455, -4.7042, -5.2001, -5.5871, -5.6626, -5.7443, -5.7443, -5.8332, -5.9308, -4.0812, -6.039, -6.039, -6.068, -6.1931, -6.3362, -6.5032, -6.5984, -6.5984, -6.6497, -6.6497, -6.6497, -6.7609, -6.7609, -6.8214, -6.8214, -6.8214, -6.8859, -6.8859, -7.0289, -7.0289, -7.0289, -3.2739, -4.2286, -7.0289, -7.0289, -7.0289, -2.9176, -4.1964, -3.6571, -4.0417, -5.2251, -5.8332, -2.4726, -4.9794, -2.4055, -2.8708, -2.0424, -4.6893, -5.8102, -3.0565, -4.1785, -5.0513, -5.6242, -4.6458, -5.7878, -5.6432, -5.0096, -5.2251, -5.2125, -4.507, -5.2905, -5.2905, -5.6242, -4.2344, -4.6456, -4.784, -5.1224, -5.1796, -5.3047, -5.4101, -5.4288, -5.5072, -5.6613, -5.7354, -5.7614, -5.8154, -5.8154, -5.8726, -5.8726, -5.8726, -5.9024, -5.9332, -5.9649, -6.0316, -6.0667, -6.0667, -6.103, -6.103, -6.103, -6.18, -6.18, -6.18, -6.2208, -5.2094, -5.0553, -4.7452, -5.9024, -4.717, -5.3917, -4.9683, -5.5489, -5.0553, -5.3737, -4.2872, -5.3047, -4.9803, -4.4433, -4.9448, -4.3753, -5.3215, -4.5471, -5.165, -5.2094, -5.3215, -4.4224, -4.5708, -4.9924, -5.1796, -5.2882, -5.2882, -3.1077, -3.2993, -3.3094, -3.7681, -4.1419, -4.4709, -4.9027, -4.9331, -4.9539, -5.089, -5.3031, -5.3334, -5.6576, -5.6576, -5.77, -5.7941, -5.8188, -5.8188, -5.8441, -5.8967, -5.9241, -6.0111, -6.0419, -6.0419, -6.0419, -6.1403, -6.2117, -6.2117, -6.2495, -6.2495, -3.0418, -5.77, -5.0192, -3.529, -5.1384, -5.0419, -4.9645, -4.5952, -4.4141, -4.2714, -5.3181, -2.8932, -4.986, -4.7457, -5.164, -5.1384, -4.4516, -4.8829, -5.164, -5.1384, -5.1511, -5.1771, -5.2664, -5.2664, -5.309, -5.3687, -5.4159, -5.7144, -5.9054, -5.9321, -5.9321, -5.9594, -6.0166, -6.2107, -6.2107, -6.247, -6.247, -6.247, -6.2848, -6.324, -6.4517, -6.4517, -6.4982, -6.4982, -6.547, -6.547, -6.5983, -6.5983, -6.6523, -6.6523, -6.6523, -6.6523, -3.0833, -2.8474, -6.6523, -4.8158, -4.6756, -2.5915, -6.247, -6.2848, -3.027, -2.6305, -4.8797, -5.239, -5.239, -4.546, -2.6567, -4.4998, -5.5541, -4.046, -5.0888, -5.631, -5.4487, -5.5728, -5.1737, -3.6571, -4.781, -5.3384, -5.0772, -5.4159, -4.3337, -4.9998, -4.4433, -5.0323, -4.9084, -5.2124, -3.9052, -4.2481, -4.9759, -4.9997, -5.0491, -5.1281, -5.1281, -5.1558, -5.17, -5.2441, -5.2441, -5.2596, -5.3242, -5.358, -5.4482, -5.5473, -5.5683, -5.7548, -5.7548, -5.7808, -5.8075, -5.8349, -5.8349, -5.892, -5.9219, -5.9219, -5.9219, -5.9526, -5.9844, -5.9844, -5.101, -4.4769, -4.6912, -4.2201, -4.8336, -4.3882, -4.4986, -4.9077, -5.0118, -2.9541, -5.4672, -4.9077, -5.4482, -4.8542, -5.199, -4.7742, -5.2289, -4.7742, -4.7936, -4.9877, -5.1162, -5.1302, -5.501, -5.5426, -5.5426, -5.6551, -5.6551, -5.7552, -5.8663, -5.927, -5.9587, -5.9587, -6.0254, -6.0254, -6.0604, -6.0968, -6.0968, -6.1345, -6.1737, -6.2145, -6.3015, -6.3015, -6.348, -6.348, -6.348, -6.348, -6.348, -6.348, -6.3968, -6.448, -6.448, -6.448, -2.9099, -3.6341, -3.9549, -4.6138, -6.0604, -4.2282, -3.1625, -4.1097, -4.2056, -3.0112, -4.7108, -3.0231, -3.9993, -2.9714, -4.8182, -3.9903, -5.4416, -4.6393, -4.6924, -5.5426, -4.6307, -5.501, -5.1302, -5.2985, -4.1459, -5.1302, -4.8286, -5.3153, -5.1882, -5.3675, -5.4038, -5.2303, -5.4763, -5.4999, -5.6266, -5.6266, -5.654, -5.7111, -5.8701, -5.9052, -5.9052, -5.9052, -5.9052, -5.9793, -6.0185, -6.0593, -6.1018, -6.1018, -6.1018, -6.2415, -6.2415, -6.2415, -6.2415, -6.2415, -6.2928, -6.2928, -6.2928, -6.2928, -6.3468, -6.3468, -6.404, -6.404, -6.404, -6.0185, -4.3625, -5.2122, -5.524, -4.1318, -5.4534, -5.1772, -4.6029, -4.7603, -5.7111, -5.741, -5.4763, -4.6127, -5.524, -5.5486, -5.3255, -5.3255, -4.8809, -4.9891, -5.0632, -5.4309, -4.3702, -4.8809, -5.1105, -4.9891, -5.2122, -4.1751, -4.6426, -4.9891, -5.2122, -4.975, -4.5933, -5.1601, -5.2303, -5.2863, -5.0945, -5.2673, -5.1772, -5.3057, -1.6217, -1.6764, -1.7072, -4.4274, -5.2089, -5.5089, -5.7319, -5.7319, -5.8654, -5.8654, -5.8654, -5.9394, -5.9394, -5.9394, -6.1063, -6.2016, -6.2016, -6.2016, -6.3068, -6.3068, -6.3068, -6.4245, -6.4245, -6.4245, -6.4245, -6.4245, -6.4245, -6.4245, -6.4245, -6.4245, -5.3267, -4.5537, -4.7913, -5.6713, -5.7319, -3.9942, -4.1356, -5.1375, -5.3267, -4.9495, -5.7964, -6.0194, -5.6142, -5.4601, -5.9394, -5.6142, -5.3692, -5.5089, -5.5089, -5.5089, -5.4601, -5.7319, -5.7964]}, "token.table": {"Topic": [17, 2, 5, 2, 3, 8, 15, 3, 8, 2, 4, 2, 20, 20, 1, 2, 3, 4, 5, 8, 10, 11, 15, 16, 17, 17, 17, 17, 7, 5, 20, 4, 12, 17, 2, 1, 5, 14, 5, 8, 3, 4, 5, 10, 12, 15, 19, 19, 6, 19, 1, 4, 6, 16, 1, 16, 1, 4, 6, 7, 8, 10, 13, 15, 19, 1, 3, 5, 6, 8, 16, 19, 16, 2, 8, 17, 18, 18, 3, 4, 5, 8, 10, 14, 16, 9, 18, 9, 15, 19, 4, 7, 8, 15, 10, 12, 12, 12, 11, 11, 14, 14, 5, 5, 8, 8, 3, 7, 8, 9, 12, 15, 16, 18, 2, 7, 8, 19, 20, 19, 20, 2, 9, 18, 20, 2, 3, 5, 8, 9, 11, 12, 13, 15, 17, 18, 19, 9, 18, 18, 18, 3, 3, 18, 2, 11, 17, 18, 19, 18, 18, 18, 18, 5, 7, 2, 3, 14, 15, 19, 20, 20, 14, 14, 14, 5, 10, 14, 14, 3, 12, 17, 1, 2, 13, 19, 3, 10, 15, 18, 4, 9, 12, 17, 5, 8, 12, 13, 14, 13, 13, 13, 13, 5, 6, 7, 13, 16, 9, 3, 9, 8, 8, 15, 3, 5, 6, 7, 8, 9, 13, 16, 17, 19, 5, 6, 7, 8, 9, 13, 16, 19, 7, 11, 17, 20, 17, 17, 2, 13, 18, 19, 7, 2, 3, 7, 8, 12, 15, 18, 2, 7, 7, 6, 1, 7, 14, 16, 17, 1, 1, 1, 16, 1, 6, 6, 19, 6, 19, 3, 8, 3, 5, 11, 17, 20, 20, 1, 4, 6, 16, 19, 6, 5, 17, 17, 6, 14, 5, 14, 15, 14, 15, 2, 4, 6, 9, 5, 5, 14, 16, 5, 16, 16, 8, 15, 4, 5, 6, 5, 5, 17, 17, 8, 15, 8, 19, 1, 6, 7, 1, 1, 1, 2, 19, 19, 2, 19, 19, 5, 6, 1, 2, 5, 10, 12, 15, 2, 19, 19, 4, 1, 4, 15, 17, 3, 8, 15, 19, 14, 15, 17, 6, 7, 9, 14, 3, 4, 5, 7, 8, 15, 18, 19, 18, 18, 19, 3, 8, 13, 18, 4, 5, 7, 18, 19, 20, 20, 1, 16, 17, 4, 5, 6, 10, 12, 13, 14, 15, 19, 19, 4, 5, 7, 8, 12, 13, 14, 15, 12, 13, 13, 15, 3, 5, 7, 12, 19, 3, 12, 5, 5, 7, 10, 12, 14, 12, 10, 12, 16, 10, 10, 12, 15, 10, 12, 2, 4, 20, 7, 5, 8, 19, 7, 4, 14, 17, 20, 3, 4, 6, 8, 11, 14, 17, 3, 3, 11, 4, 10, 12, 12, 2, 3, 7, 14, 18, 3, 3, 15, 2, 3, 14, 19, 9, 6, 1, 3, 8, 9, 15, 17, 3, 2, 14, 17, 18, 18, 18, 18, 2, 3, 6, 7, 19, 6, 6, 7, 14, 1, 3, 6, 16, 19, 19, 19, 1, 4, 8, 1, 6, 17, 20, 20, 2, 3, 5, 8, 16, 18, 19, 5, 5, 2, 7, 8, 14, 1, 1, 2, 9, 14, 17, 18, 16, 18, 18, 4, 3, 9, 11, 17, 11, 11, 1, 2, 3, 7, 9, 10, 6, 15, 17, 8, 2, 5, 8, 16, 2, 13, 13, 2, 16, 16, 16, 16, 7, 14, 3, 1, 4, 2, 7, 2, 2, 3, 6, 10, 12, 10, 12, 5, 14, 15, 2, 3, 7, 7, 7, 7, 7, 3, 15, 2, 7, 8, 11, 7, 2, 8, 10, 13, 13, 10, 3, 7, 11, 12, 2, 5, 6, 16, 6, 16, 1, 6, 16, 8, 11, 12, 2, 3, 5, 6, 15, 19, 19, 1, 2, 3, 7, 1, 2, 6, 16, 1, 6, 12, 19, 20, 6, 9, 16, 19, 16, 16, 16, 16, 16, 1, 4, 5, 6, 7, 10, 16, 18, 6, 3, 4, 5, 10, 11, 12, 17, 11, 11, 11, 3, 5, 9, 11, 12, 11, 3, 3, 10, 6, 10, 11, 12, 11, 12, 17, 1, 3, 3, 7, 9, 11, 17, 1, 7, 1, 2, 14, 13, 1, 2, 6, 15, 16, 17, 19, 1, 16, 17, 15, 15, 6, 16, 1, 6, 7, 16, 19, 15, 2, 5, 6, 13, 14, 4, 15, 3, 5, 10, 11, 15, 19, 3, 2, 3, 4, 7, 11, 12, 13, 14, 20, 14, 14, 5, 16, 16, 16, 14, 14, 17, 5, 20, 2, 7, 14, 5, 2, 10, 12, 10, 12, 12, 1, 2, 4, 6, 7, 14, 19, 1, 1, 19, 4, 4, 6, 8, 14, 19, 7, 17, 18, 4, 5, 8, 20, 7, 17, 20, 7, 14, 3, 5, 7, 8, 15, 4, 18, 4, 2, 4, 5, 6, 10, 10, 10, 10, 4, 14, 3, 7, 11, 18, 1, 2, 3, 4, 5, 6, 8, 10, 12, 14, 10, 12, 8, 20, 1, 11, 18, 19, 18, 1, 11, 15, 18, 18, 2, 17, 6, 9, 20, 20, 2, 4, 8, 10, 14, 15, 12, 12, 2, 6, 8, 14, 1, 2, 5, 15, 20, 20, 15, 2, 8, 10, 5, 5, 2, 5, 8, 14, 18, 2, 11, 10, 12, 8, 2, 19, 3, 10, 2, 7, 2, 3, 5, 8, 12, 16, 17, 19, 3, 1, 2, 6, 15, 16, 10, 15, 10, 11, 17, 5, 8, 15, 8, 15, 10, 11, 17, 11, 2, 14, 18, 19, 18, 19, 14, 6, 6, 9, 2, 2, 2, 2, 2, 3, 8, 15, 4, 5, 7, 8, 13, 14, 16, 18, 16, 2, 3, 5, 6, 7, 10, 13, 14, 16, 17, 3, 4, 5, 10, 14, 15, 18, 4, 15, 5, 5, 5, 16, 5, 16, 5, 16, 5, 16, 9, 1, 3, 4, 6, 19, 19, 1, 6, 7, 14, 6, 8, 11, 17, 11, 8, 17, 17, 11, 1, 8, 20, 11, 12, 15, 4, 18, 18, 2, 7, 8, 17, 3, 4, 6, 16, 19, 4, 4, 15, 17, 7, 3, 9, 6, 9, 11, 15, 16, 18, 19, 11, 7, 17, 4, 5, 8, 12, 15, 17, 20, 5, 2, 4, 10, 3, 9, 17, 3, 5, 8, 15, 17, 2, 4, 16, 19, 4, 16, 19, 10, 2, 3, 5, 7, 10, 14, 17, 18, 18, 18, 1, 2, 3, 4, 8, 11, 12, 16, 19, 13, 1, 11, 11, 12, 14, 11, 18, 7, 14, 17, 2, 10, 7, 7, 7, 2, 7, 7, 1, 4, 6, 14, 16, 19, 20, 1, 19, 1, 4, 8, 10, 11, 12, 13, 14, 17, 20, 20, 9, 9, 11, 15, 18, 20, 20, 4, 1, 2, 3, 5, 7, 8, 11, 13, 15, 17, 19, 11, 19, 1, 2, 4, 6, 11, 13, 2, 14, 3, 4, 7, 8, 14, 17, 19, 9, 9, 3, 1, 2, 3, 5, 6, 8, 9, 9, 6, 19, 1, 2, 3, 4, 5, 6, 7, 8, 19, 1, 6, 10, 19, 7, 1, 2, 4, 15, 16, 20, 14, 6, 10, 2, 10, 7, 2, 14, 2, 3, 5, 8, 9, 11, 3, 7, 19, 8, 15, 8, 3, 7, 9, 11, 11, 17, 11, 12, 13, 19, 3, 1, 4, 5, 6, 16, 19, 5, 8, 15, 17, 20, 15, 17, 10, 6, 10, 10, 3, 14, 18, 14, 18, 1, 2, 14, 5, 4, 4, 10, 17, 8, 14, 4, 4, 5, 15, 19, 1, 5, 7, 19, 20, 20, 2, 7, 20, 9, 17, 18, 19, 18, 3, 11, 8, 8, 3, 1, 4, 5, 10, 4, 20, 1, 4, 4, 4, 9, 13, 13, 1, 4, 7, 13, 14, 9, 9, 1, 2, 5, 6, 7, 8, 9, 11, 12, 13, 15, 16, 19, 7, 13, 19, 12, 13, 16, 13, 20, 11, 19, 9, 13, 13, 9, 13, 16, 2, 7, 13, 13, 9, 11, 15, 13, 9, 9, 13, 12, 11, 17, 6, 14, 19, 6, 11, 12, 17, 18, 11, 11, 14, 11, 11, 1, 5, 6, 16, 16, 16, 4, 10, 4, 14, 10, 3, 1, 3, 9, 11, 15, 16, 17, 18, 7, 2, 2, 4, 13, 17, 17, 1, 2, 3, 4, 5, 8, 9, 10, 14, 15, 16, 17, 18, 19, 3, 9, 11, 17, 4, 9, 7, 13, 13, 20, 2, 11, 12, 18, 19, 6, 16, 19, 19, 4, 5, 13, 14, 13, 13, 13, 17, 11, 10, 11, 2, 6, 13, 14, 15, 2, 4, 5, 6, 9, 13, 14, 19, 5, 6, 7, 9, 13, 16, 2, 17, 1, 7, 14, 14, 4, 6, 9, 12, 17, 18, 5, 7, 12, 17, 9, 19, 7, 8, 3, 11, 12, 17, 11, 9, 13, 20, 2, 8, 19, 2, 10, 8, 9, 2, 3, 4, 5, 7, 8, 9, 10, 12, 15, 10, 12, 3, 7, 13, 14, 17, 15, 15, 20, 10, 15, 5, 15, 16, 5, 16, 8, 10, 13, 19, 4, 6, 10, 18, 19, 3, 5, 6, 7, 8, 17, 8, 20, 10, 10, 8, 7, 9, 11, 17, 18, 19, 18, 9, 8, 10, 14, 14, 1, 6, 16, 19, 1, 16, 1, 5, 6, 1, 4, 19, 6, 5, 4, 5, 6, 16, 17, 3, 11, 15, 3, 11, 15, 17, 14, 2, 8, 20, 18, 11, 17, 3, 11, 3, 5, 7, 14, 4, 19, 2, 7, 2, 5, 14, 2, 13, 10, 10, 1, 4, 13, 16, 19, 7, 16, 19, 13, 16, 16, 4, 1, 7, 3, 4, 9, 15, 16, 18, 6, 4, 2, 4, 5, 7, 13, 19, 5, 14, 14, 1, 4, 6, 15, 4, 6, 9, 10, 12, 13, 17, 19, 1, 5, 6, 19, 9, 12, 18, 19, 19, 15, 2, 3, 5, 8, 16, 19, 20, 1, 4, 6, 1, 1, 3, 6, 6, 17, 17, 19, 9, 14, 20, 5, 6, 3, 1, 6, 14, 19, 3, 4, 10, 12, 4, 4, 1, 2, 6, 15, 16, 20, 3, 7, 8, 9, 11, 12, 14, 18, 19, 10, 18, 2, 6, 6, 10, 17, 20, 6, 2, 4, 5, 8, 9, 13, 15, 16, 13, 16, 2, 7, 9, 13, 15, 16, 8, 4, 8, 8, 5, 8, 20, 2, 10, 10, 10, 7, 8, 9, 19, 7, 8, 18, 5, 11, 18, 17, 2, 5, 6, 15, 20, 1, 6, 6, 2, 5, 16, 5, 16, 16, 1, 6, 16, 1, 1, 1, 4, 11, 8, 14, 14, 18, 4, 5, 10, 14, 5, 5, 7, 13, 13, 13, 2, 17, 7, 9, 9, 1, 2, 4, 6, 10, 14, 1, 10, 10, 8, 19, 1, 4, 7, 10, 1, 4, 10, 14, 16, 7, 2, 6, 1, 6, 7, 8, 9, 13, 14, 17, 3, 9, 12, 7, 7, 18, 7, 2, 13, 2, 10, 12, 19, 5, 6, 16, 19, 6, 6, 16, 6, 1, 6, 16, 19, 1, 16, 16, 16, 16, 20, 5, 6, 15, 19, 20, 6, 6, 3, 11, 17, 11, 5, 10, 12, 17, 12, 3, 6, 11, 17, 11, 3, 11, 17, 3, 15, 15, 1, 4, 8, 10, 5, 10, 14, 2, 10, 10, 8, 15, 18, 19, 3, 8, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 13, 19, 12, 16, 10, 12, 9, 11, 3, 9, 11, 17, 9, 16, 3, 9, 11, 18, 12, 12, 11, 11, 3, 6, 8, 9, 16, 19, 2, 7, 17, 17, 3, 11, 14, 11, 1, 15, 1, 3, 5, 9, 10, 11, 15, 18, 19, 11, 19, 15, 15, 15, 2, 3, 7, 14, 2, 11, 17, 2, 3, 5, 7, 9, 18, 3, 1, 4, 7, 9, 14, 16, 4, 12, 4, 16, 4, 8, 5, 15, 3, 2, 2, 10, 8, 19, 8, 19, 2, 10, 11, 10, 10, 10, 7, 12, 14, 17, 14, 18, 8, 19, 20, 2, 6, 8, 8, 6, 8, 14, 19, 4, 5, 8, 9, 10, 12, 13, 13, 10, 13, 13, 6, 8, 3, 8, 15, 3, 3, 15, 14, 1, 3, 18, 1, 2, 3, 6, 8, 12, 15, 17, 1, 2, 3, 4, 9, 14, 3, 3, 3, 15, 3, 9, 1, 4, 6, 16, 19, 2, 20, 20, 7, 11, 15, 17, 9, 10, 11, 15, 16, 17, 19, 11, 1, 9, 11, 14, 15, 17, 9, 9, 9, 17, 9, 1, 2, 4, 5, 6, 14, 19, 19, 19, 6, 8, 9, 10, 12, 9, 3, 11, 17, 2, 7, 18, 11, 10, 2, 5, 1, 9, 10, 11, 2, 5, 5, 1, 6, 15, 19, 1, 4, 6, 14, 16, 4, 4, 3, 8, 18, 3, 8, 4, 12, 12, 12, 4, 10, 15, 2, 6, 16, 19, 4, 2, 3, 9, 10, 11, 15, 17, 14, 15, 8, 14, 6, 16, 6, 1, 15, 16, 19, 1, 4, 9, 10, 4, 9, 19, 12, 1, 2, 3, 4, 5, 7, 8, 10, 14, 15, 17, 2, 1, 9, 19, 1, 6, 10, 16, 16, 7, 13, 6, 2, 6, 6, 12, 7, 12, 17, 7, 17, 5, 1, 2, 4, 5, 10, 19, 4, 7, 10, 12, 13, 14, 19, 13, 10, 12, 12, 10, 12, 13, 12, 6, 19, 6, 19, 14, 15, 15, 15, 13, 6, 9, 9, 7, 8, 10, 11, 14, 17, 2, 3, 17, 19, 19, 3, 15, 1, 4, 5, 6, 19, 1, 2, 7, 11, 18, 14, 9, 10, 12, 9, 5, 8, 14, 2, 5, 8, 10, 16, 19, 4, 3, 4, 7, 10, 19, 8, 2, 3, 4, 5, 6, 10, 11, 3, 5, 8, 11, 17, 18, 19, 18, 18, 8, 13, 2, 14, 5, 10, 6, 19, 6, 19, 6, 2, 5, 7, 9, 13, 14, 13, 9, 9, 2, 11, 12, 17, 19, 11, 12, 3, 4, 9, 12, 17, 2, 4, 5, 14, 16, 3, 9, 10, 11, 12, 12, 12, 12, 12, 9, 12, 12, 9, 12, 4, 1, 3, 4, 8, 11, 15, 19, 9, 14, 9, 19, 4, 14, 8, 18, 19, 20, 19, 3, 9, 11, 15, 3, 8, 12, 15, 15, 20, 20], "Freq": [0.9876162745841255, 1.0012561509575209, 0.9812377013086145, 0.09131368426973781, 0.5057373282631633, 0.1334584616250014, 0.27394105280921344, 0.9685292118170035, 0.033397559028172534, 0.9980670834388463, 0.9881644559858817, 0.9980670834388463, 0.9696011525488868, 0.9696011525488868, 0.17933257076202225, 0.10484057983010531, 0.09380472932167316, 0.2345118233041829, 0.15726086974515796, 0.05517925254216069, 0.013794813135540172, 0.0882868040674571, 0.005517925254216069, 0.02483066364397231, 0.03862547677951248, 0.9993860886999937, 0.9807769510273527, 0.9906912063253746, 0.994293085960902, 0.9913174429927268, 0.9696011525488868, 0.8616855352041795, 0.0940020583859105, 0.03133401946197016, 1.0016402091006151, 0.006616004520789932, 0.866696592223481, 0.1257040858950087, 1.0013404850590966, 0.9886765219456565, 0.1845270355472829, 0.17222523317746405, 0.1230180236981886, 0.06765991303400373, 0.06765991303400373, 0.10456532014346032, 0.27063965213601493, 1.0052818272526638, 0.30430308929532995, 0.7100405416891031, 0.8563730703679702, 0.007646188128285448, 0.05862077565018843, 0.07646188128285447, 0.9938520351723777, 0.9782272554360975, 0.995912768542635, 0.061828203926572865, 0.05116816876681893, 0.2259927453867836, 0.05116816876681893, 0.07248823908632682, 0.36670520949553564, 0.1343164430128997, 0.04050813360706498, 0.09404058344609523, 0.053737476254911556, 0.0973991757120272, 0.1578538364988027, 0.053737476254911556, 0.11083354477575509, 0.4366169945711564, 0.9847924567777853, 0.17717360731408127, 0.08858680365704064, 0.605343158323111, 0.13288020548556095, 1.0157267080624626, 0.4966822041381025, 0.16622033923346854, 0.0712372882429151, 0.05540677974448952, 0.12466525442510142, 0.041555084808367136, 0.043533898370670336, 0.9279557808675539, 0.06291225633000366, 0.12002800339743831, 0.780182022083349, 0.09002100254807874, 0.07254258613972867, 0.06528832752575581, 0.10155962059562014, 0.7592790682624934, 0.11136753645046446, 0.8785661208869974, 1.0071586018588248, 0.9985872050901208, 0.9614192979707425, 0.9905063700166036, 1.003060240722879, 1.003560993971607, 1.0067856146351795, 0.9974392689686087, 0.9980717693774938, 0.991831564584729, 0.22920393481932336, 0.10071081984485421, 0.2569862299489383, 0.07292852471523925, 0.03820065580322056, 0.08681967228004672, 0.05903737715043177, 0.1562754101040841, 0.19604276442712557, 0.8076961894397573, 0.9770410389280032, 0.36005900312377903, 0.6121003053104243, 0.142252335983323, 0.853514015899938, 1.0012561509575209, 0.9560860049659384, 0.0406845108496144, 0.9386412700338148, 0.046088289706484924, 0.14073388463944503, 0.029628186239883166, 0.0337432121065336, 0.08394652767966897, 0.0683094293863973, 0.05514134661311589, 0.01975212415992211, 0.05102632074646545, 0.037035232799853955, 0.4040955401050732, 0.03127419658654334, 0.9918039177976615, 0.9922544349710142, 0.9835535317718871, 0.986755060551829, 0.9845149891297588, 0.12819767796507975, 0.8699128147630413, 0.0016417282809508967, 0.013133826247607174, 0.06238567467613408, 0.8569821626563682, 0.06402740295708498, 1.0045441236851302, 1.0045441236851302, 1.0045441236851302, 1.0093405429603173, 0.9899329515892195, 0.9906447260908646, 0.20836686892728165, 0.04861893608303239, 0.5278627346157801, 0.05209171723182041, 0.10765621561242886, 0.05209171723182041, 1.0228666595007407, 1.001899968083595, 1.0070808947536574, 1.001899968083595, 0.0649590243735875, 0.012991804874717499, 0.9224181461049424, 0.9781507970803773, 0.12405891557574988, 0.6966385259253647, 0.18131687661071136, 0.9988904607224313, 0.6434571935616324, 0.2624628026369817, 0.08466542020547796, 0.3015773986803365, 0.4449502603480374, 0.05932670137973833, 0.19281177948414957, 0.6852834393122311, 0.2153747952124155, 0.08614991808496619, 0.007831810734996928, 0.03838275491295514, 0.046324014550118274, 0.04897110109583932, 0.8020672233534765, 0.06353007709730507, 0.9702029246911564, 0.992102741979743, 0.9994175260755178, 0.9478211738603441, 0.06049802563911689, 0.12099605127823378, 0.020166008546372294, 0.685644290576658, 0.10083004273186147, 0.9987995095644, 0.8663287445422523, 0.13144298193054862, 0.9980999267802304, 0.9714896326866096, 0.9718655273752628, 0.025981482236645224, 0.02550034367670735, 0.03223628351583759, 0.07842558526987355, 0.03416083775558909, 0.22998423165030404, 0.48643108409719116, 0.05725548863260707, 0.01491529535807411, 0.01491529535807411, 0.02783860133003549, 0.01948702093102484, 0.060688150899477364, 0.031179233489639746, 0.23774165535850308, 0.5261495651376707, 0.0740506795378944, 0.02227088106402839, 0.13608646028429372, 0.05443458411371748, 0.4626939649665986, 0.3538247967391636, 0.9895414532325696, 0.9960294862205077, 0.08948208158406683, 0.1073784979008802, 0.7516494853061614, 0.0536892489504401, 0.998522942989932, 0.03616981092849388, 0.041336926775421584, 0.29452560327487876, 0.041336926775421584, 0.07233962185698777, 0.03616981092849388, 0.48054177376427587, 0.1825282328997588, 0.8156730407707972, 0.994293085960902, 1.014578483011166, 0.7633536822779833, 0.03280889751911103, 0.05468149586518505, 0.06014964545170356, 0.08967765321890349, 1.0025207266436151, 0.9893664601404354, 0.9500186676189696, 0.04913889660098119, 0.896715744478592, 0.10056625171722527, 0.6843819327366213, 0.3144457528789882, 0.9208309337258992, 0.06949667424346409, 0.98657659753366, 1.0051688134190433, 0.3166030004925551, 0.1658396669246717, 0.1884541669598542, 0.21860683367343087, 0.11307250017591251, 1.0063112220914376, 0.3528771563168167, 0.1441863649466563, 0.22386830346980846, 0.1024482066726242, 0.17454138914595235, 0.998771643393994, 1.0013281707036474, 0.9859363136986777, 1.0006807813412497, 0.989413349000518, 0.9934819031996981, 0.30491192720826843, 0.5236530923794175, 0.16571300391753718, 0.8362056558611987, 0.16048391375113916, 0.30665378064430665, 0.14018458543739734, 0.551976805159752, 0.9809465830231925, 0.9959792778350075, 0.8162842812269064, 0.11212696170699264, 0.07176125549247529, 0.29353150167361053, 0.7063101759021253, 1.001294141660612, 0.8753738108872958, 0.12329208604046421, 1.0024414622793665, 0.9461963488364274, 0.048941190457056596, 1.000342258580011, 1.0013404850590966, 1.0004634823362146, 1.0004634823362146, 0.9888749795487566, 0.017979545082704666, 0.9868084348992956, 0.987883193907996, 0.9169038943263745, 0.053044026944501006, 0.027784966494738623, 1.0030558469611364, 0.9920324593035055, 0.9996727000611592, 0.29115045672408185, 0.7146420301409282, 0.9622865757299193, 0.2140988404115488, 0.7850290815090123, 0.987883193907996, 0.01707297589478228, 0.9902326018973723, 0.005110429771422099, 0.34750922445670274, 0.08687730611417568, 0.16864418245692928, 0.12776074428555248, 0.26574234811394915, 1.0018137513056256, 0.987883193907996, 0.9622865757299193, 1.0250472147156342, 0.6651387728200119, 0.27651836622854425, 0.014946938715056446, 0.04484081614516934, 0.20803244631390674, 0.1144178454726487, 0.6344989612574156, 0.04160648926278135, 0.022968916359489954, 0.8613343634808733, 0.11484458179744977, 0.11798198271170152, 0.17697297406755227, 0.17697297406755227, 0.5309189222026568, 0.061552046425790034, 0.054488696835945274, 0.026235298476566242, 0.166493240332055, 0.056506796718758065, 0.06054299648438364, 0.4702172726953796, 0.10292309402345219, 1.0041066099828138, 0.2743517116123304, 0.7133144501920591, 0.10013962209111174, 0.1835893071670382, 0.06675974806074116, 0.6509075435922262, 0.10615323319572144, 0.07076882213048095, 0.30880940566028053, 0.29915911173339677, 0.1769220553262024, 0.03860117570753507, 1.0063112220914376, 0.8578323483561873, 0.13197420743941343, 0.013197420743941344, 0.2842326313046388, 0.030453496211211302, 0.020302330807474202, 0.060906992422422604, 0.030453496211211302, 0.08120932322989681, 0.1116628194411081, 0.13196515024858232, 0.24870355239155897, 1.008500819709546, 0.10785177273187371, 0.14960084604743773, 0.08001905718816438, 0.09393541496001905, 0.03305134970815485, 0.1878708299200381, 0.13046585411113756, 0.21744309018522928, 0.192329512047257, 0.8174004262008423, 1.005017640617968, 0.9986264385154209, 0.4763544980546841, 0.0965583442002738, 0.05149778357347936, 0.2639261408140817, 0.10943279009364365, 0.9324180338702985, 0.058889560033913595, 0.9925205308576311, 0.9959792778350075, 0.9923527192734173, 0.4783289224697935, 0.4994471971925217, 0.021646231590796396, 0.9938995949099865, 0.9742845388635685, 0.9816938309267614, 0.9756687859564599, 1.012047353113127, 0.3486134013983463, 0.6488082748246999, 1.0082911884006456, 0.9845349642762237, 0.9989408958600289, 1.0012561509575209, 1.0069512310895508, 0.9830492056628494, 1.0043684007278717, 0.7828475005750503, 0.22156061337029725, 0.9622865757299193, 0.9989445509928606, 0.1985035317165632, 0.02835764738808046, 0.17014588432848274, 0.5955105951496896, 0.6298817037719159, 0.0097656078104173, 0.014648411715625951, 0.053710842957295155, 0.10742168591459031, 0.09277327419896436, 0.08789047029375571, 1.0082398575319498, 0.9999149671488327, 0.9944671163559369, 0.1561295758675033, 0.140516618280753, 0.6947766126103897, 1.0008702081988239, 0.3807486242336945, 0.01586452600973727, 0.21681518879974268, 0.1533570847607936, 0.22739153947290086, 0.9912203468364766, 0.9901143567303106, 0.9843994370262504, 0.06919272629790935, 0.34596363148954673, 0.057660605248257786, 0.5304775682839716, 1.0000726434167662, 1.0109473324300673, 0.01921571620505387, 0.7333998351595561, 0.1056864391277963, 0.03522881304259876, 0.04323536146137121, 0.06245107766642508, 0.965983675636278, 0.14562846276559624, 0.07060773952271332, 0.09267265812356124, 0.6928384440666245, 1.0038093182877874, 1.0038093182877874, 0.9786846703849228, 0.11205400350123865, 0.037351334500412885, 0.7470266900082576, 0.060695918563170935, 0.042020251312964496, 0.9809815329245998, 0.8431097250482368, 0.14051828750803946, 0.01561314305644883, 0.015068616457393947, 0.003767154114348487, 0.6253475829818488, 0.052740157600878815, 0.3013723291478789, 0.9957797649021717, 0.9849246914453077, 0.9977094543890408, 1.0144884057647041, 1.0071377678638542, 0.03235597073950986, 0.9706791221852958, 0.9962487151742717, 1.0000738305761665, 1.0001039913266685, 0.06201946193248768, 0.19559984147938422, 0.32918022102628075, 0.11449746818305417, 0.042936550568645315, 0.19559984147938422, 0.06679018977344826, 0.9910449854819002, 1.000321596328642, 0.08620281567979785, 0.06465211175984839, 0.840477452878029, 0.9878877852242627, 0.9946085790241233, 0.9331177014441735, 0.06861159569442453, 0.11191978729640328, 0.783438511074823, 0.020349052235709687, 0.09157073506069359, 0.9556555232273058, 0.993054201797141, 1.0132497697963447, 1.0250472147156342, 0.6740841479961343, 0.05039881480344929, 0.012599703700862323, 0.2614438517928932, 1.0040210395817026, 0.9957944135841436, 0.014395456584891192, 0.7557614707067876, 0.08637273950934715, 0.05038409804711917, 0.043186369754673576, 0.05038409804711917, 1.0005454627612227, 0.9868071737455734, 0.9967520170219162, 0.9962875749581009, 0.6060630280290391, 0.24793487510278872, 0.08264495836759624, 0.06611596669407699, 0.9989579727859289, 0.9960839223377274, 0.9899837798104666, 0.07867509244351283, 0.9216225114811503, 1.01121206901985, 0.9847924567777853, 0.9556555232273058, 0.3377332742327272, 0.6581468933765966, 0.992937993185717, 0.08111841316826979, 0.9193420159070576, 0.9183867453590778, 0.078160574073113, 0.999340269562384, 0.002771269109985665, 0.002771269109985665, 0.01662761465991399, 0.2771269109985665, 0.6983598157163876, 0.23509896761504143, 0.7650045771600554, 0.01621964840648375, 0.9893985527955087, 0.9773409034608559, 0.07416939078419821, 0.030903912826749255, 0.8900326894103786, 0.9968510183236542, 0.9953318942145869, 0.9940734084011087, 0.998522942989932, 0.8172259452195633, 0.17511984540419212, 0.5928044398393926, 0.173367336179445, 0.02236997886186387, 0.2069223044722408, 0.999951770515259, 0.980649624314266, 0.025144862161904258, 0.13314386862096808, 0.8787495328983893, 1.005017640617968, 0.989301579890212, 0.7861126554772266, 0.10768666513386667, 0.06461199908032, 0.03230599954016, 0.027337054242017413, 0.054674108484034825, 0.33351206175261244, 0.5850129607791726, 0.3042647216707732, 0.6845956237592397, 0.01413583345533797, 0.2261733352854075, 0.7633350065882504, 0.7322836312918187, 0.04674150838032886, 0.21812703910820133, 0.3751635116862086, 0.10809796099433129, 0.2925003650434847, 0.038152221527411045, 0.158967589697546, 0.025434814351607365, 0.9996847107387054, 0.7601959023923811, 0.11049359046400888, 0.09723435960832781, 0.030938205329922485, 0.996106390800572, 0.999340269562384, 0.8948397281968451, 0.11185496602460564, 0.041640338910930295, 0.14990522007934906, 0.04996840669311635, 0.2748262368121399, 0.47469986358460536, 0.031012041802001938, 0.0016322127264211546, 0.9564766576827967, 0.009793276358526927, 0.9851230994236954, 0.9947213722950213, 0.9908285955847362, 0.9938744984997844, 0.9855141452758013, 0.012503088481272975, 0.23755868114418652, 0.06876698664700136, 0.48136890652900954, 0.043760809684455414, 0.11877934057209326, 0.037509265443818926, 0.006251544240636488, 1.0047212369459186, 0.2603544182315147, 0.04814773487843079, 0.0659802292778496, 0.005349748319825643, 0.5331915825426224, 0.057063982078140195, 0.028531991039070097, 0.9809790503009068, 0.9719360150034656, 0.9957944135841436, 0.24969879509578202, 0.05618222889655095, 0.09363704816091827, 0.5368524094559314, 0.062424698773945506, 0.9925067457938211, 0.9846483715971035, 0.05007187175659251, 0.9263296274969615, 0.08008038553320553, 0.16905859168121165, 0.2669346184440184, 0.4849312235066334, 0.8204256852972137, 0.06310966809978567, 0.11044191917462493, 0.9657894659280928, 0.9818722380638151, 0.25888317428732643, 0.06472079357183161, 0.06472079357183161, 0.016180198392957902, 0.5905772413429634, 0.9938520351723777, 0.995824360802527, 0.055546326113852504, 0.18052555987002064, 0.7637619840654719, 0.992102741979743, 0.2229318435234074, 0.038811913309807655, 0.07999040669948163, 0.2806763974721456, 0.33558105532504423, 0.0184593246229573, 0.02319248478268994, 1.0028551249831454, 0.9597422278532611, 0.050512748834382164, 1.000136346090557, 0.9972608640617089, 0.16818415885976817, 0.8320689964641163, 0.2592205174745951, 0.09426200635439823, 0.03299170222403938, 0.5773547889206891, 0.03652652746232932, 0.9915245798835934, 0.668121593172299, 0.042646059138657384, 0.014215353046219128, 0.07107676523109564, 0.21323029569328691, 1.0069512310895508, 1.0020572225566406, 0.6304023740350606, 0.05043218992280485, 0.06484138704360623, 0.09005748200500865, 0.1404896719278135, 0.02161379568120208, 0.9748609737218772, 0.07555129014995346, 0.07252923854395532, 0.12088206423992555, 0.3596241411137785, 0.06648513533195904, 0.006044103211996278, 0.16923488993589575, 0.10274975460393672, 0.02719846445398325, 1.0051394083783092, 0.99848986857432, 0.4450662918980382, 0.5518822019535674, 0.9938808384435437, 1.0001964718483514, 1.0009442210776007, 1.0010725756108965, 0.9870760143547936, 0.26007152684278073, 0.7802145805283422, 0.6018730825318318, 0.11356095896827015, 0.29525849331750237, 1.0058807152372335, 0.09524069750119167, 0.5841429446739755, 0.3238183715040517, 0.6587375411931068, 0.334284722396502, 0.9928415100458365, 0.2777167194717299, 0.10590891844260886, 0.3483226651001358, 0.0706059456284059, 0.03530297281420295, 0.07531300867029964, 0.08708066627503395, 0.979004602234783, 1.0025207266436151, 0.9933691969904922, 0.9816300491991548, 0.18694729532614812, 0.07010523574730554, 0.6543155336415184, 0.07010523574730554, 0.01557894127717901, 0.17704640753843773, 0.7672010993332302, 0.04917955764956604, 0.9882348805319314, 0.6316454942879826, 0.3651700513852399, 1.0063112220914376, 0.9922514877522252, 0.14964088413965768, 0.8978453048379461, 0.8122535019259445, 0.18744311582906412, 0.0022052783390870785, 0.0507214017990028, 0.10364808193709268, 0.73215240857691, 0.11026391695435392, 0.9846595383894048, 0.028133129668268708, 1.0250472147156342, 0.02678721984807169, 0.05113923789177323, 0.004870403608740308, 0.0036528027065552307, 0.9132006766388077, 0.999573439494657, 0.9912991158090275, 0.9960715845759458, 0.9601075825493129, 0.039348671415955445, 0.14633967822919958, 0.0054199880825629475, 0.0921397974035701, 0.7587983315588126, 0.05026653795692671, 0.013631603513742836, 0.07156591844714989, 0.09542122459619985, 0.02470728136865889, 0.029819132686312453, 0.07923369542363023, 0.1337601094786016, 0.3885007001416708, 0.11331270420798732, 0.21554341384767714, 0.7851938647308239, 1.0069333106473775, 1.0063112220914376, 0.027055961067578703, 0.037201946467920716, 0.8556447687621764, 0.08116788320273612, 1.0157267080624626, 0.0046737699529462535, 0.11684424882365635, 0.018695079811785014, 0.8599736713421107, 0.9786846703849228, 0.9938537046293577, 0.009510561766788111, 0.17993216710127868, 0.31987940818005095, 0.49981157528132963, 0.9443185104410406, 0.5291057265964928, 0.08942631998813964, 0.2757311532967639, 0.03726096666172485, 0.0745219333234497, 0.9773409034608559, 1.00326949789919, 0.9938995949099865, 0.013481106233837905, 0.16177327480605486, 0.8088663740302743, 0.013481106233837905, 0.9657894659280928, 0.037541659885849926, 0.3378749389726493, 0.009385414971462482, 0.6194373881165238, 1.0012309059966853, 0.9868071737455734, 0.6726577016789996, 0.27226621258435696, 0.06406263825514281, 1.0058807152372335, 0.9929666132062225, 0.9996880660895818, 0.08784822223788512, 0.7153355239370646, 0.07529847620390152, 0.1254974603398359, 0.8539988564514706, 0.13902306965489056, 0.09754387553356218, 0.897403654908772, 0.9849473939752016, 0.7950908293705142, 0.20613465946642962, 0.9234031356700447, 0.05431783151000263, 0.9346822556386613, 0.07789018796988843, 0.14193335692540937, 0.12490135409436025, 0.1788360297260158, 0.11922401981734387, 0.0028386671385081876, 0.13057868837137662, 0.12774002123286843, 0.17315869544899945, 0.9845149891297588, 0.2311641775949778, 0.05547940262279467, 0.13869850655698668, 0.12945193945318756, 0.45308178808615646, 0.9184604691859847, 0.07850089480222092, 1.012047353113127, 0.23179784224891575, 0.7488853364964971, 0.018484438068514165, 0.03696887613702833, 0.944554785301074, 1.017352637546165, 1.00845160861028, 0.07038697290755473, 0.8681059991931751, 0.05865581075629561, 0.9614192979707425, 0.06727103711029632, 0.30645694683579433, 0.42604990169854334, 0.20181311133088894, 0.7563722708858889, 0.25875893477675144, 0.9949747393565611, 1.014578483011166, 0.07153564412247264, 0.9299633735921443, 1.0008133727538329, 0.9985357743981557, 0.9985357743981557, 1.0005655879740407, 0.04774180013170178, 0.12119072341124298, 0.796920817583022, 0.036724461639770604, 0.09374597077736255, 0.2150642859010082, 0.10201767408124748, 0.20954981703175157, 0.09098873634273423, 0.17094853494695522, 0.09374597077736255, 0.02481510991165479, 1.0141624044997604, 0.16398383720854662, 0.0030940346643122, 0.11447928257955141, 0.046410519964683004, 0.0990091092579904, 0.0030940346643122, 0.077350866607805, 0.3650960903888396, 0.0804449012721172, 0.0495045546289952, 0.05021321886918995, 0.10879530754991155, 0.029291044340360804, 0.343123662272798, 0.025106609434594977, 0.3389392273670322, 0.10879530754991155, 1.0024414622793665, 0.992855311363374, 1.0022043982372897, 1.0117293194791197, 0.8718338556551313, 0.12815375161077652, 0.6889193887833532, 0.30722080851149536, 0.8808058279478858, 0.11518230057780043, 0.6814698533685333, 0.3206916957028392, 1.0049496158617213, 0.2501904042439361, 0.012145165254560006, 0.10201938813830405, 0.49795177543696023, 0.13602585085107208, 0.9947003291068112, 0.6847192742535432, 0.2779708980341219, 0.012563656408321893, 0.025127312816643786, 0.9796934047155403, 0.006449346390284791, 0.8771111090787317, 0.11608823502512625, 0.9674800092829665, 0.9714896326866096, 1.0013179726622161, 0.9876162745841255, 0.9977920159512564, 0.04879889585846919, 0.7807823337355071, 0.16591624591879528, 0.8301457225948394, 0.1702863020707363, 0.9955052173614559, 1.0149947625443116, 0.986755060551829, 0.9773208385789353, 0.22145196941313147, 0.7750818929459602, 0.9868084348992956, 0.9780163985660782, 1.005474389550587, 0.8130949474858816, 0.09345918936619328, 0.03271071627816765, 0.06074847308802563, 0.9712273766315999, 1.0024414622793665, 0.9285682270144986, 0.07847055439559143, 0.9930482128285277, 0.9657720988867394, 0.03307438694817601, 0.03768847894540229, 0.1669061210439244, 0.6245519368095236, 0.08255571578516692, 0.001794689473590585, 0.0358937894718117, 0.048456615786945795, 1.0067378511925724, 0.9970185921350098, 0.9859452541980706, 1.0144884057647041, 0.004305843555764133, 0.16792789867480118, 0.17653958578632944, 0.6415706898088558, 0.008611687111528266, 1.0467193230410385, 1.0087172445458794, 0.06398329120221838, 0.32631478513131373, 0.6078412664210746, 0.21524511258920853, 0.7246585457170021, 0.060986115233609084, 0.5065669992644102, 0.0464740366297624, 0.2788442197785744, 0.0697110549446436, 0.09759547692250103, 0.028854244095311798, 0.8483147764021668, 0.04039594173343652, 0.08079188346687304, 0.9057993797009638, 0.040558181180640175, 0.05407757490752023, 0.9840593509374157, 0.09570977737535391, 0.028712933212606175, 0.11724447728480854, 0.20099053248824322, 0.05742586642521235, 0.016749211040686936, 0.059818610859596194, 0.42351576488594106, 1.0085896004795276, 1.0093405429603173, 0.03781088714125631, 0.05115590613228794, 0.03781088714125631, 0.2780212289798258, 0.027802122897982576, 0.16458856755605686, 0.3870055507399175, 0.008896679327354424, 0.008896679327354424, 0.9478211738603441, 0.9657894659280928, 0.9944671163559369, 0.07486567367844268, 0.9193504727712762, 0.0029946269471377075, 0.9944671163559369, 1.0045441236851302, 0.37408826786430815, 0.5459666612073686, 0.07077345608243668, 0.9348789339206628, 0.058429933370041424, 0.9987634524787952, 0.9910936276935898, 0.9976763140470931, 0.08456944500687613, 0.9114706850741094, 0.999951770515259, 0.2042788484742199, 0.11392474241831495, 0.3103467121050648, 0.11785318181204994, 0.14142381817445993, 0.07071190908722996, 0.04321283333108498, 0.9693310993241301, 0.027607531309864465, 1.0082058400980605, 0.28582558057703267, 0.0857476741731098, 0.44779340957068453, 0.003175839784189252, 0.03493423762608177, 0.025406718273514015, 0.038110077410271025, 0.07622015482054205, 1.0063112220914376, 1.0063112220914376, 1.0030975152616408, 0.14695663103992004, 0.23792978358844197, 0.25892358802271626, 0.25892358802271626, 0.10496902217137145, 0.9443185104410406, 0.9882348805319314, 0.051735531654575014, 0.09312395697823503, 0.07501652089913377, 0.046561978489117514, 0.12933882913643754, 0.09312395697823503, 0.3207602962583651, 0.0025867765827287505, 0.04138842532366001, 0.08536362723004877, 0.06208263798549002, 0.9996488070906693, 0.9933691969904922, 0.561441051860306, 0.16041172910294457, 0.11105427399434624, 0.17275109288009416, 0.9775053587454239, 1.0233674486839521, 0.1625731251101002, 0.829122938061511, 0.04771969838410828, 0.4598443662468616, 0.07374862477544007, 0.17786433034076723, 0.13448278635521424, 0.09110124236966126, 0.013014463195665895, 1.0000848770000326, 0.9849415951876802, 1.0137906415130322, 0.05640107778643053, 0.21307073830429313, 0.13160251483500457, 0.05326768457607328, 0.12220233520393282, 0.32273950066679696, 0.10026858273143206, 0.9736506079016217, 1.0020757624019851, 0.9926820773821013, 0.2772189186446944, 0.04224288284109629, 0.03960270266352777, 0.2877796393549685, 0.0290419819532537, 0.23761621598116664, 0.0290419819532537, 0.0290419819532537, 0.026401801775685182, 0.07310165897472576, 0.7894979169270382, 0.043860995384835455, 0.08772199076967091, 0.9977861335321636, 0.20580374274109142, 0.1152500959350112, 0.5844826293846996, 0.024696449128930972, 0.0576250479675056, 0.008232149709643657, 0.9949747393565611, 0.025039054980001422, 0.9765231442200554, 0.1467162868985426, 0.8509544640115471, 1.0001956623772281, 0.9465123028344368, 0.04981643699128615, 0.13835611236884715, 0.43691403905951737, 0.07281900650991956, 0.1784065659493029, 0.03640950325495978, 0.13471516204335118, 1.0137906415130322, 0.9806207211227753, 0.015565408271790085, 0.6869531759974473, 0.3153227693103037, 1.0014208568251541, 0.1266203359687192, 0.04808367188685539, 0.16829285160399385, 0.6555407267241284, 0.9835624710343266, 0.9960294862205077, 0.5462360335977454, 0.3354080908056331, 0.08624779477859137, 0.03354080908056331, 0.9912203468364766, 0.3228999995965313, 0.15911014472872556, 0.04679710139080164, 0.2059072461195272, 0.06551594194712229, 0.20122753598044704, 0.034645349184341055, 0.9700697771615495, 0.8668135515515799, 0.1256251523987797, 0.01256251523987797, 0.888317613759805, 0.11280223666791174, 1.0064398528031726, 0.14494241212282857, 0.8535497602788793, 0.9730743570714403, 0.009137748320607634, 0.5574026475570657, 0.43861191938916644, 0.9878877852242627, 1.0045441236851302, 0.03604677100131925, 0.9732628170356197, 0.9966053804201599, 0.9929666132062225, 0.9712273766315999, 0.16120247938757584, 0.05607042761306986, 0.784985986582978, 1.0115625679147204, 0.9934819031996981, 0.9942928812938695, 0.8860173084020924, 0.057963749147800434, 0.04968321355525751, 0.00828053559254292, 1.0025207266436151, 1.0013404850590966, 0.1341119776046133, 0.02235199626743555, 0.8493758581625509, 1.0131194688426064, 0.8754670699457301, 0.12121851737710108, 1.0467193230410385, 0.044523341182089525, 0.42853715887761173, 0.4897567530029848, 0.03895792353432834, 1.0029341133068508, 0.7003487590676768, 0.2971176553620447, 0.9978009537922194, 0.9963491560043387, 0.9801376516015227, 0.07941277427094884, 0.7279504308170309, 0.11911916140642324, 0.07500095347811835, 0.9712273766315999, 0.9443185104410406, 0.1921702451798713, 0.8071150297554595, 1.0144884057647041, 0.05052971991562302, 0.4241183048655572, 0.5260061007609937, 0.9937285571725611, 0.002305689900737854, 0.013834139404427124, 0.004611379801475708, 0.9776125179128501, 0.002305689900737854, 1.003096646089251, 0.9987574564356936, 0.0038846570104576284, 0.010359085361220342, 0.038069638702484754, 0.031336233217691536, 0.0696648490542068, 0.05309031247625425, 0.21443306697726108, 0.027710553341264415, 0.07225462039451189, 0.37629377574632894, 0.01450271950570848, 0.06759303198196273, 0.0212361249905017, 0.005256261767362336, 0.9934334740314814, 0.9947003291068112, 0.99907898950124, 1.0233674486839521, 0.9983118930973269, 0.35295234957086935, 0.6353142292275649, 0.09394302244792425, 0.9394302244792425, 0.2721303112350619, 0.7282005550641933, 0.9978181947298983, 0.3637030627211568, 0.6325270656020119, 1.008398566652891, 0.01981531937305848, 0.2179685131036433, 0.7628897958627515, 1.0009386312577369, 0.8449885186263314, 0.0666394730777864, 0.08796410446267804, 0.9981340434442529, 1.0085180500887718, 0.8093122502893993, 0.1827479274847031, 0.9920535935630069, 0.9809790503009068, 0.9780163985660782, 0.9111299088309738, 0.035498567876531446, 0.053247851814797166, 0.9938456250710103, 0.7292852617079018, 0.15492588461152318, 0.0906883226994282, 0.026450760787333223, 0.9922627998058058, 0.9905063700166036, 0.9954098598791723, 1.0035525914471897, 1.0037660944576563, 0.09036146263854047, 0.12840839427582068, 0.2473050556423213, 0.5326570429219227, 0.9847924567777853, 0.9938744984997844, 0.7435864280295609, 0.25797896482658234, 0.9682967016102809, 0.034582025057510034, 0.9983840071821389, 0.9845149891297588, 0.029159043931129586, 0.05467320737086797, 0.17130938309538632, 0.23691723194042788, 0.09112201228477995, 0.051028326879476775, 0.15308498063843032, 0.2077581880092983, 0.9945350654986218, 0.9986994883250062, 1.0035895464717994, 0.3434352036119677, 0.06326437961273089, 0.5964927220628913, 0.9895414532325696, 0.06438582994509245, 0.06828800145691623, 0.037070629362325955, 0.3024182921663433, 0.034144000728458114, 0.03316845785050217, 0.0653613728230484, 0.07999451599238759, 0.11218743096493382, 0.021461943315030815, 0.03902171511823785, 0.0653613728230484, 0.07511680160260786, 0.0019510857559118923, 0.24035788594631363, 0.6193837830155006, 0.14791254519773148, 1.0006807813412497, 0.35104287688733926, 0.6536660466178041, 0.04108207374229454, 0.9566254314277158, 0.9702029246911564, 0.9386412700338148, 0.17384821555561006, 0.043462053888902515, 0.017384821555561005, 0.6693156298890987, 0.08692410777780503, 0.25680343057917615, 0.08346111493823224, 0.6612688337413786, 1.0029623948674828, 0.058255639654936, 0.0983063919177045, 0.7172725632513995, 0.1274342117451725, 1.0006074437129922, 0.9970365363945585, 1.0027700805485085, 0.9997969255278972, 1.003215144100378, 0.9426004497153506, 0.04833848460078721, 0.3202173218392501, 0.16337618461186232, 0.11763085292054086, 0.36596265353057156, 0.03267523692237246, 0.056358190462640276, 0.0040255850330457345, 0.21738159178446964, 0.3099700475445215, 0.060383775495686014, 0.24556068701578979, 0.012076755099137203, 0.09661404079309763, 0.063139269929424, 0.11365068587296319, 0.037883561957654396, 0.056825342936481595, 0.6503344802730672, 0.07576712391530879, 0.9980670834388463, 0.9780163985660782, 0.7498639539561889, 0.13353741645795145, 0.11299319854134353, 0.9910824343023393, 0.12348500899043945, 0.10976445243594618, 0.13263204669343495, 0.07317630162396412, 0.5030870736647532, 0.0640292639209686, 0.0176011532921817, 0.13200864969136275, 0.0528034598765451, 0.7920518981481764, 0.9736506079016217, 1.008500819709546, 0.9968658980194207, 0.9908746583167494, 0.1331441774588577, 0.39309233344996086, 0.006340198926612272, 0.46283452164269584, 0.9935212111629484, 1.0014808938461792, 1.0031607167356404, 1.0200696950992039, 0.28079791791686154, 0.2246383343334892, 0.4913963563545077, 0.9734668232327717, 0.03605432678639895, 0.987928439033342, 1.0014808938461792, 0.11355402243483727, 0.21111452058307775, 0.06877215443236624, 0.03518575343051296, 0.059176039860408154, 0.10235855543421951, 0.09276244086226143, 0.05437798257442912, 0.23510480701297296, 0.02718899128721456, 0.16196492477571034, 0.8347423046132764, 0.9770966038004981, 0.23680505234239502, 0.5571883584526941, 0.19501592545844296, 0.013929708961317353, 0.9991928191494369, 0.9991250004168527, 1.0063112220914376, 0.02916554850557577, 0.9624631006840004, 0.8683865557954586, 0.003679604049980757, 0.12878614174932648, 1.0117293194791197, 1.0141624044997604, 0.008139222356494545, 0.7406692344410035, 0.14650600241690181, 0.10580989063442908, 0.3566417522478132, 0.08391570641125017, 0.48251531186468843, 0.012587355961687524, 0.06293677980843762, 0.13166790001861173, 0.09404850001329408, 0.06896890000974899, 0.03761940000531763, 0.3824639000540626, 0.28214550003988226, 0.3486262973196213, 0.639148211752639, 0.9845349642762237, 0.9968727016199935, 0.9980910347972048, 0.02665915063621333, 0.24846328392950823, 0.10450387049395625, 0.034123712814353065, 0.5033247640117077, 0.0831765499849856, 1.0045441236851302, 1.0041767138087963, 0.9855920310567584, 0.10274279690075426, 0.8835880533464866, 1.0049938945153685, 0.5530329351561247, 0.3649794823565572, 0.05118614691585863, 0.0300440427549605, 0.8978950443983705, 0.10182314936476367, 0.914109446460829, 0.02770028625638876, 0.05540057251277752, 0.9936846079442484, 0.9584500407293751, 0.05637941416055148, 0.9967189049426546, 0.9913174429927268, 0.05620571724933242, 0.5941747252072285, 0.04282340361853899, 0.22749933172348835, 0.0802938817847606, 0.1388093703953557, 0.1388093703953557, 0.7162563512400354, 0.4178198117618195, 0.04340985057265657, 0.5046395129071326, 0.03255738792949243, 1.0008020984463903, 0.056487003317194306, 0.11297400663438861, 0.8473050497579145, 0.983846912612925, 0.22311325251791825, 0.7808963838127139, 0.9606623909461534, 0.03723497639326176, 0.997497440666152, 0.08130511294647824, 0.04516950719248791, 0.8672545380957679, 1.0024414622793665, 1.008500819709546, 0.2637247324270439, 0.7346617546181938, 0.10696703441599618, 0.8081953711430823, 0.08319658232355259, 0.9980670834388463, 0.972732771346098, 1.0011335616278407, 1.0090194842562819, 0.05468953335578832, 0.17044904562554028, 0.05104356446540244, 0.6899996125055293, 0.03281372001347299, 0.004259606396073805, 0.9860988806910858, 0.01064901599018451, 0.04881933100785184, 0.9763866201570368, 0.9782272554360975, 0.9949393379516852, 0.997079190881357, 0.9889134493058567, 0.24545824946561898, 0.08496631712271427, 0.36346702324716657, 0.042483158561357134, 0.042483158561357134, 0.22185649470930946, 0.995476169693863, 1.0250472147156342, 0.3581323722998843, 0.07483363003281164, 0.05345259288057975, 0.20311985294620305, 0.016035777864173924, 0.2939892608431886, 0.7844239980968457, 0.21508399947816736, 1.0078757277853725, 0.21297399519723836, 0.10648699759861918, 0.6779672180445421, 0.0035495665866206393, 0.5088846862806393, 0.13722733113185778, 0.06861366556592889, 0.034306832782964446, 0.05717805463827408, 0.0800492764935837, 0.10292049834889334, 0.011435610927654815, 0.2894225203479985, 0.010336518583857089, 0.6822102265345679, 0.020673037167714178, 0.06357953595386799, 0.03912586827930337, 0.6993748954925478, 0.20052007493142981, 1.008500819709546, 0.9960103944794544, 0.13019445453303846, 0.06342806759301874, 0.2870954638420848, 0.11016453845103254, 0.22700571559606705, 0.18360756408505421, 0.003338319347000986, 0.7889667240178372, 0.1247378219791047, 0.08419802983589567, 1.0025207266436151, 0.9893664601404354, 0.965983675636278, 1.0005454627612227, 0.9985334219132406, 0.9845799007248796, 1.0054392606212879, 0.9947003291068112, 0.21119242889053244, 0.789192760590937, 1.0467193230410385, 0.029888669722455248, 0.9564374311185679, 0.993827968284985, 0.39066891973332435, 0.5319746992113353, 0.07480894207659403, 1.008500819709546, 0.1471099846264835, 0.5416322161247802, 0.22735179442274722, 0.08024180979626373, 1.0029327049038608, 0.9834044891164316, 0.20087488483126248, 0.00836978686796927, 0.6946923100414494, 0.01673957373593854, 0.03347914747187708, 0.05021872120781562, 0.0340699311795772, 0.04645899706305981, 0.061945329417413084, 0.08672346118437832, 0.10530706000960224, 0.003097266470870654, 0.049556263533930466, 0.5451188988732352, 0.07123712883002505, 0.04296534591763621, 0.9452376101879967, 1.0036613578990197, 0.9884660421119097, 0.2699125681090995, 0.053982513621819904, 0.2699125681090995, 0.4048688521636493, 1.0035597947885118, 1.0016402091006151, 0.17002099269182502, 0.050546781611083114, 0.059737105540370955, 0.0689274294696588, 0.2757097178786352, 0.08271291536359056, 0.28949520377256693, 0.2789284606942762, 0.7252139978051181, 0.19846751186600226, 0.013231167457733483, 0.12569609084846808, 0.4763220284784054, 0.08600258847526764, 0.10584933966186787, 0.9820813539208689, 0.9880823067004972, 1.0008815651837923, 1.017352637546165, 0.07559310588432644, 0.41576208236379536, 0.4913551882481218, 0.007860255663374296, 0.9903922135851614, 0.9946977647777535, 1.0064398528031726, 0.39703218742907065, 0.48061791109834867, 0.048758338807078846, 0.06965476972439835, 0.16402055044334973, 0.8337711314203611, 0.9786846703849228, 0.06878427882835464, 0.7910192065260783, 0.13756855765670928, 1.007141307399571, 0.6642370327272473, 0.1629260646312116, 0.13786051622640982, 0.025065548404801786, 0.012532774202400893, 0.14585737373195237, 0.8569120706752201, 0.9884660421119097, 1.0008133727538329, 0.5222473311428698, 0.4725094900816441, 0.4058902592879833, 0.581776038312776, 1.0053722361032007, 0.9137869868967268, 0.03322861770533552, 0.05221639925124153, 0.9889583509127478, 0.997626515295351, 0.10305194061802807, 0.10305194061802807, 0.7986525397897175, 1.0115625679147204, 0.995893018805234, 1.001899968083595, 1.0115420311736338, 0.006476297036713094, 0.4209593073863511, 0.31733855479894163, 0.25257558443181066, 0.9956745706617857, 0.9848029824432547, 0.15495166789667353, 0.8447365120818654, 0.9702029246911564, 0.992102741979743, 0.9733853319089288, 0.03476376185389032, 0.0061230519147750535, 0.9949959361509462, 1.001873965533191, 0.24338397046260574, 0.03281581624214909, 0.1941602460993821, 0.06563163248429818, 0.42387095979442574, 0.043754421656198784, 0.9872322756413842, 0.9857368523592911, 1.010089992088224, 0.21859861028800456, 0.7772395032462385, 0.12597810332925727, 0.7642671601974941, 0.06718832177560388, 0.04199270110975242, 0.09300813778753834, 0.5090971752581046, 0.08321780749411326, 0.1958066058685018, 0.11748396352110108, 1.0008868120872683, 1.001042912556335, 0.9809815329245998, 0.0046444506276969305, 0.03483337970772698, 0.22525585544330112, 0.22757808075714958, 0.03483337970772698, 0.08127788598469628, 0.3251115439387851, 0.06966675941545396, 0.7408422388918403, 0.21860918524677256, 0.036434864207795424, 1.000143741899444, 0.9051752227990859, 0.09317980234696473, 1.0031626829150602, 0.9816790524920066, 0.027268862569222407, 0.1669984903823791, 0.47006982478003007, 0.13607284401527187, 0.22884978311659357, 0.0019913455503587914, 0.5635507907515379, 0.35246816241350604, 0.08363651311506923, 1.0076636412437472, 0.9887070026288376, 1.0053722361032007, 0.9991024226521344, 0.34530817444604944, 0.1932294447677375, 0.4341698635522004, 0.027433810059617057, 0.9951776899385966, 0.9938808384435437, 1.001294141660612, 0.9970652293506392, 0.9847924567777853, 1.0063112220914376, 0.018491742342786017, 0.8382923195396327, 0.05547522702835805, 0.06163914114262005, 0.024655656457048022, 1.0017294334893811, 0.9891024322967192, 0.12305320251089874, 0.7383192150653924, 0.13567404379406783, 0.9895188375971841, 0.9959792778350075, 0.0167529954341593, 0.7594691263485549, 0.22337327245545732, 0.9770787323877851, 0.11322014921604037, 0.00943501243467003, 0.3774004973868012, 0.5000556590375116, 0.9957944135841436, 0.1840817497775074, 0.13216125625051814, 0.6844065055830404, 0.4567918936151193, 0.542979043353821, 1.0089561188984781, 0.14089749906085078, 0.25925139827196547, 0.0056358999624340315, 0.5974053960180074, 0.10228490855330775, 0.8523742379442313, 0.034094969517769254, 0.05378370979967269, 0.941214921494272, 0.9930652046212491, 0.7288764806503346, 0.07977752524531026, 0.11604003672045128, 0.07252502295028206, 0.1055477242838632, 0.03056846659587295, 0.13496266383838246, 0.02566597667011974, 0.2560830031805206, 0.16207054931019432, 0.035759338281964585, 0.021628632025381806, 0.14822822481394998, 0.043834027571440455, 0.035759338281964585, 0.00028838176033842405, 0.992102741979743, 0.9793837037210049, 0.9928415100458365, 0.9847924567777853, 1.0090194842562819, 0.9985872050901208, 0.31036303199751997, 0.6881962883423269, 0.12141488537098294, 0.19079481986868746, 0.6868613515272749, 0.9957738410223098, 1.0087556917024572, 0.9756687859564599, 0.15075921004064918, 0.16885031524552707, 0.6000216559617837, 0.07839478922113757, 0.9891876473498865, 0.9865853374356056, 0.9905063700166036, 0.9614192979707425, 0.28340129168340666, 0.09635643917235827, 0.09635643917235827, 0.11902854250703081, 0.0170040775010044, 0.39109378252310123, 0.86922948647748, 0.1293891220329073, 0.9934198595489016, 1.0083887595648247, 0.1542820352791411, 0.8292659396253834, 0.01928525440989264, 0.9753501278924499, 1.0006768367126067, 0.9858213293022589, 0.00451727711804429, 0.1140612472306183, 0.00903455423608858, 0.09712145803795222, 0.10389737371501866, 0.5409439348858036, 0.04065549406239861, 0.05985392181408684, 0.030491620546798954, 0.8980524970008695, 0.08980524970008695, 0.9992651882009363, 1.0000653049982002, 0.9925308571768225, 0.6952115402150113, 0.05431340157929776, 0.21725360631719104, 0.043450721263438206, 1.0026588692864722, 0.07638431830642056, 0.9166118196770467, 0.185601510308007, 0.47173717203285115, 0.08764515764544775, 0.10826754767967076, 0.05413377383983538, 0.09537855390828137, 1.0158611736403655, 0.029899645182281206, 0.6637721230466428, 0.05979929036456241, 0.04783943229164993, 0.11361865169266858, 0.08371900651038738, 1.0024414622793665, 0.9762850442151887, 0.8673593394574853, 0.12046657492465074, 0.8763373242240704, 0.10687040539317931, 0.9913174429927268, 1.0020572225566406, 0.9990803838041414, 0.9985357743981557, 0.9989579727859289, 0.9919133778178938, 0.39241524391284877, 0.5979660859624363, 0.3857440185290421, 0.5979032287200152, 0.02885651947957189, 0.8656955843871567, 0.10460488311344811, 0.9946980007906946, 1.012047353113127, 1.0090194842562819, 0.10951018498387254, 0.08517458832078975, 0.6570611099032353, 0.14601357997849673, 0.9848377907315081, 1.0132497697963447, 0.340886115571509, 0.4748056609746018, 0.18261756191330838, 1.0012561509575209, 0.07695227336249744, 0.9234272803499692, 1.0071377678638542, 0.06116122262665168, 0.31803835765858873, 0.13455468977863372, 0.4770575364878831, 0.17022491827522257, 0.037236700872704934, 0.005319528696100705, 0.06915387304930916, 0.4548197035166103, 0.06383434435320846, 0.19948232610377645, 0.972732771346098, 0.9946977647777535, 0.9899837798104666, 0.992102741979743, 0.03717492298319896, 0.966547997563173, 0.21500098723690964, 0.007166699574563655, 0.7740035540528747, 0.9818722380638151, 0.7394944448861112, 0.25920423841368845, 0.9815982093602232, 0.16431535876189246, 0.8380083296856516, 1.0085896004795276, 0.018242578698215037, 0.0510792203550021, 0.3612030582246577, 0.036485157396430073, 0.27363868047322554, 0.13134656662714828, 0.06567328331357414, 0.062024767573931126, 0.04651869938861978, 0.026837711185742182, 0.7926070703522524, 0.005367542237148437, 0.028626891931458325, 0.10019412176010414, 1.0137906415130322, 0.9985948655892447, 0.985857280693241, 0.012221371248263318, 0.8152862375346772, 0.18594247522720708, 0.14981603908536345, 0.2996320781707269, 0.4630677571729416, 0.006809819958425611, 0.07490801954268173, 0.05388385334380757, 0.9160255068447287, 1.0200696950992039, 0.9935699928698231, 0.8884941443592024, 0.0413253090399629, 0.0413253090399629, 0.3699329574578209, 0.05284756535111727, 0.27643341875969035, 0.12195592004103986, 0.048782368016415945, 0.04065197334701329, 0.0853691440287279, 0.9775053587454239, 0.023402695219395916, 0.7319843004733279, 0.05850673804848979, 0.026002994688217686, 0.050705839642024485, 0.1079124279561034, 0.9983258201510448, 0.9926717745555667, 0.9394073997732959, 0.051240403623997954, 0.9987995095644, 0.1406091822783274, 0.09666881281635009, 0.20212569952509563, 0.021970184730988655, 0.180155514794107, 0.004394036946197731, 0.3559169926420162, 0.9926820773821013, 0.9933691969904922, 0.03458557803563201, 0.5101372760255721, 0.2939774133028721, 0.06917115607126402, 0.08646394508908002, 1.0041767138087963, 0.48860378949873084, 0.435984919860406, 0.07516981376903552, 0.020074024009581116, 0.9836271764694746, 0.9890433477812807, 0.9775053587454239, 1.0023560571406145, 0.7771561952042735, 0.21688079866165771, 0.0334303426662159, 0.22286895110810598, 0.011143447555405299, 0.7354675386567497, 1.001976501746635, 0.9997619017465887, 0.9929666132062225, 0.5360410571191614, 0.25877844136787104, 0.0462104359585484, 0.15711548225906455, 0.5371477264524109, 0.15831722463860531, 0.20920490398672847, 0.005654186594235905, 0.08481279891353857, 1.0028459800467633, 0.9882348805319314, 0.8022709623713393, 0.029173489540775976, 0.16045419247426787, 0.9846483715971035, 0.9908746583167494, 0.0843569086121049, 0.915144644943441, 0.9928415100458365, 1.0135123116480327, 0.10087253290109865, 0.8876782895296682, 0.02017450658021973, 0.18352824312746907, 0.13347508591088658, 0.6340066580767113, 0.033368771477721645, 1.0144884057647041, 0.06946519931793703, 0.4843789574061555, 0.07885238841495554, 0.05256825894330369, 0.14456271209408517, 0.09762676660899258, 0.07322007495674443, 0.9949747393565611, 1.0040021040339657, 0.0319045009351436, 0.957135028054308, 0.8463950102647769, 0.1526286084084024, 0.9945279317339941, 0.21418276254207044, 0.06693211329439702, 0.7228668235794877, 1.0022086099211733, 0.01601030545758067, 0.04803091637274201, 0.7524843565062915, 0.17611336003338737, 0.8627096607092368, 0.11502795476123158, 0.9719862909101434, 1.0001106578623755, 0.047263162101020696, 0.05356491704782346, 0.08507369178183725, 0.2646737077657159, 0.03781052968081656, 0.02520701978721104, 0.1291859764094566, 0.07562105936163312, 0.08192281430843588, 0.07247018188823173, 0.13233685388285796, 1.0014547649428909, 1.0025207266436151, 0.8445422599604852, 0.1498381428962151, 0.2456324704674741, 0.19223410732237103, 0.0854373810321649, 0.48058526830592757, 1.0141624044997604, 0.18832451403926706, 0.8071050601682874, 0.9984185528550452, 0.05655640289747671, 0.9331806478083657, 1.0067850564606964, 0.991258600071983, 0.43920873956441964, 0.010214156734056271, 0.5515644636390387, 0.9860381869337136, 0.9920909898463682, 0.9939004479965168, 0.030773110933207688, 0.41287257168720315, 0.2718291465766679, 0.20515407288805126, 0.04615966639981153, 0.033337536844308326, 0.12262319803438357, 0.06751164835600894, 0.23697966361701095, 0.16671243777708328, 0.3630473335062929, 0.03306692980702478, 0.009644521193715562, 1.0000817977021015, 0.7896142626124096, 0.2085773523881837, 0.9856237393537535, 1.0090194842562819, 1.00326949789919, 0.992102741979743, 0.9898338899110439, 0.8958076223746697, 0.10710743311001486, 0.9891024322967192, 0.9947003291068112, 0.8702192095890408, 0.13650497405318288, 0.9718655273752628, 1.0089561188984781, 1.0233674486839521, 0.9796934047155403, 1.0004176522750297, 1.0001170580084493, 0.007596763423067173, 0.48619285907629906, 0.17472555873054496, 0.1899190855766793, 0.015193526846134345, 0.12154821476907476, 0.9980670834388463, 0.6064735226115768, 0.10342959300352474, 0.2867820533279549, 0.9933691969904922, 0.9887196168723033, 0.9917641329741721, 0.20095693893471006, 0.5305263187876346, 0.004019138778694201, 0.12459330213952023, 0.14066985725429704, 0.053068188381448556, 0.24259743260090766, 0.166785734913124, 0.5231007140457071, 0.015162339537556729, 0.9878877852242627, 0.6151798589229862, 0.1615623871918954, 0.22370176688108592, 0.9819785212766033, 0.6939473098166606, 0.29321717316196927, 0.009773905772065643, 0.748039154013863, 0.09424115326158904, 0.017670216236547943, 0.005890072078849315, 0.03534043247309589, 0.10013122534043836, 0.998110942157434, 0.16761100493640022, 0.18357395778748595, 0.04389812034048577, 0.5746663026390865, 0.03192590570217147, 0.9948641163053424, 0.129547663759056, 0.04484342207044245, 0.44843422070442457, 0.0896868441408849, 0.14947807356814152, 0.0896868441408849, 0.04982602452271384, 0.11034721060784662, 0.05359721658095407, 0.04729166168907713, 0.0630555489187695, 0.059902771472831025, 0.6384374328025412, 0.025222219567507802, 0.9890205108737863, 0.9942378457686505, 0.9744891093805377, 0.022662537427454364, 1.001976501746635, 1.0049938945153685, 0.9815697477238148, 0.023370708279138445, 0.7198614884495195, 0.2778412762436742, 0.4179773566421882, 0.5921345885764333, 1.0008424173894532, 0.03926906321714135, 0.13962333588316928, 0.03490583397079232, 0.5235875095618847, 0.17452916985396158, 0.08726458492698079, 0.9994175260755178, 1.0155764672601242, 0.9972135173595985, 0.04525891619530908, 0.31034685391069083, 0.4590547214095635, 0.09698339184709089, 0.08728505266238179, 0.14048357827859628, 0.8429014696715778, 0.965983675636278, 0.27723749567351086, 0.15639038217480097, 0.5402576838765852, 0.028434614940872908, 0.11827606525633733, 0.06702310364525782, 0.2996326986493879, 0.5125296161107951, 0.003942535508544578, 0.049178685836471496, 0.4122331018645405, 0.024589342918235748, 0.03688401437735362, 0.47804575496923024, 1.0042052213903299, 1.0135123116480327, 1.0040512310605347, 0.9816938309267614, 0.9871745752786555, 0.9990846543133535, 0.9928006415315245, 0.8643041716170548, 0.12060058208610065, 1.0069512310895508, 0.08991191120727568, 0.04995106178181982, 0.6193931660945659, 0.054946167960001804, 0.18481892859273336, 0.9773409034608559, 0.9793837037210049, 0.9987995095644, 0.9904259936231669, 0.9863499218599503, 1.010549778863525, 0.9092171194129401, 0.09570706520136213, 0.15743778942005524, 0.712613152111829, 0.12429299164741203, 0.9996452387854406, 0.9919799615381816, 0.09809142878182779, 0.007006530627273414, 0.46243102140004533, 0.4273983682636782, 0.2619516278516132, 0.008731720928387107, 0.09604893021225817, 0.6374156277722588, 0.9813445024208853, 0.9386412700338148, 1.0063112220914376], "Term": ["abercrombie", "acne", "adapter", "adjustable", "adjustable", "adjustable", "adjustable", "adjustable straps", "adjustable straps", "aging", "ago", "alcohol", "alex", "alex ani", "also", "also", "also", "also", "also", "also", "also", "also", "also", "also", "also", "american", "american apparel", "american eagle", "anastasia", "android", "ani", "anymore", "anymore", "anymore", "appearance", "apple", "apple", "apple", "apple iphone", "approximately", "around", "around", "around", "around", "around", "around", "around", "around business", "arrive", "arrive", "ask", "ask", "ask", "ask", "ask bundle", "ask lower", "ask questions", "authentic", "authentic", "authentic", "authentic", "authentic", "authentic", "authentic", "authentic", "available", "available", "available", "available", "available", "available", "available", "available phone", "baby", "baby", "baby", "baby", "baby blue", "back", "back", "back", "back", "back", "back", "back", "background", "background", "backpack", "backpack", "backpack", "bag", "bag", "bag", "bag", "barely", "barely", "barely used", "barely worn", "baseball", "basketball", "bath", "bath body", "batteries", "battery", "bead", "beads", "beautiful", "beautiful", "beautiful", "beautiful", "beautiful", "beautiful", "beautiful", "beautiful", "beauty", "beauty", "bed", "belly", "belly", "belly button", "belly button", "benefits", "bikini", "bikini", "binding", "black", "black", "black", "black", "black", "black", "black", "black", "black", "black", "black", "black", "black background", "black color", "black gray", "black grey", "black lace", "black white", "black white", "blue", "blue", "blue", "blue", "blue", "blue black", "blue color", "blue pink", "blue white", "bluetooth", "blush", "body", "body", "body", "body", "body", "body", "body jewelry", "body lotion", "body works", "bomb", "book", "book", "book", "books", "boots", "boots", "boots", "borax", "bottle", "bottle", "bottle", "bottom", "bottom", "bottom", "bottom", "bought", "bought", "bought", "bought", "box", "box", "box", "box", "box", "box authentic", "box free", "box never", "box price", "boxes", "boxes", "boxes", "boxes", "boxes", "boyshorts", "bra", "bra", "bracelet", "bracelets", "bradley", "brand", "brand", "brand", "brand", "brand", "brand", "brand", "brand", "brand", "brand", "brand new", "brand new", "brand new", "brand new", "brand new", "brand new", "brand new", "brand new", "brands", "brands", "brands", "brands", "brandy", "brandy melville", "bright", "bright", "bright", "bright", "bronzer", "brown", "brown", "brown", "brown", "brown", "brown", "brown", "brush", "brush", "brushes", "bubble wrap", "bundle", "bundle", "bundle", "bundle", "bundle", "bundle discounts", "bundle items", "bundle save", "bundle save", "bundles", "bundles", "business", "business", "business day", "business day", "bust", "butterfly", "button", "button", "button", "button", "button", "button ring", "buy", "buy", "buy", "buy", "buy", "buyer", "cable", "calvin", "calvin klein", "cancel", "candle", "card", "card", "card", "cards", "cards", "care", "care", "care", "carly", "carrier", "case", "case", "case", "cases", "cases", "cases cases", "chain", "chain", "changed", "charge", "charge", "charger", "charging", "charlotte", "charlotte russe", "charm", "charm", "charms", "chart", "check", "check", "check", "check closet", "check items", "check listings", "choice", "choice", "choice color", "choose", "choose", "choose color", "class", "class", "clean", "clean", "clean", "clean", "clean", "clean", "cleansing", "clearance", "click", "closed", "closet", "closet", "closet", "closet", "closure", "closure", "closure", "closure", "coach", "coach", "coach", "collection", "collection", "collection", "collection", "color", "color", "color", "color", "color", "color", "color", "color", "color pink", "color size", "color size", "colored", "colored", "colored", "colored", "colors", "colors", "colors", "colors", "colors", "colors", "colors brands", "combine", "combine", "combine", "come", "come", "come", "come", "come", "come", "come", "come", "come", "come new", "comes", "comes", "comes", "comes", "comes", "comes", "comes", "comes", "comes box", "comes box", "comes original", "comes smoke", "comfortable", "comfortable", "comfortable", "comfortable", "comfortable", "comfy", "comfy", "compatible", "compatible iphone", "concealer", "condition", "condition", "condition", "condition flaws", "condition minor", "condition never", "condition price", "condition rips", "condition size", "condition size", "condition smoke", "condition stains", "condition worn", "conditioner", "considered", "contact leaving", "contour", "cord", "cord", "corset", "cosmetics", "costume", "costume", "costume", "costume", "cotton", "cotton", "cotton", "cotton", "cotton", "cotton", "cotton", "cotton polyester", "cotton spandex", "could fit", "couple", "couple", "couple", "couple times", "cream", "cream", "cream", "cream", "cream", "crop", "crop top", "crossbody", "cup", "cup", "cup", "cup", "curvy", "customer", "cute", "cute", "cute", "cute", "cute", "cute", "cute comfy", "dark", "dark", "dark", "dark", "dark blue", "dark brown", "dark grey", "day", "day", "day", "day", "day", "day next", "day shipping", "day shipping", "day shipping", "days", "days", "days", "days", "days", "days arrive", "days take", "deals", "decided", "decor", "delivery", "delivery", "denim", "description", "description yet", "design", "design", "design", "design", "design", "design", "design", "device", "devices", "diamond", "diamond", "diamond", "disc", "discount", "discounts", "discounts", "disney", "disney", "disney", "disney", "dont ask", "dot", "dots", "drawer", "dress", "dress", "dress", "dress", "dri", "dri fit", "dry", "dry", "dry", "dry", "dry", "dry", "dunn", "dust bag", "eagle", "earrings", "easy", "easy", "easy", "easy", "easy clean", "eau", "eau parfum", "edge", "edge", "edge edge", "edge plus", "edge samsung", "edition", "edition", "elastic", "else", "else", "essential", "essential", "essential oils", "excellent", "excellent", "excellent", "excellent", "excellent", "excellent condition", "excellent condition", "exclusive", "exclusive", "exterior", "eye", "eye", "eye", "eye shadow", "eyeliner", "eyeshadow", "eyeshadow palette", "fabric", "fabric", "face", "face", "face", "face", "faced", "facial", "facial", "factory", "factory", "factory sealed", "fading", "fall", "fall", "fall", "fall", "fast", "fast", "fast", "fast", "fast free", "fast free", "fast shipping", "fast shipping", "fast shipping", "faux", "faux", "faux", "features", "features", "features", "features", "features", "features", "fedex", "feel", "feel", "feel", "feel", "feel free", "fiber", "final", "final", "find", "find", "find", "find", "find", "firm", "firm", "firm", "firm", "firm bundle", "firm free", "firm please", "firm price", "firm unless", "first", "first", "first", "first", "first", "first", "first", "first", "first class", "fit", "fit", "fit", "fit", "fit", "fit", "fit", "fit like", "fit medium", "fit small", "fits", "fits", "fits", "fits", "fits", "fits like", "flattering", "flaw", "flaw", "flaws", "flaws", "flaws", "flaws", "fleece", "fleece", "fleece", "floam", "flowy", "forever", "forever", "forever", "forever", "forever", "forget", "foundation", "fragrance", "fragrance", "fragrance", "france", "free", "free", "free", "free", "free", "free", "free", "free ask", "free fast", "free fast", "free home", "free pet", "free ship", "free ship", "free shipping", "free shipping", "free shipping", "free shipping", "free shipping", "free smoke", "fresh", "fresh", "fresh", "fresh", "fresh", "friend", "friendly home", "front", "front", "front", "front", "front", "front", "front back", "full", "full", "full", "full", "full", "full", "full", "full", "full", "funko", "funko pop", "galaxy", "galaxy", "galaxy edge", "galaxy note", "game", "games", "gap", "gauge", "gauge", "gel", "gel", "gel", "generation", "gently", "gently", "gently", "gently used", "gently used", "gently worn", "get", "get", "get", "get", "get", "get", "get", "get free", "get one", "get order", "get rid", "gift", "gift", "gift", "gift", "gift", "girl", "girl", "girl", "given", "glass", "glass", "glaze", "gloss", "gloves", "gloves", "glow", "glow", "gold", "gold", "gold", "gold", "gold", "gone", "gone", "gone asap", "good", "good", "good", "good", "good", "good condition", "good shape", "good used", "got", "got", "gray", "gray", "gray", "gray", "great", "great", "great", "great", "great", "great", "great", "great", "great", "great", "great condition", "great condition", "great gift", "great halloween", "green", "green", "green", "green", "green color", "grey", "grey", "grey", "grey", "grey white", "hair", "hair", "halloween", "halloween", "halloween", "halloween costume", "hand", "hand", "hand", "hand", "hand", "handbag", "handful", "handful times", "handmade", "handmade", "handmade", "handmade", "happy bundle", "hard", "hard", "hard", "hard", "hard find", "hardware", "head", "head", "head", "headphones", "headset", "healthy", "heart", "heart", "heart", "heart", "heat", "heat", "heel", "heel", "hello kitty", "helps", "helps", "hem", "hem", "henna", "henna", "high", "high", "high", "high", "high", "high", "high", "high", "hits", "holds", "holds", "holds", "holds", "holds", "holes", "holes", "holes stains", "hollister", "hollister", "home", "home", "home", "home decor", "home free", "hoodie", "hoodie", "hoodie", "hoodie size", "hot", "hot", "hot", "hot", "hot pink", "hot pink", "hot topic", "hrs", "htf", "htf", "human", "human hair", "hydrating", "ideal", "inches", "inches", "inches", "inches", "included", "included", "included", "included", "included", "included", "included", "included", "included price", "includes", "includes", "includes", "includes", "includes", "includes", "includes", "includes", "includes", "includes", "inside", "inside", "inside", "inside", "inside", "inside", "inside", "instead", "interior", "ipad", "ipad mini", "iphone", "iphone", "iphone iphone", "iphone iphone", "iphone plus", "iphone plus", "ipod", "ipod", "irma", "item", "item", "item", "item", "item", "item come", "items", "items", "items", "items", "items shipped", "jacket", "jacket", "jacket", "jacket size", "japan", "jeans", "jeans size", "jersey", "jewelry", "jewelry", "jewelry", "jordan", "jordan", "kate spade", "keep mind", "kendra", "kendra scott", "kit", "kit", "kitty", "klein", "knit", "know", "know", "know", "know", "know questions", "know want", "kors", "kors", "kylie", "lace", "lace", "large", "large", "large", "large", "large", "large", "large", "large fits", "lashes", "lauren", "least", "leather", "leather", "leather", "leather", "leather", "leaving bad", "led", "left", "left", "left", "leggings", "leggings", "leggings", "length", "length", "length", "length", "length", "let", "let", "let", "let", "let know", "let know", "let know", "life left", "light", "light", "light", "light", "light", "light", "light", "light", "light blue", "light pink", "like", "like", "like", "like", "like", "like", "like", "like", "like", "like brand", "like bundle", "like medium", "like new", "like new", "like new", "like small", "lime green", "limited", "limited", "limited", "lines", "lines", "lip", "lips", "lipstick", "liquid", "liquid", "liquid lipstick", "listing", "listing", "listing", "listing", "listing", "listing", "listing", "listings", "listings", "listings bundle", "little", "little", "little", "little", "little", "little", "little", "little", "littlest", "littlest pet", "llr", "logo", "logo", "logo", "logo", "logo", "logo waistband", "lol", "long", "long", "long", "long", "long", "long", "long", "long", "long", "long", "long", "long sleeve", "long take", "looking", "looking", "looking", "looking", "loose fit", "loss gain", "lotion", "lotion", "love", "love", "love", "love", "love", "love", "love", "lularoe", "lularoe leggings", "machine wash", "made", "made", "made", "made", "made", "made", "made", "made china", "mail", "mailers", "make", "make", "make", "make", "make", "make", "make", "make", "make", "make sure", "make sure", "make sure", "make sure", "makeup", "many", "many", "many", "many", "many", "many", "mario", "mark", "mark", "marks", "marks", "mascara", "mask", "mask", "material", "material", "material", "material", "material", "material", "maternity", "matte", "matte", "measures", "measures", "measures inches", "medium", "medium", "medium", "medium", "medium fits", "melville", "men", "men", "men", "men", "mesh", "message", "message", "message", "message", "message", "message", "metal", "metal", "michael", "michael", "michael", "michael kors", "michael kors", "minimal", "minor", "minor", "minor scratches", "mint", "mint", "mint", "mint condition", "mint green", "minutes", "minutes", "mist", "mobile", "money back", "months", "months", "months", "mother", "movie", "moving", "much", "much", "much", "much", "multiple items", "music", "nail", "nail", "nail", "nail polish", "natural", "natural", "navel", "navy", "navy", "navy", "navy", "navy blue", "neck", "neck", "necklace", "necklaces", "neckline", "need", "need", "need", "need", "need gone", "needle", "negotiable", "negotiable", "negotiate price", "never", "never", "never", "never opened", "never used", "never used", "never used", "never used", "never used", "never wore", "never worn", "new", "new", "new", "new", "new", "new", "new", "new", "new", "new", "new", "new", "new", "new box", "new box", "new clearance", "new condition", "new factory", "new free", "new full", "new full", "new item", "new item", "new never", "new never", "new original", "new package", "new package", "new price", "new sealed", "new sealed", "new sealed", "new still", "new tags", "new tags", "new tags", "new unopened", "new victoria", "new without", "new without", "new worn", "new york", "newborn", "next", "next", "next", "next day", "nike", "nike", "nike", "nike", "nike air", "nike dri", "nintendo", "north", "north face", "note", "note", "note", "note", "note edge", "note note", "nothing", "nothing", "nothing wrong", "nothing wrong", "noticeable", "nursing", "nwt", "nwt", "nwt", "nwt", "nwt", "nwt", "nwt", "nwt", "nyx", "oil", "oils", "old", "old", "old", "old navy", "one", "one", "one", "one", "one", "one", "one", "one", "one", "one", "one", "one", "one", "one", "one size", "one size", "one size", "onesie", "online", "online", "opened", "opened", "opened used", "opi", "orange", "orange", "orange", "orange", "orange", "order", "order", "order", "order ship", "original", "original", "original", "original", "original box", "original packaging", "original price", "outfitters", "oversized", "owned", "owned", "pack", "pack", "pack", "pack", "pack", "package", "package", "package", "package", "package", "package", "package", "package", "packaging", "packaging", "packaging", "packaging", "packaging", "packaging", "packets", "pacsun", "page", "page", "page", "pages", "pair", "pair", "pair", "pair", "pair", "pair", "pairs", "pairs", "pairs", "pairs", "paisley", "pale pink", "palette", "pandora", "pants", "pants", "pants", "pants", "pants size", "panty", "parfum", "patches", "pcs", "pcs", "pcs", "peel", "peel", "pendant", "per llr", "perfect", "perfect", "perfect", "perfect", "perfect", "perfect", "perfect", "perfect", "perfect", "perfect", "perfect condition", "perfect condition", "perfect fall", "perfume", "perfume", "perfume", "perfume", "pet", "pet free", "pet shop", "pet smoke", "pet smoke", "phone", "phone", "phone", "phone case", "phone size", "pic", "pic", "pic", "pic", "picture", "picture", "picture", "picture", "picture", "piece", "piece", "piece", "piece", "piece", "piece", "piercing", "piercing", "piling", "pilling", "pillow", "pink", "pink", "pink", "pink", "pink", "pink", "pink color", "pink victoria", "plated", "played", "played", "playstation", "please", "please", "please", "please", "please ask", "please ask", "please check", "please check", "please check", "please feel", "please let", "please let", "please look", "plug", "plus", "plus", "plus", "plus", "plus", "pocket", "pocket", "pocket", "pockets", "pockets", "pockets", "pockets", "pokemon", "polish", "polish", "polish", "polka", "polo", "polo", "polyester", "polyester", "polyester spandex", "pop", "pop", "pop", "possibly", "postage", "powder", "powder", "power", "power", "power", "powerful", "practically", "pre", "pre owned", "price", "price", "price", "price", "price", "price firm", "price firm", "price firm", "price free", "price free", "price includes", "price negotiable", "prices", "primer", "print", "print", "print", "print", "print", "print", "priority", "problem", "product", "product", "product", "product", "product", "product", "protector", "protector", "pumpkin", "purchase", "purchase", "purchase", "purchase", "purchased", "purchased", "purchased", "purchased", "purchased", "purchased", "purchased", "purchased", "purchasing", "purchasing", "purchasing", "purchasing", "purple", "purple", "purple", "purple", "purple pink", "purse", "quality", "quality", "quality", "quality", "quality", "quality", "quality", "questions", "questions", "questions", "questions feel", "questions please", "racerback", "rae", "rae dunn", "ralph", "ralph lauren", "rambler", "rare", "rare", "rare hard", "ratings", "ratings", "rayon", "read", "read", "read", "reading", "really", "really", "really", "really", "reasonable", "reasonable offers", "receive", "receive", "receive", "receive", "receive", "receive", "red", "red", "red", "red", "red", "red", "red", "red", "red", "red white", "red white", "remove", "request", "reserved", "reserved", "reserved", "reserved", "responsible", "results", "retail", "retail", "retail", "retail", "retail", "retail", "retail", "retail price", "retail price", "retails", "retails", "retails", "retails", "retails", "retails", "rhinestone", "rid", "ring", "ring size", "rings", "rings", "rings", "rips", "rips", "rips stains", "rips tears", "rose", "rose", "rose", "rose", "rose gold", "rose gold", "royal blue", "runs", "runs", "runs", "russe", "safe", "safe", "safe", "safe", "safe", "sales", "sales", "sales final", "salt", "samsung", "samsung", "samsung galaxy", "samsung galaxy", "samsung note", "save", "save", "save", "save money", "save shipping", "says", "says", "says", "scarf", "scent", "scented", "scott", "scratches", "scratches", "scratches", "scratches", "screen", "screen protector", "sealed", "sealed", "sealed box", "sealed packaging", "seat", "seat", "secret", "secret", "secret pink", "see", "see", "see", "see", "see", "see", "see listings", "see pic", "see pics", "self", "self", "sell", "sell", "sell", "sell", "selling", "selling", "selling", "selling", "selling", "sephora", "serum", "service", "set", "set", "set", "set", "set", "set", "set", "set", "sexy", "sexy", "sexy", "shade", "shades", "shades", "shadow", "shampoo", "shampoo", "shape", "shape", "shape", "shape", "ship", "ship", "ship", "ship", "ship day", "ship next", "ship today", "shipped", "shipping", "shipping", "shipping", "shipping", "shipping bundle", "shipping firm", "shipping included", "shipping price", "shipping takes", "shipping\u2022", "ships", "ships", "ships", "ships", "ships", "ships next", "ships within", "shirt", "shirt", "shirt", "shirt size", "shock", "shoes", "shoes", "shoes", "shoes size", "short", "short", "short", "short", "short sleeve", "shorts", "shorts", "shorts", "shoulder", "shoulder", "shoulder strap", "shown", "shown", "shown", "shown", "shows", "shows", "shows", "signs", "signs", "signs wear", "silver", "silver", "silver", "silver", "size", "size", "size", "size", "size", "size", "size", "size", "size", "size", "size", "size", "size box", "size chart", "size excellent", "size free", "size good", "size great", "size large", "size large", "size medium", "size medium", "size medium", "size months", "size never", "size price", "size small", "size small", "size small", "size small", "size women", "size worn", "size xxl", "size youth", "sizes", "sizes", "sizes", "sizes", "sizes", "sizes", "skin", "skin", "skinny", "skinny jeans", "sleeve", "sleeve", "sleeve", "sleeve shirt", "slime", "slots", "small", "small", "small", "small", "small", "small", "small", "small", "small", "small medium", "small medium", "smoke", "smoke free", "smoke pet", "smooth", "smooth", "smooth", "smooth", "soap", "socks", "socks", "soft", "soft", "soft", "soft", "soft", "soft", "soft material", "sold", "sold", "sold", "sold", "sold", "sold", "sold separately", "soles", "someone", "someone", "something", "something", "sound", "spade", "spandex", "spf", "spinner", "stain", "stainless", "stainless", "stainless steel", "stainless steel", "stains", "stains", "stains", "stains holes", "stains rips", "stains tears", "star", "star", "star", "star", "star wars", "stars", "steel", "steel", "steel", "step", "sterling", "sterling", "sterling silver", "stickers", "stickers", "stickers", "stickers", "still", "still", "still", "still", "still", "still", "still", "still box", "still good", "still original", "still sealed", "stone", "stone", "strap", "strap", "strap", "strapless", "straps", "straps", "strawberry", "stretchy", "stretchy", "stripes", "style", "style", "style", "style", "style", "style", "style", "style", "super", "super", "super", "super", "super", "super", "super comfortable", "super comfy", "super cute", "super cute", "super soft", "super soft", "sure", "sure", "sure", "sure", "sure", "surgical", "surgical", "surgical steel", "swatched", "sweatshirt", "sweatshirt", "sweatshirt", "tag", "tag", "tag", "tag", "tag", "tag", "tag", "tag says", "tags", "tags", "tags", "tags", "tags", "tags", "tags attached", "tags never", "tags size", "tags size", "tags victoria", "take", "take", "take", "take", "take", "take", "take", "take around", "take get", "tall", "tall", "tall", "tall", "tall", "tall curvy", "tank", "tank", "tank", "tarte", "tarte", "teal", "team", "tears", "technology", "technology", "tee", "tee", "tee", "tee", "teeth", "tempered", "tempered glass", "thank", "thank", "thank", "thank", "thanks", "thanks", "thanks", "thanks", "thanks", "though", "thought", "tie", "tie", "tie", "ties", "tiffany", "times", "times", "times great", "times still", "tiny", "tiny", "tiny", "today", "today", "today", "today", "ton", "top", "top", "top", "top", "top", "top", "top", "topic", "tote", "toys", "toys", "tracking", "tracking", "tracking number", "trades", "trades", "trades", "trainer", "tried", "tried", "tried", "tried", "trying", "trying", "tumbler", "twice", "two", "two", "two", "two", "two", "two", "two", "two", "two", "two", "two", "types", "understand", "unicorn", "unicorn", "unless", "unless", "unless", "unless", "unless bundled", "unopened", "unopened", "update", "upon", "upon", "upon request", "upper", "urban", "urban", "urban", "urban decay", "urban outfitters", "usb", "use", "use", "use", "use", "use", "use", "used", "used", "used", "used", "used", "used", "used", "used comes", "used condition", "used condition", "used couple", "used good", "used great", "used price", "used twice", "usps", "usps", "usps first", "vacuum", "vanilla", "vanilla", "velcro", "vera bradley", "versace", "via usps", "victoria", "victoria secret", "vintage", "vintage", "vintage", "vintage", "vintage", "vintage", "vitamin", "waist", "waist", "waist", "waist trainer", "waistband", "wallet", "want", "want", "want", "want", "want", "warm", "warm", "warm", "warm", "warm", "wars", "washed", "washed", "washed", "washed per", "watch", "watch", "watch", "water", "water", "water", "water", "water", "water", "ways", "wear", "wear", "wear", "wear", "wear", "wedding", "well", "well", "well", "well", "well", "well", "well", "white", "white", "white", "white", "white", "white", "white", "white black", "white blue", "width", "width", "wig", "wii", "wireless", "wireless", "within", "within", "within days", "within days", "within hours", "without", "without", "without", "without", "without", "without", "without box", "without tag", "without tags", "women", "women", "women", "women", "women", "women size", "women size", "wool", "wore", "wore", "wore", "wore", "works", "works", "works", "works", "works", "worn", "worn", "worn", "worn", "worn", "worn couple", "worn excellent", "worn great", "worn handful", "worn tags", "worn times", "worn twice", "worn washed", "worn washed", "worth", "would", "would", "would", "would", "would", "wristlet", "write message", "wrong size", "xbox", "xxs", "yeah", "years", "years", "yellow", "yellow", "yellow", "yet", "yeti", "zip", "zip", "zip", "zip", "zipper", "zipper", "zipper", "zipper", "zippered", "\u2764\ufe0fplease", "\u2764\ufe0fplease contact"]}, "R": 30, "lambda.step": 0.01, "plot.opts": {"xlab": "PC1", "ylab": "PC2"}, "topic.order": [2, 20, 19, 13, 3, 6, 12, 14, 18, 1, 4, 15, 8, 5, 16, 17, 10, 11, 9, 7]};

function LDAvis_load_lib(url, callback){
  var s = document.createElement('script');
  s.src = url;
  s.async = true;
  s.onreadystatechange = s.onload = callback;
  s.onerror = function(){console.warn("failed to load library " + url);};
  document.getElementsByTagName("head")[0].appendChild(s);
}

if(typeof(LDAvis) !== "undefined"){
   // already loaded: just create the visualization
   !function(LDAvis){
       new LDAvis("#" + "ldavis_el499021163441769524580143789", ldavis_el499021163441769524580143789_data);
   }(LDAvis);
}else if(typeof define === "function" && define.amd){
   // require.js is available: use it to load d3/LDAvis
   require.config({paths: {d3: "https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.5/d3.min"}});
   require(["d3"], function(d3){
      window.d3 = d3;
      LDAvis_load_lib("https://cdn.rawgit.com/bmabey/pyLDAvis/files/ldavis.v1.0.0.js", function(){
        new LDAvis("#" + "ldavis_el499021163441769524580143789", ldavis_el499021163441769524580143789_data);
      });
    });
}else{
    // require.js not available: dynamically load d3 & LDAvis
    LDAvis_load_lib("https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.5/d3.min.js", function(){
         LDAvis_load_lib("https://cdn.rawgit.com/bmabey/pyLDAvis/files/ldavis.v1.0.0.js", function(){
                 new LDAvis("#" + "ldavis_el499021163441769524580143789", ldavis_el499021163441769524580143789_data);
            })
         });
}
</script>
"""

In [ ]:
h = IPython.display.display(HTML(html_string))